# c_buiding & canopy_data_zonalst – Zonal Statistics for Built-up Areas and Canopy Data

Below is the detailed workflow:

### 07_00 Auxiliary Code: Implementing Multi-Processing

#### 07 Data Preprocessing – Converting Raster Data to a Multi-Polygon Shapefile
> 07_00 Obtain the input raster filenames.
> 07_01 Raster processing: Process each 1000m raster grid cell into distinct data to facilitate later conversion into a shapefile.
> 07_02 Raster to unique value
> 07_03 Raster to Shapefile


#### 08 Zonal Statistics – Calculating Building Height-Related Indicators (Mean = Building Height)
> 08_00 (Optional, Auxiliary) Clip the source building dataset into 500,000m raster tiles for easier processing.
> 08_01 Compute the latitude and longitude boundaries of the urban data. Identify land parcels that intersect with the urban data. If multiple parcels contain urban data, merge the files.Then Remove NoData values from the building dataset to prevent calculation errors.
> 08_02 Perform zonal statistics to compute *MEAN* (=building height).

#### **09 Zonal Statistics – Calculating Building Density and Floor Area Ratio (FAR)**
> **09_01** Perform zonal statistics to compute building density and floor area ratio (FAR).

#### 10 Zonal Statistics – Calculating Canopy-Related Indicators (Canopy Density, Canopy Coverage Ratio)
> 10_00 (Optional, Auxiliary) Clip the source canopy dataset into 100,000m raster tiles for easier processing.
> 10_01 Compute the latitude and longitude boundaries of the canopy dataset. Identify land parcels that intersect with the dataset. If multiple parcels contain relevant data, merge the files.
> 10_02 Data preprocessing: Remove NoData values from the canopy dataset to prevent calculation errors.
> 10_03 Perform zonal statistics to compute canopy density and canopy coverage ratio.
> 10_04 Converged into one file

#### 11 Zonal Statistics – Calculating Distance to Water Bodies and Mountain Areas
* DistWB, DistMA = Distance to Water Body, Distance to Mountain Area
> 11_01 Obtain the preprocessed water body dataset.
> 11_02 Perform zonal statistics to compute the distance to water bodies (DistWB).


# 07_00 Auxiliary Code: Implementing Multi-Processing

In [2]:
import os
import geopandas as gpd
import numpy as np
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.features import shapes
from shapely.geometry import shape
from osgeo import gdal, ogr
import fiona
import pandas as pd
def create_show_progress(threshold=10):
    """
    Create a progress callback function that only prints progress
    when the percentage increases by the specified threshold.
    :param threshold: Percentage threshold for printing progress
    :return: Progress callback function
    """
    last_percentage = -threshold

    def Show_Progress(complete, message, unknown):
        nonlocal last_percentage
        percentage = int(complete * 100)
        if percentage >= last_percentage + threshold:
            print(f"Progress: {percentage}% - {message}")
            last_percentage = percentage

    return Show_Progress

# 07_00 Obtain the input raster filenames.

In [3]:
import os
def get_subfolder_names(parent_folder):
    if not os.path.isdir(parent_folder):
        raise FileNotFoundError(f"The directory {parent_folder} does not exist")

    subfolders = [f for f in os.listdir(parent_folder) if os.path.isdir(os.path.join(parent_folder, f))]

    return subfolders

parent_folder = r'D:\file\a_clip_data\city_SHP\b_output_final_tem'
places = get_subfolder_names(parent_folder)
places2=[]
for place in places:
    place1= os.path.splitext(place)[0]
    place2 = place1.split('_')[-1]
    places2.append(place2)
print(places)
print(places2)

['宫鹏urban_cfa14_100km_0', '宫鹏urban_cfa14_100km_1', '宫鹏urban_cfa14_100km_10', '宫鹏urban_cfa14_100km_100', '宫鹏urban_cfa14_100km_101', '宫鹏urban_cfa14_100km_102', '宫鹏urban_cfa14_100km_103', '宫鹏urban_cfa14_100km_104', '宫鹏urban_cfa14_100km_105', '宫鹏urban_cfa14_100km_106', '宫鹏urban_cfa14_100km_107', '宫鹏urban_cfa14_100km_108', '宫鹏urban_cfa14_100km_109', '宫鹏urban_cfa14_100km_11', '宫鹏urban_cfa14_100km_110', '宫鹏urban_cfa14_100km_111', '宫鹏urban_cfa14_100km_112', '宫鹏urban_cfa14_100km_113', '宫鹏urban_cfa14_100km_114', '宫鹏urban_cfa14_100km_115', '宫鹏urban_cfa14_100km_116', '宫鹏urban_cfa14_100km_117', '宫鹏urban_cfa14_100km_118', '宫鹏urban_cfa14_100km_119', '宫鹏urban_cfa14_100km_12', '宫鹏urban_cfa14_100km_120', '宫鹏urban_cfa14_100km_121', '宫鹏urban_cfa14_100km_122', '宫鹏urban_cfa14_100km_123', '宫鹏urban_cfa14_100km_124', '宫鹏urban_cfa14_100km_125', '宫鹏urban_cfa14_100km_126', '宫鹏urban_cfa14_100km_127', '宫鹏urban_cfa14_100km_128', '宫鹏urban_cfa14_100km_129', '宫鹏urban_cfa14_100km_13', '宫鹏urban_cfa14_100km_130', '宫鹏urban

# 07_01 Raster processing: Process each 1000m raster grid cell into distinct data (to facilitate later conversion into a shapefile).

In [14]:
import os
import shutil

for place, place2 in zip(places, places2):
    # 输入目录路径
    directory_path = rf'D:\file\a_clip_data\city_SHP\b_output_final_tem\{place}\2017day'

    # 获取目录中的第一个文件
    def get_first_file(directory):
        files = os.listdir(directory)
        if not files:
            raise FileNotFoundError(f"No files found in the directory: {directory}")
        return os.path.join(directory, files[0])  # 选择第一个文件

    input_tif = get_first_file(directory_path)

    workspace = rf'D:\file\d_zonalst\{place2}'
    os.makedirs(workspace, exist_ok=True)

    output_tif = os.path.join(workspace, f'{place2}_process.tif')

    if os.path.exists(output_tif):
        print(f"File already exists, skipping: {output_tif}")
        continue

    shutil.copy(input_tif, output_tif)

    print(f"Copied and renamed: {input_tif} -> {output_tif}")


File already exists, skipping: D:\file\d_zonalst\0\0_process.tif
File already exists, skipping: D:\file\d_zonalst\1\1_process.tif
File already exists, skipping: D:\file\d_zonalst\10\10_process.tif
File already exists, skipping: D:\file\d_zonalst\100\100_process.tif
File already exists, skipping: D:\file\d_zonalst\101\101_process.tif
File already exists, skipping: D:\file\d_zonalst\102\102_process.tif
File already exists, skipping: D:\file\d_zonalst\103\103_process.tif
File already exists, skipping: D:\file\d_zonalst\104\104_process.tif
File already exists, skipping: D:\file\d_zonalst\105\105_process.tif
File already exists, skipping: D:\file\d_zonalst\106\106_process.tif
File already exists, skipping: D:\file\d_zonalst\107\107_process.tif
File already exists, skipping: D:\file\d_zonalst\108\108_process.tif
File already exists, skipping: D:\file\d_zonalst\109\109_process.tif
Copied and renamed: D:\file\a_clip_data\city_SHP\b_output_final_tem\宫鹏urban_cfa14_100km_11\2017day\gf_Day2017_001

# 07_02 Raster to unique value

In [45]:
import rasterio
import numpy as np

def process_raster_with_mask(input_tif, mask_tif, output_tif):
    try:
        # 读取输入栅格数据
        with rasterio.open(input_tif) as src:
            src_image = src.read(1)  # 读取栅格数据
            src_meta = src.meta.copy()  # 读取栅格的元数据

            # 在栅格数据上添加随机数
            random_data = src_image + np.random.random(src_image.shape)

        # 读取掩膜栅格数据
        with rasterio.open(mask_tif) as mask_src:
            mask_image = mask_src.read(1)  # 读取掩膜数据
            mask_meta = mask_src.meta.copy()  # 读取掩膜的元数据

        # 确保掩膜和目标栅格文件的空间参考系统和大小一致
        if src_meta['crs'] != mask_meta['crs'] or src_meta['width'] != mask_meta['width'] or src_meta['height'] != mask_meta['height']:
            raise ValueError("掩膜和目标栅格的空间参考系统或大小不一致")

        mask = mask_image > -1000  # 这里假设掩膜值大于 0 的区域为有效区域
        out_image = np.where(mask, random_data, np.nan)  # 使用 NaN 代替掩膜区域外的值

        # 更新栅格的元数据
        out_meta = src_meta.copy()
        out_meta.update({
            "driver": "GTiff",
            "height": out_image.shape[0],
            "width": out_image.shape[1],
            "count": 1,
            "dtype": "float32"
        })

        # 保存处理后的栅格数据
        with rasterio.open(output_tif, "w", **out_meta) as dest:
            dest.write(out_image.astype(rasterio.float32), 1)

        print(f'处理后的栅格保存到: {output_tif}')

    except Exception as e:
        print(f"处理栅格数据时出错: {e}")

for place2 in places2:
    workspace = rf'D:\file\d_zonalst\{place2}'
    output_tif_wgs84 = os.path.join(workspace, f'{place2}_process.tif')
    mask_tif = output_tif_wgs84
    process_tif = os.path.join(workspace, f'{place2}_processed_raster.tif')
    # 检查输出文件是否已存在
    # if os.path.exists(process_tif):
    #     print(f"文件已存在，跳过处理: {process_tif}")
    #     continue
    process_raster_with_mask(output_tif_wgs84, mask_tif, process_tif)

处理后的栅格保存到: D:\file\d_zonalst\0\0_processed_raster.tif
处理后的栅格保存到: D:\file\d_zonalst\1\1_processed_raster.tif
处理后的栅格保存到: D:\file\d_zonalst\10\10_processed_raster.tif
处理后的栅格保存到: D:\file\d_zonalst\100\100_processed_raster.tif
处理后的栅格保存到: D:\file\d_zonalst\101\101_processed_raster.tif
处理栅格数据时出错: D:/file/d_zonalst/102/102_process.tif: No such file or directory
处理后的栅格保存到: D:\file\d_zonalst\103\103_processed_raster.tif
处理后的栅格保存到: D:\file\d_zonalst\104\104_processed_raster.tif
处理后的栅格保存到: D:\file\d_zonalst\105\105_processed_raster.tif
处理后的栅格保存到: D:\file\d_zonalst\106\106_processed_raster.tif
处理后的栅格保存到: D:\file\d_zonalst\107\107_processed_raster.tif
处理后的栅格保存到: D:\file\d_zonalst\108\108_processed_raster.tif
处理后的栅格保存到: D:\file\d_zonalst\109\109_processed_raster.tif
处理后的栅格保存到: D:\file\d_zonalst\11\11_processed_raster.tif
处理栅格数据时出错: D:/file/d_zonalst/110/110_process.tif: No such file or directory
处理后的栅格保存到: D:\file\d_zonalst\111\111_processed_raster.tif
处理后的栅格保存到: D:\file\d_zonalst\112\112_processed_r

# 07_03 Raster to Shapefile

In [ ]:
import os
import rasterio
from rasterio.features import shapes
import numpy as np
import fiona
from shapely.geometry import shape

def raster_to_polygon(input_tif, output_shp):
    try:
        with rasterio.open(input_tif) as src:
            # Read raster data
            image = src.read(1)  # Read the first band

            # Handle NaN values
            no_data_value = src.nodata
            if no_data_value is not None:
                image[np.isnan(image)] = no_data_value  # Replace NaN values with NoData value
                mask = image != no_data_value
            else:
                mask = np.ones_like(image, dtype=bool)

            # Scale pixel values and convert to integer
            image = np.round(image * 1000000).astype(np.int32)

            # Define the schema for the shapefile (no properties)
            schema = {
                'geometry': 'Polygon',
                'properties': {}
            }

            # Convert raster to polygons
            results = (
                {'geometry': shape(s)}
                for s, v in shapes(image, mask=mask, transform=src.transform)
                if v != no_data_value  # Filter out NoData values
            )

            # Write to shapefile
            with fiona.open(output_shp, 'w', driver='ESRI Shapefile', schema=schema, crs=src.crs) as shp:
                for result in results:
                    shp.write(result)

        print(f"Shapefile created at: {output_shp}")
    except Exception as e:
        print(f"Error converting raster to shapefile: {e}")

for place2 in places2:
    workspace = rf'D:\file\d_zonalst\{place2}'
    process_tif = os.path.join(workspace, f'{place2}_processed_raster.tif')
    output_shp = os.path.join(workspace, f'{place2}_output_file.shp')
    # 检查输出文件是否已存在
    if os.path.exists(output_shp):
        print(f"文件已存在，跳过处理: {output_shp}")
        continue
    raster_to_polygon(process_tif, output_shp)

# 08_00 (Optional, Auxiliary) Clip the source building dataset into 500,000m raster tiles for easier processing.

In [55]:
'''
from osgeo import gdal, gdalconst
import os
import numpy as np

def crop_tiff(input_tiff, output_folder, crop_width, crop_height):
    # 打开输入的TIFF文件
    dataset = gdal.Open(input_tiff, gdalconst.GA_ReadOnly)
    if dataset is None:
        print("无法打开TIFF文件！")
        return

    # 获取TIFF文件的基本信息
    width = dataset.RasterXSize
    height = dataset.RasterYSize
    bands = dataset.RasterCount
    driver = gdal.GetDriverByName("GTiff")
    if driver is None:
        print("未找到GTiff驱动程序！")
        return
    transform = dataset.GetGeoTransform()
    projection = dataset.GetProjection()

    # 计算裁剪后的栅格数量
    num_cols = (width + crop_width - 1) // crop_width  # 向上取整
    num_rows = (height + crop_height - 1) // crop_height  # 向上取整
    print(f'num_rows: {num_rows}')
    print(f'num_cols: {num_cols}')

    # 逐个栅格裁剪并保存
    num_saved_files = 0
    for i in range(num_rows):
        for j in range(num_cols):
            x_offset = j * crop_width
            y_offset = i * crop_height
            x_end = min(x_offset + crop_width, width)
            y_end = min(y_offset + crop_height, height)

            # 计算裁剪块的地理坐标
            min_lon = transform[0] + x_offset * transform[1]
            max_lon = transform[0] + x_end * transform[1]
            min_lat = transform[3] + y_offset * transform[5]
            max_lat = transform[3] + y_end * transform[5]

            # 检查栅格中的像素值是否全部为0
            is_all_zero = True
            for b in range(bands):
                band = dataset.GetRasterBand(b + 1)
                data = np.asarray(band.ReadAsArray(x_offset, y_offset, x_end - x_offset, y_end - y_offset))
                if not np.all(data == 0):
                    is_all_zero = False
                    break

            if not is_all_zero:
                output_tiff_path = os.path.join(output_folder, f"raster_{(min_lon/100000):.1f}_{(max_lat/100000):.1f}.tif")

                # 检查输出文件是否已经存在，并且文件大小是否小于10KB
                if os.path.exists(output_tiff_path):
                    if os.path.getsize(output_tiff_path) < 10 * 1024:  # 文件小于10KB
                        print(f"文件大小小于10KB，覆盖: {output_tiff_path}")
                    else:
                        print(f"文件已存在，跳过: {output_tiff_path}")
                        continue
                else:
                    print(f"创建新文件: {output_tiff_path}")

                new_dataset = driver.Create(output_tiff_path, x_end - x_offset, y_end - y_offset, bands, gdalconst.GDT_Float32)
                if new_dataset is None:
                    print(f"无法创建输出TIFF文件: {output_tiff_path}")
                    continue

                new_transform = (transform[0] + x_offset * transform[1], transform[1], transform[2],
                                 transform[3] + y_offset * transform[5], transform[4], transform[5])
                new_dataset.SetGeoTransform(new_transform)
                new_dataset.SetProjection(projection)
                for b in range(bands):
                    band = dataset.GetRasterBand(b + 1)
                    data = np.asarray(band.ReadAsArray(x_offset, y_offset, x_end - x_offset, y_end - y_offset))
                    new_dataset.GetRasterBand(b + 1).WriteArray(data)
                new_dataset = None
                num_saved_files += 1

    print(f"总共保存的文件数: {num_saved_files}")

    # 关闭输入的TIFF文件
    dataset = None

# 指定裁剪后的栅格大小
crop_width = 5000
crop_height = 5000
# 指定输入的TIFF文件和输出文件夹路径
input_tiff = rf'F:\BUILT\GHS_BUILT_H_AGBH_E2018_GLOBE_R2023A_54009_100_V1_0.tif'
output_folder = rf'F:\BUILT\{crop_height}m_raster'
os.makedirs(output_folder, exist_ok=True)

# 裁剪TIFF文件
crop_tiff(input_tiff, output_folder, crop_width, crop_height)
'''

'\nfrom osgeo import gdal, gdalconst\nimport os\nimport numpy as np\n\ndef crop_tiff(input_tiff, output_folder, crop_width, crop_height):\n    # 打开输入的TIFF文件\n    dataset = gdal.Open(input_tiff, gdalconst.GA_ReadOnly)\n    if dataset is None:\n        print("无法打开TIFF文件！")\n        return\n\n    # 获取TIFF文件的基本信息\n    width = dataset.RasterXSize\n    height = dataset.RasterYSize\n    bands = dataset.RasterCount\n    driver = gdal.GetDriverByName("GTiff")\n    if driver is None:\n        print("未找到GTiff驱动程序！")\n        return\n    transform = dataset.GetGeoTransform()\n    projection = dataset.GetProjection()\n\n    # 计算裁剪后的栅格数量\n    num_cols = (width + crop_width - 1) // crop_width  # 向上取整\n    num_rows = (height + crop_height - 1) // crop_height  # 向上取整\n    print(f\'num_rows: {num_rows}\')\n    print(f\'num_cols: {num_cols}\')\n\n    # 逐个栅格裁剪并保存\n    num_saved_files = 0\n    for i in range(num_rows):\n        for j in range(num_cols):\n            x_offset = j * crop_width\n            y

# 由于building的数据是54009 mollweide坐标系 和84的边界不一样，因此干脆换坐标系

In [11]:
import os
import geopandas as gpd

def get_subfolder_names(parent_folder):
    if not os.path.isdir(parent_folder):
        raise FileNotFoundError(f"The directory {parent_folder} does not exist")

    subfolders = [f for f in os.listdir(parent_folder) if os.path.isdir(os.path.join(parent_folder, f))]

    return subfolders

parent_folder = r'D:\file\a_clip_data\city_SHP\b_output_final_tem'
places = get_subfolder_names(parent_folder)
places2=[]
for place in places:
    place1= os.path.splitext(place)[0]
    place2 = place1.split('_')[-1]
    places2.append(place2)
print(places)
print(places2)


['宫鹏urban_cfa14_100km_0', '宫鹏urban_cfa14_100km_1', '宫鹏urban_cfa14_100km_10', '宫鹏urban_cfa14_100km_100', '宫鹏urban_cfa14_100km_101', '宫鹏urban_cfa14_100km_102', '宫鹏urban_cfa14_100km_103', '宫鹏urban_cfa14_100km_104', '宫鹏urban_cfa14_100km_105', '宫鹏urban_cfa14_100km_106', '宫鹏urban_cfa14_100km_107', '宫鹏urban_cfa14_100km_108', '宫鹏urban_cfa14_100km_109', '宫鹏urban_cfa14_100km_11', '宫鹏urban_cfa14_100km_110', '宫鹏urban_cfa14_100km_111', '宫鹏urban_cfa14_100km_112', '宫鹏urban_cfa14_100km_113', '宫鹏urban_cfa14_100km_114', '宫鹏urban_cfa14_100km_115', '宫鹏urban_cfa14_100km_116', '宫鹏urban_cfa14_100km_117', '宫鹏urban_cfa14_100km_118', '宫鹏urban_cfa14_100km_119', '宫鹏urban_cfa14_100km_12', '宫鹏urban_cfa14_100km_120', '宫鹏urban_cfa14_100km_121', '宫鹏urban_cfa14_100km_122', '宫鹏urban_cfa14_100km_123', '宫鹏urban_cfa14_100km_124', '宫鹏urban_cfa14_100km_125', '宫鹏urban_cfa14_100km_126', '宫鹏urban_cfa14_100km_127', '宫鹏urban_cfa14_100km_128', '宫鹏urban_cfa14_100km_129', '宫鹏urban_cfa14_100km_13', '宫鹏urban_cfa14_100km_130', '宫鹏urban

In [ ]:

def convert_shapefiles_to_54009(target_workspace, places2):
    """ 批量转换 Shapefile 到 ESRI:54009 坐标系 """
    output_dir = os.path.join(target_workspace, "c_converted_54009")
    os.makedirs(output_dir, exist_ok=True)

    for place2 in places2:
        shp_file = os.path.join(target_workspace, f'a_city_split\\polygon_{place2}.shp')
        output_shp = os.path.join(output_dir, f'polygon_{place2}_54009.shp')

        if not os.path.exists(shp_file):
            print(f"⚠️  文件不存在: {shp_file}，跳过...")
            continue

        # 读取 Shapefile
        gdf = gpd.read_file(shp_file)
        # 先创建一个字段映射字典（手动或自动）
        field_mapping = {
            "气候分": "climate_score",  # 你可以改成适合你的英文字段名
            "温度": "temperature",
            "湿度": "humidity"
        }

        # 修改列名
        gdf.rename(columns=field_mapping, inplace=True)

        if gdf.crs is None:
            print(f"⚠️ {place2} Shapefile 缺少坐标系信息，跳过...")
            continue

        # 转换坐标系
        target_crs = "ESRI:54009"
        if gdf.crs.to_string() != target_crs:
            gdf = gdf.to_crs(target_crs)
            print(f"✅ {place2} 已转换至 {target_crs}")
        else:
            print(f"🔹 {place2} 已经是 {target_crs}，无需转换")

        # 保存新的 Shapefile
        gdf.to_file(output_shp, driver="ESRI Shapefile")
        print(f"📂 已保存: {output_shp}")

    print("🎯 所有 Shapefile 处理完成！")

target_workspace = r'D:\file\a_clip_data\\city_SHP'
# 调用函数
convert_shapefiles_to_54009(target_workspace, places2)


# 08_01 Compute the latitude and longitude boundaries of the urban data. Identify land parcels that intersect with the urban data. If multiple parcels contain urban data, merge the files.Then Remove NoData values from the building dataset to prevent calculation errors.

In [41]:
import os
import rasterio
import numpy as np
import geopandas as gpd
from rasterio.merge import merge
from rasterio.mask import mask
from shapely.geometry import box
from pyproj import Transformer

def get_shapefile_bounding_box(shp_path):
    """ 获取 Shapefile 的边界框，并转换到 WGS 84 (EPSG:4326) """
    gdf = gpd.read_file(shp_path)
    if gdf.crs is None:
        raise ValueError(f"Shapefile {shp_path} 缺少坐标系信息！")

    min_x, min_y, max_x, max_y = gdf.total_bounds
    crs = gdf.crs.to_string()

    # 如果坐标系不是 WGS 84，则转换
    if crs != 'ESRI:54009':
        transformer = Transformer.from_crs(crs, 'ESRI:54009', always_xy=True)
        min_x, min_y = transformer.transform(min_x, min_y)
        max_x, max_y = transformer.transform(max_x, max_y)

    return min_x, min_y, max_x, max_y

def get_raster_bounding_box(raster_path):
    """ 获取 TIFF 栅格文件的边界框，并转换到 WGS 84 (EPSG:4326) """
    try:
        with rasterio.open(raster_path) as src:
            if src.crs is None:
                raise ValueError(f"TIFF 文件 {raster_path} 缺少坐标系信息！")

            min_x, min_y, max_x, max_y = src.bounds
            crs = src.crs.to_string()

            # 如果坐标系不是 WGS 84，则转换
            if crs != 'ESRI:54009':
                transformer = Transformer.from_crs(crs, 'ESRI:54009', always_xy=True)
                min_x, min_y = transformer.transform(min_x, min_y)
                max_x, max_y = transformer.transform(max_x, max_y)

            return min_x, min_y, max_x, max_y

    except rasterio.errors.RasterioIOError as e:
        print(f"⚠️ 无法读取 {raster_path}，可能是损坏的 TIFF 文件，已跳过。错误: {e}")
        return None

def is_bbox_intersect(file_bbox, target_bbox, buffer=0.5):
    """ 判断文件的边界框是否与目标范围有交集 """
    f_min_x, f_min_y, f_max_x, f_max_y = file_bbox
    t_min_x, t_min_y, t_max_x, t_max_y = target_bbox

    # 目标范围扩展 buffer
    t_min_x -= buffer
    t_max_x += buffer
    t_min_y -= buffer
    t_max_y += buffer

    return not (f_max_x < t_min_x or f_min_x > t_max_x or
                f_max_y < t_min_y or f_min_y > t_max_y)

def find_intersecting_files(raster_folder, target_bbox):
    """ 查找与目标 bbox 相交的 TIFF 文件 """
    intersecting_files = []
    for file in os.listdir(raster_folder):
        if file.endswith(".tif") or file.endswith(".tiff"):
            raster_path = os.path.join(raster_folder, file)
            file_bbox = get_raster_bounding_box(raster_path)
            if file_bbox and is_bbox_intersect(file_bbox, target_bbox):
                print(file_bbox)
                print(file)
                intersecting_files.append(raster_path)
    return intersecting_files

from rasterio.warp import transform_bounds

def mask_and_merge_rasters(raster_files, output_path, target_bbox):
    """ 合并多个栅格文件并裁剪到 target_bbox，并只保留值为 80 的像素 """
    if not raster_files:
        print("❌ 没有找到相交的栅格文件，跳过处理。")
        return

    # 打开栅格文件
    src_files = [rasterio.open(fp) for fp in raster_files]
    print(f'{src_files} = src files')

    # 获取栅格文件的坐标系
    with rasterio.open(raster_files[0]) as ref_src:
        raster_crs = ref_src.crs
        print(f"📌 栅格文件 CRS: {raster_crs}")

    # 获取目标 shapefile 坐标系
    target_crs = "ESRI:54009"
    print(f"📌 目标 BBOX: {target_bbox}, CRS 需要转换: {raster_crs} → {target_crs}")

    if raster_crs != target_crs:
        print("⚠️ 正在转换 target_bbox 坐标系...")
        min_x, min_y, max_x, max_y = transform_bounds(raster_crs, target_crs, *target_bbox)

        target_bbox = (min_x, min_y, max_x, max_y)

    # 合并栅格文件
    mosaic, out_trans = merge(src_files)

    out_meta = src_files[0].meta.copy()

    out_meta.update({
        "driver": "GTiff",
        "count": 1,
        "height": mosaic.shape[1],
        "width": mosaic.shape[2],
        "transform": out_trans,
        "dtype": "float32",
        "nodata": np.nan
    })

    # 生成裁剪范围
    min_x, min_y, max_x, max_y = target_bbox
    bbox_geom = [box(min_x, min_y, max_x, max_y)]

    with rasterio.io.MemoryFile() as memfile:
        with memfile.open(**out_meta) as mem_raster:
            mem_raster.write(mosaic)

            # 使用目标坐标系进行裁剪
            bbox_mask, bbox_transform = mask(
                dataset=mem_raster,
                shapes=bbox_geom,
                crop=True
            )

    # 只保留 0 < 值 < 101 的数据
    data = bbox_mask[0].astype(float)
    data[(data <= 0) | (data >= 101)] = np.nan  # 0 和 >=101 的像素设为 NaN

    # 如果整个栅格都是无效值，则不保存
    if np.all(data == np.nan):
        print(f"🚫 {output_path} 仅包含 nodata 值，已跳过。")
        return
    mosaic = mosaic.astype(np.float32)

    out_meta.update({
        "height": data.shape[0],
        "width": data.shape[1],
        "transform": bbox_transform,
        "dtype": mosaic.dtype.name,
        "nodata": np.nan
    })

    # 保存裁剪后的栅格文件
    with rasterio.open(output_path, "w", compress="LZW", **out_meta) as dest:
        dest.write(data, 1)

    print(f"✅ 已保存: {output_path}")


# 批量处理多个地区
target_workspace = r'D:\file'
raster_workspace = rf'F:\BUILT\500000m_raster'
zonalst_folder = r'D:\file\d_zonalst'
for place2 in os.listdir(zonalst_folder):
    # 你的代码逻辑
    print(place2)

    shp_file = os.path.join(target_workspace, rf'a_clip_data\city_SHP\c_converted_54009\polygon_{place2}_54009.shp')
    output_workspace = os.path.join(target_workspace, rf"d_zonalst\{place2}")
    os.makedirs(output_workspace, exist_ok=True)

    output_path = os.path.join(output_workspace, f'{place2}_merged_output.tif')

    if os.path.exists(output_path):
        print(f"⚠️ 文件已存在，跳过: {output_path}")
        continue

    target_bbox = get_shapefile_bounding_box(shp_file)
    print(f'📍 目标区域 {place2} 边界: {target_bbox}')

    intersecting_files = find_intersecting_files(raster_workspace, target_bbox)

    if intersecting_files:
        print(f"📂 {place2} 相交的栅格文件: {len(intersecting_files)} 个")
        mask_and_merge_rasters(intersecting_files, output_path, target_bbox)
    else:
        print(f"🚫 {place2} 没有相交的栅格文件，跳过")

0
⚠️ 文件已存在，跳过: D:\file\d_zonalst\0\0_merged_output.tif
1
⚠️ 文件已存在，跳过: D:\file\d_zonalst\1\1_merged_output.tif
10
⚠️ 文件已存在，跳过: D:\file\d_zonalst\10\10_merged_output.tif
100
⚠️ 文件已存在，跳过: D:\file\d_zonalst\100\100_merged_output.tif
101
⚠️ 文件已存在，跳过: D:\file\d_zonalst\101\101_merged_output.tif
103
⚠️ 文件已存在，跳过: D:\file\d_zonalst\103\103_merged_output.tif
104
⚠️ 文件已存在，跳过: D:\file\d_zonalst\104\104_merged_output.tif
105
⚠️ 文件已存在，跳过: D:\file\d_zonalst\105\105_merged_output.tif
106
⚠️ 文件已存在，跳过: D:\file\d_zonalst\106\106_merged_output.tif
107
⚠️ 文件已存在，跳过: D:\file\d_zonalst\107\107_merged_output.tif
108
⚠️ 文件已存在，跳过: D:\file\d_zonalst\108\108_merged_output.tif
109
⚠️ 文件已存在，跳过: D:\file\d_zonalst\109\109_merged_output.tif
11
⚠️ 文件已存在，跳过: D:\file\d_zonalst\11\11_merged_output.tif
111
⚠️ 文件已存在，跳过: D:\file\d_zonalst\111\111_merged_output.tif
112
⚠️ 文件已存在，跳过: D:\file\d_zonalst\112\112_merged_output.tif
113
⚠️ 文件已存在，跳过: D:\file\d_zonalst\113\113_merged_output.tif
114
⚠️ 文件已存在，跳过: D:\file\d_zonalst\114\114

# ******* 删除多余的文件 保持223

In [31]:
import os
import re
import shutil

# 目标文件夹
ma_folder = r'D:\file\c_moving_average(MA)'
zonalst_folder = r'D:\file\d_zonalst'

# 获取文件夹中的所有文件名
file_list = os.listdir(ma_folder)

# 提取 UH_MA_ 后的数字
numbers_set = set()

for filename in file_list:
    match = re.search(r'UH_MA_(\d+)', filename)  # 提取 UH_MA_ 后的数字
    if match:
        numbers_set.add(int(match.group(1)))  # 转换为整数并添加到 set

# 转换为列表并排序
numbers_list = sorted(numbers_set)
print(f"有效的 place2 列表: {numbers_list}")

# 遍历 d_zonalst 目录下的所有子文件夹
for place2 in os.listdir(zonalst_folder):
    place2_path = os.path.join(zonalst_folder, place2)

    # 确保是文件夹
    if os.path.isdir(place2_path):
        try:
            # 检查文件夹名是否是数字，且是否在 numbers_list 里
            if place2.isdigit() and int(place2) not in numbers_list:
                print(f"删除文件夹: {place2_path}")
                shutil.rmtree(place2_path)  # 递归删除整个文件夹
        except Exception as e:
            print(f"删除 {place2_path} 时出错: {e}")

print("清理完成！")

有效的 place2 列表: [0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 16, 18, 19, 20, 21, 22, 24, 26, 27, 28, 29, 30, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 70, 71, 72, 73, 74, 75, 78, 80, 81, 83, 85, 88, 89, 90, 92, 94, 96, 97, 98, 99, 100, 101, 103, 104, 105, 106, 107, 108, 109, 111, 112, 113, 114, 115, 116, 117, 118, 119, 121, 122, 123, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 143, 144, 145, 147, 148, 149, 151, 152, 153, 154, 155, 156, 157, 159, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 177, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 225, 227, 228, 229, 230, 232, 235, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 256

# 08_02 Perform zonal statistics to compute *MEAN* (=building height).

In [ ]:
import arcpy
input_folder = r"D:\file\d_zonalst"

for place2 in os.listdir(input_folder):
    # 定义原始文件和目标文件路径
    in_shp = r"D:\file\d_zonalst\{}\{}_output_file.shp".format(place2,place2)
    out_shp = r"D:\file\d_zonalst\{}\{}_output_file_copy.shp".format(place2,place2)

    # 复制原始文件
    arcpy.management.CopyFeatures(in_shp, out_shp)

    # 检查并添加字段 "Landarea"（类型为 DOUBLE）
    field_names = [f.name for f in arcpy.ListFields(out_shp)]
    if "Landarea" not in field_names:
        arcpy.AddField_management(out_shp, "Landarea", "DOUBLE")
        arcpy.AddMessage("Field 'Landarea' added.")

    # 计算几何属性，使用自定义 Sinusoidal 投影
    arcpy.management.CalculateGeometryAttributes(
        out_shp,
        "Landarea AREA",
        '',
        "SQUARE_METERS",
        "PROJCS['unnamed_ellipse_Sinusoidal',GEOGCS['GCS_unnamed_ellipse',"
        "DATUM['D_unknown',SPHEROID['Unknown',6371007.181,0.0]],"
        "PRIMEM['Greenwich',0.0],UNIT['Degree',0.0174532925199433]],"
        "PROJECTION['Sinusoidal'],PARAMETER['false_easting',0.0],"
        "PARAMETER['false_northing',0.0],PARAMETER['central_meridian',0.0],"
        "UNIT['Meter',1.0]]",
        "SAME_AS_INPUT"
    )
    arcpy.AddMessage("Geometry attributes calculated. Output saved to: {}".format(out_shp))


In [ ]:
import os
import geopandas as gpd
import numpy as np
import rasterio
from rasterio.mask import mask
from shapely.ops import transform as shapely_transform
import pandas as pd
from pyproj import Transformer

def remove_nodata(data, no_data):
    """移除数据中的 NoData、0 和 NaN 值。"""
    if no_data is not None:
        mask_arr = (data == no_data) | (data == 0) | np.isnan(data)
    else:
        mask_arr = (data == 0) | np.isnan(data)
    return data[~mask_arr]

def create_show_progress(threshold=10):
    last_percentage = -threshold
    def Show_Progress(complete, message, unknown):
        nonlocal last_percentage
        percentage = int(complete * 100)
        if percentage >= last_percentage + threshold:
            last_percentage = percentage
    return Show_Progress

def zonal_statistics(raster_path, vector_path, output_path, no_overlap_files, place2):
    try:
        # 如果栅格文件不存在，则直接处理矢量数据，并转换到 ESRI:54009 输出
        if not os.path.exists(raster_path):
            print(f"Raster file {raster_path} not found. Setting Buiheight to NoData.")
            vector = gpd.read_file(vector_path)
            vector["Buiheight"] = -9999
            transformer = Transformer.from_crs(vector.crs, "ESRI:54009", always_xy=True)
            project = lambda x, y, z=None: transformer.transform(x, y)
            vector = vector.to_crs("ESRI:54009")
            vector.to_file(output_path)
            return

        # 打开栅格文件，获取 NoData 值和 CRS
        with rasterio.open(raster_path) as raster:
            no_data = raster.nodata
            raster_crs = raster.crs

        # 读取矢量数据并确保其 CRS 与栅格一致
        vector = gpd.read_file(vector_path)
        if vector.crs != raster_crs:
            vector = vector.to_crs(raster_crs)

        # 构造转换器，将当前 CRS 转换到 ESRI:54009，用于面积计算
        transformer = Transformer.from_crs(raster_crs, "ESRI:54009", always_xy=True)
        project = lambda x, y, z=None: transformer.transform(x, y)

        show_progress = create_show_progress(threshold=10)
        total_zones = vector.shape[0]
        results = []
        has_overlap = False

        for idx, row in vector.iterrows():
            geom = row['geometry']
            with rasterio.open(raster_path) as raster:
                out_image, out_transform = mask(raster, [geom], crop=True)
                if out_image.size == 0:
                    print(f"No data for feature {idx + 1}, skipping...")
                    continue
                else:
                    has_overlap = True

                data = out_image[0]
                valid_data = remove_nodata(data, no_data)
                mean_val = float(np.nanmean(valid_data)) if valid_data.size > 0 else -9999

                results.append({'Buiheight': mean_val})

            completed = (idx + 1) / total_zones
            show_progress(completed, 'Processing zones', None)

        # 合并统计结果到矢量数据中
        stats_df = pd.DataFrame(results)
        vector = vector.join(stats_df)

        # 将整个矢量数据重投影到 ESRI:54009 后输出
        vector = vector.to_crs("ESRI:54009")
        vector.to_file(output_path)
        print(f"Zonal statistics saved to: {output_path}")

        if not has_overlap:
            no_overlap_files.append(f'{place2}')
    except Exception as e:
        print(f"Error processing {output_path}: {e}")
        no_overlap_files.append(f'{place2}')

# 主流程：遍历输入文件夹中每个子文件夹，执行区域统计
no_overlap_files = []
input_folder = r"D:\file\d_zonalst"

for place2 in os.listdir(input_folder):
    workspace = os.path.join(input_folder, place2)
    vector_path = os.path.join(workspace, f'{place2}_output_file_copy.shp')
    raster_path = os.path.join(workspace, f'{place2}_merged_output.tif')
    output_path = os.path.join(workspace, f'{place2}_output_file(MEAN).shp')
    zonal_statistics(raster_path, vector_path, output_path, no_overlap_files, place2)

print(f"Files without overlap: {no_overlap_files}")


# ******* 用来批量删除文件的

In [ ]:
# import os
# import shutil
#
# input_folder = r'D:\file\d_zonalst'
#
# for place2 in os.listdir(input_folder):
#     workspace = os.path.join(input_folder, place2)
#     merged_tif_path = os.path.join(workspace, 'water')
#
#     # 判断是文件还是文件夹
#     if os.path.exists(merged_tif_path):
#         if os.path.isdir(merged_tif_path):
#             shutil.rmtree(merged_tif_path)  # 删除文件夹
#             print(f"Deleted folder: {merged_tif_path}")
#         else:
#             os.remove(merged_tif_path)  # 删除文件
#             print(f"Deleted file: {merged_tif_path}")
#     else:
#         print(f"Not found: {merged_tif_path}")


# 09_01 Perform zonal statistics to compute building density and floor area ratio (FAR).

In [9]:
import os
import rasterio
from rasterio.mask import mask
import geopandas as gpd
import pandas as pd
import numpy as np
from pyproj import Transformer, CRS

def create_show_progress(threshold=10):
    """
    创建一个进度回调函数，每当进度增加达到阈值时打印一次信息。
    """
    last_percentage = -threshold
    def Show_Progress(complete, message, unknown):
        nonlocal last_percentage
        percentage = int(complete * 100)
        if percentage >= last_percentage + threshold:
            last_percentage = percentage
            print("Progress: {}% - {}".format(percentage, message))
    return Show_Progress

def calculate_pixel_size_in_meters(raster_path, target_crs_str="ESRI:54009"):
    """
    利用中心像元的角点坐标，将其转换到目标投影后，
    计算像元的宽度、高度及面积（平行四边形假设）。
    """
    with rasterio.open(raster_path) as src:
        height, width = src.height, src.width
        center_row = height // 2
        center_col = width // 2

        # 获取中心像元的左上 (ul)、右上 (ur) 和左下 (ll) 角点坐标
        ul = src.xy(center_row, center_col, offset="ul")
        ur = src.xy(center_row, center_col, offset="ur")
        ll = src.xy(center_row, center_col, offset="ll")

        # 创建目标 CRS 对象
        target_crs = CRS.from_string(target_crs_str)
        transformer = Transformer.from_crs(src.crs, target_crs, always_xy=True)

        # 将角点坐标转换到目标投影
        ul_t = transformer.transform(ul[0], ul[1])
        ur_t = transformer.transform(ur[0], ur[1])
        ll_t = transformer.transform(ll[0], ll[1])

        # 计算转换后像元宽度和高度
        pixel_width = abs(ur_t[0] - ul_t[0])
        pixel_height = abs(ul_t[1] - ll_t[1])
        pixel_area = pixel_width * pixel_height

        return pixel_width, pixel_height, pixel_area

def remove_nodata(data, no_data):
    """
    移除数据中的 NoData 值、0 值以及 NaN 值。
    """
    if no_data is not None:
        mask_arr = (data == no_data) | (data == 0) | np.isnan(data)
    else:
        mask_arr = (data == 0) | np.isnan(data)
    return data[~mask_arr]

def zonal_statistics(raster_path, vector_path, output_path):
    """
    对每个分区（矢量多边形）进行栅格掩模和统计：
    1. 计算目标投影下的像元尺寸（以便后续计算像元面积）。
    2. 针对每个分区，利用 rasterio.mask.mask 提取该区域栅格数据，
       并计算有效像元个数、乘以像元面积和建筑覆盖率后得到建筑面积。
    3. 结果合并到矢量数据中，并保存输出。
    """
    try:
        # 打开栅格文件，获取 NoData 值和 CRS
        with rasterio.open(raster_path) as raster:
            no_data = raster.nodata
            raster_crs = raster.crs

        # 计算像元尺寸（目标投影下的尺寸，单位：米）
        pixel_size_x, pixel_size_y, pixel_area = calculate_pixel_size_in_meters(raster_path)
        print("Pixel width: {:.3f} m, Pixel height: {:.3f} m, Pixel area: {:.3f} m^2".format(pixel_size_x, pixel_size_y, pixel_area))

        # 读取矢量数据
        vector = gpd.read_file(vector_path)
        # 如果栅格不存在，则给出缺省值后退出
        if not os.path.exists(raster_path):
            print("Raster file {} not found. Setting BldArea, CoverRatio, FAR to NoData.".format(raster_path))
            vector["BldArea"] = -9999
            vector["CoverRatio"] = -9999
            vector["FAR"] = -9999
            vector.to_file(output_path)
            return

        # 保证矢量数据和栅格数据在相同 CRS 下
        if vector.crs != raster_crs:
            vector = vector.to_crs(raster_crs)

        # 初始化进度回调
        show_progress = create_show_progress(threshold=10)
        total_zones = vector.shape[0]
        results = []

        # 对每个分区（矢量多边形）进行处理
        for idx, row in vector.iterrows():
            geom = row['geometry']
            #print("Processing feature {}/{}".format(idx+1, total_zones))

            # 裁出一个小片栅格（掩模），默认只考虑像元中心落在多边形内部的像元
            with rasterio.open(raster_path) as raster:
                out_image, out_transform = mask(raster, [geom], crop=True, all_touched=False)
                data = out_image[0]
                valid_data = remove_nodata(data, no_data)

                # 如果存在有效数据，则计算建筑面积
                if valid_data.size > 0:
                    # 此处假设建筑覆盖率为有效像元平均值/100，
                    # 并以像元总面积乘以覆盖率得到建筑面积
                    count_val = valid_data.size
                    building_percentage = np.average(valid_data) / 100.0
                    valid_building_area = count_val * pixel_area
                    area_val = round(valid_building_area * building_percentage , 1)
                    #print("Computed BldArea:", area_val)
                else:
                    area_val = -9999

                results.append({'BldArea': area_val})

            completed = (idx + 1) / float(total_zones)
            show_progress(completed, 'Processing zones', None)

        # 将统计结果合并到矢量数据中
        stats_df = pd.DataFrame(results)
        vector = vector.join(stats_df)

        # 这里CoverRatio和FAR的计算示例仅供参考，根据实际需求调整公式
        vector['CoverRatio'] = np.where(vector['BldArea'] == -9999, -9999, vector['BldArea'] / vector['Landarea'])
        vector['FAR'] = np.where(np.logical_or(vector['BldArea'] == -9999, vector['Buiheight'] == -9999),
                                 -9999,
                                 vector['CoverRatio'] * vector['Buiheight'] / 3)

        # 保存结果到输出文件
        vector.to_file(output_path)
        print("Zonal statistics saved to:", output_path)

    except Exception as e:
        print("Error calculating zonal statistics:", e)

#设置输入文件夹和输出文件路径
input_folder = r"D:\file\d_zonalst"

for place2 in os.listdir(input_folder):
    workspace = rf'D:\file\d_zonalst\{place2}'
    vector_path2 = os.path.join(workspace, f'{place2}_output_file(MEAN).shp')
    raster_path2 = r'F:\BUILT_S\GHS_BUILT_S_E2018_GLOBE_R2023A_54009_10_V1_0.tif'
    output_path2 = os.path.join(workspace, f'{place2}_output_file(MEAN_BD_FAR).shp')
    #
    # if os.path.exists(output_path2):
    #     print(rf'跳过{output_path2}:输出文件已经存在')
    #     continue
    zonal_statistics(raster_path2, vector_path2, output_path2)

Pixel width: 10.000 m, Pixel height: 10.000 m, Pixel area: 100.000 m^2
Progress: 0% - Processing zones
Progress: 10% - Processing zones
Progress: 20% - Processing zones
Progress: 30% - Processing zones
Progress: 40% - Processing zones
Progress: 50% - Processing zones
Progress: 60% - Processing zones
Progress: 70% - Processing zones
Progress: 80% - Processing zones
Progress: 90% - Processing zones
Progress: 100% - Processing zones
Zonal statistics saved to: D:\file\d_zonalst\0\0_output_file(MEAN_BD_FAR).shp
Pixel width: 10.000 m, Pixel height: 10.000 m, Pixel area: 100.000 m^2
Progress: 0% - Processing zones
Progress: 10% - Processing zones
Progress: 20% - Processing zones
Progress: 30% - Processing zones
Progress: 40% - Processing zones
Progress: 50% - Processing zones
Progress: 60% - Processing zones
Progress: 70% - Processing zones
Progress: 80% - Processing zones
Progress: 90% - Processing zones
Progress: 100% - Processing zones
Zonal statistics saved to: D:\file\d_zonalst\1\1_outpu

KeyboardInterrupt: 

In [ ]:
# import os
# import rasterio
# from rasterio.mask import mask
# import geopandas as gpd
#
# # 示例：定义输入栅格、矢量和输出栅格路径
# input_raster = r"F:\BUILT_S\GHS_BUILT_S_E2018_GLOBE_R2023A_54009_10_V1_0.tif"
# input_vector = r"D:\file\d_zonalst\50\50_output_file(MEAN).shp"
# output_raster = r"D:\file\d_zonalst\50\50_clipped_raster.tif"
#
# # 读取矢量文件，这里假设我们取第一个要素
# vector = gpd.read_file(input_vector)
# geom = vector.iloc[0].geometry  # 取第一个几何
#
# # 裁剪栅格
# with rasterio.open(input_raster) as src:
#     out_image, out_transform = mask(src, [geom], crop=True, all_touched=False)
#     out_meta = src.meta.copy()
#
#     # 更新元数据：新的高度、宽度和仿射变换
#     out_meta.update({
#         "driver": "GTiff",
#         "height": out_image.shape[1],
#         "width": out_image.shape[2],
#         "transform": out_transform
#     })
#
#     # 写入输出栅格
#     with rasterio.open(output_raster, "w", **out_meta) as dest:
#         dest.write(out_image)
#
# print("Clipped raster saved to:", output_raster)


In [ ]:
# import rasterio
# import numpy as np
# from pyproj import Transformer, CRS
#
# def calculate_pixel_size_in_meters(raster_path, target_crs_str="ESRI:54009"):
#     """
#     通过获取中心像元的三个角点，在目标投影下计算像元的宽度、高度及面积（平行四边形假设）。
#     """
#     with rasterio.open(raster_path) as src:
#         height, width = src.height, src.width
#         center_row = height // 2
#         center_col = width // 2
#
#         # 获取中心像元的左上 (ul)、右上 (ur) 和左下 (ll) 角点坐标
#         ul = src.xy(center_row, center_col, offset="ul")
#         ur = src.xy(center_row, center_col, offset="ur")
#         ll = src.xy(center_row, center_col, offset="ll")
#
#         # 创建目标 CRS 对象
#         target_crs = CRS.from_string(target_crs_str)
#         transformer = Transformer.from_crs(src.crs, target_crs, always_xy=True)
#
#         # 将角点转换到目标投影
#         ul_t = transformer.transform(ul[0], ul[1])
#         ur_t = transformer.transform(ur[0], ur[1])
#         ll_t = transformer.transform(ll[0], ll[1])
#
#         # 计算像元宽度和高度（平行四边形假设）
#         pixel_width = abs(ur_t[0] - ul_t[0])
#         pixel_height = abs(ul_t[1] - ll_t[1])
#         pixel_area = pixel_width * pixel_height
#
#         return pixel_width, pixel_height, pixel_area
#
# # 定义裁剪后栅格的路径（例如保存了一个小区域的栅格）
# raster_path = r"D:\file\d_zonalst\50\50_clipped_raster.tif"
#
# # 读取裁剪栅格，统计有效像元数量
# with rasterio.open(raster_path) as src:
#     data = src.read(1)
#     nodata = src.nodata
#     if nodata is not None:
#         valid_data = data[(data != nodata) & (data != 0)]
#     else:
#         valid_data = data[data != 0]  # 如果不排除 0，请去掉判断
#     count_pixels = valid_data.size
#
# # 获取像元尺寸和面积
# px_width, px_height, px_area = calculate_pixel_size_in_meters(raster_path)
# total_area = count_pixels * px_area
#
# print("Valid pixel count: {}".format(count_pixels))
# print("Pixel width: {:.3f} m, Pixel height: {:.3f} m, Pixel area: {:.3f} m^2".format(px_width, px_height, px_area))
# print("Total area (based on valid pixels): {:.3f} m^2".format(total_area))


# 10_00 (Optional, Auxiliary) Clip the source canopy dataset into 100,000m raster tiles for easier processing.

In [15]:
'''
from osgeo import gdal, gdalconst
import os
import numpy as np

def crop_tiff(input_tiff, output_folder, crop_width, crop_height):
    # 打开输入的TIFF文件
    dataset = gdal.Open(input_tiff, gdalconst.GA_ReadOnly)
    if dataset is None:
        print("无法打开TIFF文件！")
        return

    # 获取TIFF文件的基本信息
    width = dataset.RasterXSize
    height = dataset.RasterYSize
    bands = dataset.RasterCount
    driver = gdal.GetDriverByName("GTiff")
    if driver is None:
        print("未找到GTiff驱动程序！")
        return
    transform = dataset.GetGeoTransform()
    projection = dataset.GetProjection()

    # 计算裁剪后的栅格数量
    num_cols = (width + crop_width - 1) // crop_width  # 向上取整
    num_rows = (height + crop_height - 1) // crop_height  # 向上取整
    print(f'num_rows: {num_rows}')
    print(f'num_cols: {num_cols}')

    # 逐个栅格裁剪并保存
    num_saved_files = 0
    for i in range(num_rows):
        for j in range(num_cols):
            x_offset = j * crop_width
            y_offset = i * crop_height
            x_end = min(x_offset + crop_width, width)
            y_end = min(y_offset + crop_height, height)

            # 计算裁剪块的地理坐标
            min_lon = transform[0] + x_offset * transform[1]
            max_lon = transform[0] + x_end * transform[1]
            min_lat = transform[3] + y_offset * transform[5]
            max_lat = transform[3] + y_end * transform[5]

            # 检查栅格中的像素值是否全部为0
            is_all_zero = True
            for b in range(bands):
                band = dataset.GetRasterBand(b + 1)
                data = np.asarray(band.ReadAsArray(x_offset, y_offset, x_end - x_offset, y_end - y_offset))
                if not np.all(data == 0):
                    is_all_zero = False
                    break

            if not is_all_zero:
                output_tiff_path = os.path.join(output_folder, f"raster_{(min_lon):.1f}_{(max_lat):.1f}.tif")

                # 检查输出文件是否已经存在，并且文件大小是否小于10KB
                if os.path.exists(output_tiff_path):
                    if os.path.getsize(output_tiff_path) < 10 * 1024:  # 文件小于10KB
                        print(f"文件大小小于10KB，覆盖: {output_tiff_path}")
                    else:
                        print(f"文件已存在，跳过: {output_tiff_path}")
                        continue
                else:
                    print(f"创建新文件: {output_tiff_path}")

                new_dataset = driver.Create(output_tiff_path, x_end - x_offset, y_end - y_offset, bands, gdalconst.GDT_Byte)
                if new_dataset is None:
                    print(f"无法创建输出TIFF文件: {output_tiff_path}")
                    continue

                new_transform = (transform[0] + x_offset * transform[1], transform[1], transform[2],
                                 transform[3] + y_offset * transform[5], transform[4], transform[5])
                new_dataset.SetGeoTransform(new_transform)
                new_dataset.SetProjection(projection)
                for b in range(bands):
                    band = dataset.GetRasterBand(b + 1)
                    data = np.asarray(band.ReadAsArray(x_offset, y_offset, x_end - x_offset, y_end - y_offset))
                    new_dataset.GetRasterBand(b + 1).WriteArray(data)
                new_dataset = None
                num_saved_files += 1

    print(f"总共保存的文件数: {num_saved_files}")

    # 关闭输入的TIFF文件
    dataset = None


# 指定裁剪后的栅格大小
crop_width = 15000
crop_height = 15000

meter=crop_width*2

# 指定输入文件夹和输出文件夹路径
input_folder = rf'F:\全球25米及1米分辨率树冠高度数据'
output_folder = rf'H:\CANOPY\{meter}m_raster'
os.makedirs(output_folder, exist_ok=True)

# 遍历输入文件夹中的所有TIFF文件并进行裁剪
for file_name in os.listdir(input_folder):
    if file_name.endswith('.tif'):
        input_tiff = os.path.join(input_folder, file_name)
        print(f"正在处理文件: {input_tiff}")
        crop_tiff(input_tiff, output_folder, crop_width, crop_height)
'''

'\nfrom osgeo import gdal, gdalconst\nimport os\nimport numpy as np\n\ndef crop_tiff(input_tiff, output_folder, crop_width, crop_height):\n    # 打开输入的TIFF文件\n    dataset = gdal.Open(input_tiff, gdalconst.GA_ReadOnly)\n    if dataset is None:\n        print("无法打开TIFF文件！")\n        return\n\n    # 获取TIFF文件的基本信息\n    width = dataset.RasterXSize\n    height = dataset.RasterYSize\n    bands = dataset.RasterCount\n    driver = gdal.GetDriverByName("GTiff")\n    if driver is None:\n        print("未找到GTiff驱动程序！")\n        return\n    transform = dataset.GetGeoTransform()\n    projection = dataset.GetProjection()\n\n    # 计算裁剪后的栅格数量\n    num_cols = (width + crop_width - 1) // crop_width  # 向上取整\n    num_rows = (height + crop_height - 1) // crop_height  # 向上取整\n    print(f\'num_rows: {num_rows}\')\n    print(f\'num_cols: {num_cols}\')\n\n    # 逐个栅格裁剪并保存\n    num_saved_files = 0\n    for i in range(num_rows):\n        for j in range(num_cols):\n            x_offset = j * crop_width\n            y

# 10_01 Compute the latitude and longitude boundaries of the canopy dataset. Identify land parcels that intersect with the dataset. If multiple parcels contain relevant data, merge the files.

In [69]:
import os
import rasterio
import numpy as np
import geopandas as gpd
from rasterio.merge import merge
from rasterio.mask import mask
from shapely.geometry import box
from pyproj import Transformer

def get_shapefile_bounding_box(shp_path):
    """ 获取 Shapefile 的边界框，并转换到 WGS 84 (EPSG:4326) """
    gdf = gpd.read_file(shp_path)
    if gdf.crs is None:
        raise ValueError(f"Shapefile {shp_path} 缺少坐标系信息！")

    min_x, min_y, max_x, max_y = gdf.total_bounds
    crs = gdf.crs.to_string()

    # 如果坐标系不是 WGS 84，则转换
    if crs != 'EPSG:4326':
        transformer = Transformer.from_crs(crs, 'EPSG:4326', always_xy=True)
        min_x, min_y = transformer.transform(min_x, min_y)
        max_x, max_y = transformer.transform(max_x, max_y)

    return min_x, min_y, max_x, max_y

def get_raster_bounding_box(raster_path):
    """ 获取 TIFF 栅格文件的边界框，并转换到 WGS 84 (EPSG:4326) """
    try:
        with rasterio.open(raster_path) as src:
            if src.crs is None:
                raise ValueError(f"TIFF 文件 {raster_path} 缺少坐标系信息！")

            min_x, min_y, max_x, max_y = src.bounds
            crs = src.crs.to_string()

            # 如果坐标系不是 WGS 84，则转换
            if crs != 'EPSG:4326':
                transformer = Transformer.from_crs(crs, 'EPSG:4326', always_xy=True)
                min_x, min_y = transformer.transform(min_x, min_y)
                max_x, max_y = transformer.transform(max_x, max_y)

            return min_x, min_y, max_x, max_y

    except rasterio.errors.RasterioIOError as e:
        print(f"⚠️ 无法读取 {raster_path}，可能是损坏的 TIFF 文件，已跳过。错误: {e}")
        return None

def is_bbox_intersect(file_bbox, target_bbox, buffer=0.5):
    """ 判断文件的边界框是否与目标范围有交集 """
    f_min_x, f_min_y, f_max_x, f_max_y = file_bbox
    t_min_x, t_min_y, t_max_x, t_max_y = target_bbox

    # 目标范围扩展 buffer
    t_min_x -= buffer
    t_max_x += buffer
    t_min_y -= buffer
    t_max_y += buffer

    return not (f_max_x < t_min_x or f_min_x > t_max_x or
                f_max_y < t_min_y or f_min_y > t_max_y)

def find_intersecting_files(raster_folder, target_bbox):
    """ 查找与目标 bbox 相交的 TIFF 文件 """
    intersecting_files = []
    for file in os.listdir(raster_folder):
        if file.endswith(".tif") or file.endswith(".tiff"):
            raster_path = os.path.join(raster_folder, file)
            file_bbox = get_raster_bounding_box(raster_path)
            if file_bbox and is_bbox_intersect(file_bbox, target_bbox):
                print(file_bbox)
                print(file)
                intersecting_files.append(raster_path)
    return intersecting_files

from rasterio.warp import transform_bounds

def mask_and_merge_rasters(raster_files, output_path, target_bbox):
    """ 合并多个栅格文件并裁剪到 target_bbox，并只保留值为 80 的像素 """
    if not raster_files:
        print("❌ 没有找到相交的栅格文件，跳过处理。")
        return

    # 打开栅格文件
    src_files = [rasterio.open(fp) for fp in raster_files]
    print(f'{src_files} = src files')

    # 获取栅格文件的坐标系
    with rasterio.open(raster_files[0]) as ref_src:
        raster_crs = ref_src.crs
        print(f"📌 栅格文件 CRS: {raster_crs}")

    # 获取目标 shapefile 坐标系
    target_crs = "EPSG:4326"
    print(f"📌 目标 BBOX: {target_bbox}, CRS 需要转换: {raster_crs} → {target_crs}")

    if raster_crs != target_crs:
        print("⚠️ 正在转换 target_bbox 坐标系...")
        min_x, min_y, max_x, max_y = transform_bounds(raster_crs, target_crs, *target_bbox)

        target_bbox = (min_x, min_y, max_x, max_y)

    # 合并栅格文件
    mosaic, out_trans = merge(src_files)

    out_meta = src_files[0].meta.copy()

    out_meta.update({
        "driver": "GTiff",
        "count": 1,
        "height": mosaic.shape[1],
        "width": mosaic.shape[2],
        "transform": out_trans,
        "dtype": "float32",
        "nodata": np.nan
    })

    # 生成裁剪范围
    min_x, min_y, max_x, max_y = target_bbox
    bbox_geom = [box(min_x, min_y, max_x, max_y)]

    with rasterio.io.MemoryFile() as memfile:
        with memfile.open(**out_meta) as mem_raster:
            mem_raster.write(mosaic)

            # 使用目标坐标系进行裁剪
            bbox_mask, bbox_transform = mask(
                dataset=mem_raster,
                shapes=bbox_geom,
                crop=True
            )

    # 只保留 0 < 值 < 101 的数据
    data = bbox_mask[0].astype(float)
    data[(data <= 0) | (data >= 40000)] = np.nan  # 0 和 >=101 的像素设为 NaN

    # 如果整个栅格都是无效值，则不保存
    if np.all(data == np.nan):
        print(f"🚫 {output_path} 仅包含 nodata 值，已跳过。")
        return
    mosaic = mosaic.astype(np.float32)

    out_meta.update({
        "height": data.shape[0],
        "width": data.shape[1],
        "transform": bbox_transform,
        "dtype": mosaic.dtype.name,
        "nodata": np.nan
    })

    # 保存裁剪后的栅格文件
    with rasterio.open(output_path, "w", compress="LZW", **out_meta) as dest:
        dest.write(data, 1)

    print(f"✅ 已保存: {output_path}")

# 批量处理多个地区
target_workspace = r'D:\file'
raster_workspace = rf'F:\全球25米及1米分辨率树冠高度数据'

for place2 in places2:
    # 你的代码逻辑
    print(place2)

    shp_file = os.path.join(target_workspace, rf'a_clip_data\city_SHP\a_city_split\polygon_{place2}.shp')
    output_workspace = os.path.join(target_workspace, rf"d_zonalst\{place2}")
    os.makedirs(output_workspace, exist_ok=True)

    output_path = os.path.join(output_workspace, f'{place2}_canopy_merged_output.tif')

    # if os.path.exists(output_path):
    #     print(f"⚠️ 文件已存在，跳过: {output_path}")
    #     continue

    target_bbox = get_shapefile_bounding_box(shp_file)
    print(f'📍 目标区域 {place2} 边界: {target_bbox}')

    intersecting_files = find_intersecting_files(raster_workspace, target_bbox)

    if intersecting_files:
        print(f"📂 {place2} 相交的栅格文件: {len(intersecting_files)} 个")
        mask_and_merge_rasters(intersecting_files, output_path, target_bbox)
    else:
        print(f"🚫 {place2} 没有相交的栅格文件，跳过")

10.0
10.0
100.0
Processing feature 1/4116
总area是25.135922330097088
Processing feature 2/4116
总area是36.99551799206044
Processing feature 3/4116
总area是37.182552083333334
Processing feature 4/4116
总area是42.09161109161109
Processing feature 5/4116
总area是41.65086759054385
Processing feature 6/4116
总area是40.13912420952153
Processing feature 7/4116
总area是34.74526555603018
Processing feature 8/4116
总area是37.916438014467445
Processing feature 9/4116
总area是43.59111524595883
Processing feature 10/4116
总area是41.58424482196882
Processing feature 11/4116
总area是43.70802919708029
Processing feature 12/4116
总area是39.64244256348247
Processing feature 13/4116
总area是35.81462212486309
Processing feature 14/4116
总area是35.516019796822086
Processing feature 15/4116
总area是32.6754055807917
Processing feature 16/4116
总area是38.01507289350136
Processing feature 17/4116
总area是43.683239677498165
Processing feature 18/4116
总area是39.73823460547881
Processing feature 19/4116
总area是43.65536591068506
Processing feature 2


KeyboardInterrupt



In [13]:
import os
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling

target_workspace = r"D:\file/d_zonalst"       # 目标输出根目录
places2 = os.listdir(target_workspace)       # 或者你已有区域列表

dst_crs = "ESRI:54009"  # 目标投影

for place2 in places2:
    print("Processing region:", place2)
    # 构造输出文件夹
    output_workspace = os.path.join(target_workspace, f"{place2}")
    os.makedirs(output_workspace, exist_ok=True)

    # 构造输入和输出文件路径
    # 假设输入栅格文件命名为：<place2>_canopy_merged_output.tif，在 source_workspace 下
    input_raster = os.path.join(output_workspace, f"{place2}_canopy_merged_output.tif")
    # 输出栅格文件放在目标文件夹中，名称不变
    output_raster = os.path.join(output_workspace, f"{place2}_canopy_merged_output_54009.tif")

    # 重新投影
    with rasterio.open(input_raster) as src:
        transform, width, height = calculate_default_transform(src.crs, dst_crs, src.width, src.height, *src.bounds)
        kwargs = src.meta.copy()
        kwargs.update({
            'crs': dst_crs,
            'transform': transform,
            'width': width,
            'height': height
        })
        with rasterio.open(output_raster, 'w', **kwargs) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=dst_crs,
                    resampling=Resampling.nearest
                )
    print("Reprojected raster saved to:", output_raster)


Processing region: 0
Reprojected raster saved to: D:\file/d_zonalst\0\0_canopy_merged_output_54009.tif
Processing region: 1
Reprojected raster saved to: D:\file/d_zonalst\1\1_canopy_merged_output_54009.tif
Processing region: 10
Reprojected raster saved to: D:\file/d_zonalst\10\10_canopy_merged_output_54009.tif
Processing region: 100
Reprojected raster saved to: D:\file/d_zonalst\100\100_canopy_merged_output_54009.tif
Processing region: 101
Reprojected raster saved to: D:\file/d_zonalst\101\101_canopy_merged_output_54009.tif
Processing region: 103
Reprojected raster saved to: D:\file/d_zonalst\103\103_canopy_merged_output_54009.tif
Processing region: 104
Reprojected raster saved to: D:\file/d_zonalst\104\104_canopy_merged_output_54009.tif
Processing region: 105
Reprojected raster saved to: D:\file/d_zonalst\105\105_canopy_merged_output_54009.tif
Processing region: 106
Reprojected raster saved to: D:\file/d_zonalst\106\106_canopy_merged_output_54009.tif
Processing region: 107
Reprojected

In [42]:
import os
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling

target_workspace = r"D:\file/d_zonalst"       # 目标输出根目录
places2 = os.listdir(target_workspace)       # 或者你已有区域列表

dst_crs = "ESRI:54008"  # 目标投影

for place2 in places2:
    print("Processing region:", place2)
    # 构造输出文件夹
    output_workspace = os.path.join(target_workspace, f"{place2}")
    os.makedirs(output_workspace, exist_ok=True)

    # 构造输入和输出文件路径
    # 假设输入栅格文件命名为：<place2>_canopy_merged_output.tif，在 source_workspace 下
    input_raster = os.path.join(output_workspace, f"{place2}_canopy_merged_output.tif")
    # 输出栅格文件放在目标文件夹中，名称不变
    output_raster = os.path.join(output_workspace, f"{place2}_canopy_merged_output_54008.tif")

    # 重新投影
    with rasterio.open(input_raster) as src:
        transform, width, height = calculate_default_transform(src.crs, dst_crs, src.width, src.height, *src.bounds)
        kwargs = src.meta.copy()
        kwargs.update({
            'crs': dst_crs,
            'transform': transform,
            'width': width,
            'height': height
        })
        with rasterio.open(output_raster, 'w', **kwargs) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=dst_crs,
                    resampling=Resampling.nearest
                )
    print("Reprojected raster saved to:", output_raster)


Processing region: 0
Reprojected raster saved to: D:\file/d_zonalst\0\0_canopy_merged_output_54008.tif
Processing region: 1
Reprojected raster saved to: D:\file/d_zonalst\1\1_canopy_merged_output_54008.tif
Processing region: 10
Reprojected raster saved to: D:\file/d_zonalst\10\10_canopy_merged_output_54008.tif
Processing region: 100
Reprojected raster saved to: D:\file/d_zonalst\100\100_canopy_merged_output_54008.tif
Processing region: 101
Reprojected raster saved to: D:\file/d_zonalst\101\101_canopy_merged_output_54008.tif
Processing region: 103
Reprojected raster saved to: D:\file/d_zonalst\103\103_canopy_merged_output_54008.tif
Processing region: 104
Reprojected raster saved to: D:\file/d_zonalst\104\104_canopy_merged_output_54008.tif
Processing region: 105
Reprojected raster saved to: D:\file/d_zonalst\105\105_canopy_merged_output_54008.tif
Processing region: 106
Reprojected raster saved to: D:\file/d_zonalst\106\106_canopy_merged_output_54008.tif
Processing region: 107
Reprojected

# 10_02 Perform zonal statistics to compute canopy density and canopy coverage ratio.

In [70]:
import geopandas as gpd
# 设置输入文件夹和输出文件路径
input_folder = r"D:\file\d_zonalst"

for place2 in os.listdir(input_folder):
    # 定义输入矢量文件路径
    input_vector = fr"D:\file\d_zonalst\{place2}\{place2}_output_file(MEAN_BD_FAR).shp"

    # 读取 shapefile
    gdf = gpd.read_file(input_vector)
    #print("Original CRS:", gdf.crs)

    # 如果数据不是投影坐标系，则转换到 ESRI:54008 投影

    gdf = gdf.to_crs("ESRI:54008")
    #print("Reprojected CRS:", gdf.crs)

    # 如果需要保存结果到新文件，也可以：
    output_vector = rf"D:\file\d_zonalst\{place2}\{place2}_output_file(MEAN_BD_FAR)_54008.shp"
    gdf.to_file(output_vector)
    print("Output saved to:", output_vector)


Original CRS: PROJCS["World_Mollweide",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mollweide"],PARAMETER["central_meridian",0],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["ESRI","54009"]]
Reprojected CRS: ESRI:54008
Output saved to: D:\file\d_zonalst\0\0_output_file(MEAN_BD_FAR)_54008.shp
Original CRS: PROJCS["World_Mollweide",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mollweide"],PARAMETER["central_meridian",0],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["ESRI","54009"]]
Re

In [71]:
import pyproj
from pyproj import Transformer
from rasterio.mask import mask
import geopandas as gpd
import os
import rasterio
import numpy as np
import pandas as pd
import traceback
def remove_nodata2(data, no_data):
    if no_data is not None:
        # Create a mask for NoData values and NaNs
        mask = (data == no_data) | np.isnan(data) | (data == 0)
    else:
        # Create a mask for NaNs and zeros
        mask = np.isnan(data) | (data == 0)

    valid_data = data[~mask]
    return valid_data


def create_show_progress(threshold=10):
    """
    Create a progress callback function that only prints progress
    when the percentage increases by the specified threshold.
    :param threshold: Percentage threshold for printing progress
    :return: Progress callback function
    """
    last_percentage = -threshold

    def Show_Progress(complete, message, unknown):
        nonlocal last_percentage
        percentage = int(complete * 100)
        if percentage >= last_percentage + threshold:
            #print(f"Progress: {percentage}% - {message}")
            last_percentage = percentage

    return Show_Progress
def calculate_pixel_size_in_meters(raster_path, target_crs_str="ESRI:54008"):
    """
    利用中心像元的角点坐标，将其转换到目标投影后，
    计算像元的宽度、高度及面积（平行四边形假设）。
    """
    with rasterio.open(raster_path) as src:
        height, width = src.height, src.width
        center_row = height // 2
        center_col = width // 2

        # 获取中心像元的左上 (ul)、右上 (ur) 和左下 (ll) 角点坐标
        ul = src.xy(center_row, center_col, offset="ul")
        ur = src.xy(center_row, center_col, offset="ur")
        ll = src.xy(center_row, center_col, offset="ll")

        # 创建目标 CRS 对象
        target_crs = CRS.from_string(target_crs_str)
        transformer = Transformer.from_crs(src.crs, target_crs, always_xy=True)

        # 将角点坐标转换到目标投影
        ul_t = transformer.transform(ul[0], ul[1])
        ur_t = transformer.transform(ur[0], ur[1])
        ll_t = transformer.transform(ll[0], ll[1])

        # 计算转换后像元宽度和高度
        pixel_width = abs(ur_t[0] - ul_t[0])
        pixel_height = abs(ul_t[1] - ll_t[1])
        pixel_area = pixel_width * pixel_height

        return pixel_width, pixel_height, pixel_area


def zonal_statistics(raster_path, vector_path, output_path):
    try:
        # Calculate pixel size in meters
        # pixel_size = calculate_pixel_size_in_meters(raster_path)
        # print(f"Pixel size (meters): {pixel_size}")
        pixel_size_x, pixel_size_y, pixel_area = calculate_pixel_size_in_meters(raster_path)


        print(f"Pixel size (meters): {pixel_size_x}, {pixel_size_y}")
        print(f"Pixel area: {pixel_area}")

        # Open the raster file and get NoData value
        with rasterio.open(raster_path) as raster:
            no_data = raster.nodata
            raster_crs = raster.crs
            # print(f"Raster CRS: {raster_crs}, NoData value: {no_data}")

        # Open the vector file
        vector = gpd.read_file(vector_path)

        # 如果 raster 文件不存在，直接写入 NaN 并返回
        if not os.path.exists(raster_path):
            print(f"Raster file {raster_path} not found. Setting GreenArea, GreenHeight, GreenRatio, GreenVolRatio to NoData.")
            vector["GreenArea"] = -9999  # 赋值 NaN
            vector["GrHeight"] = -9999  # 赋值 NaN
            vector["GreenRatio"] = -9999  # 赋值 NaN
            vector["GrVolRatio"] = -9999  # 赋值 NaN
            vector.to_file(output_path)  # 仍然保存
            return

        vector_crs = vector.crs
        print(f"Vector CRS: {vector_crs}")

        # # Reproject vector data if necessary
        # if vector_crs != raster_crs:
        #     vector = vector.to_crs(raster_crs)
        #     print(f"Reprojected vector data to raster CRS{raster_crs}.")

        # Initialize progress callback
        show_progress = create_show_progress(threshold=10)
        total_zones = vector.shape[0]

        results = []
        for idx, row in vector.iterrows():
            geom = row['geometry']
            #print(f"Processing feature {idx+1}/{total_zones}")

            # Mask the raster using the geometry
            with rasterio.open(raster_path) as raster:
                out_image, out_transform = mask(raster, [geom], crop=True)
                #print(f"Masked 图像 shape: {out_image.shape}, NoData: {no_data}")

                # Remove NoData values
                data = out_image[0]
                valid_data = remove_nodata2(data, no_data)

                #print(f"Masked data min: {np.nanmin(valid_data)}, max: {np.nanmax(valid_data)}")

                # Calculate statistics
                if valid_data.size > 0:
                    mean_val = float(np.nanmean(valid_data) / 100) # cm --> m
                    count_val = valid_data.size
                    area_val = count_val * pixel_area  # Pixel size squared
                    #print(area_val)
                else:
                    mean_val = area_val = -9999

                stats = {
                    'GreenArea': area_val,
                    'GrHeight': mean_val,
                }
                results.append(stats)

            # Update progress
            completed = (idx + 1) / total_zones
            #show_progress(completed, 'Processing zones', None)

        # Create DataFrame to store results
        stats_df = pd.DataFrame(results)
        vector = vector.join(stats_df)

        # Add new fields
        vector['GreenRatio'] = np.where(vector['GreenArea'] == -9999, -9999, vector['GreenArea'] / vector['Landarea'])  # Assuming 'cell' is provided correctly
        print(vector['GreenRatio'])
        vector['GrVolRatio'] = np.where(np.logical_or(vector['GreenArea'] == -9999 , vector['GrHeight'] == -9999 ),
                                      -9999,
                                      vector['GreenRatio'] * vector['GrHeight'])  # Adjust as needed

        vector.rename(columns={'Landarea': 'LandArea'}, inplace=True)
        # Save results to a new shapefile
        vector.to_file(output_path)
        print(f"Zonal statistics saved to: {output_path}")



    except Exception as e:
        print(f"Error calculating zonal statistics: {e}")
        traceback.print_exc()
#
# # 设置输入文件夹和输出文件路径
# input_folder = r"D:\file\d_zonalst"
#
# for place2 in os.listdir(input_folder):
#     workspace = rf'D:\file\d_zonalst\{place2}'
#     masked_tif =os.path.join(workspace, f'{place2}_canopy_merged_output_54009.tif')
#     if not os.path.exists(masked_tif):
#         print(f"文件不存在: {masked_tif}")
#     vector_path3 = os.path.join(workspace, f'{place2}_output_file(MEAN_BD_FAR).shp')
#     output_shp3 = os.path.join(workspace, f'{place2}_output_file(MEAN_BD_FAR_CD_CR).shp')
#
#     # if os.path.exists(output_shp3):
#     #      print(f'{output_shp3}已经存在，跳过')
#     #      continue
#
#     zonal_statistics(masked_tif, vector_path3, output_shp3)

input_folder = r"D:\file\d_zonalst"

for place2 in os.listdir(input_folder):
    workspace = rf'D:\file\d_zonalst\{place2}'
    masked_tif =os.path.join(workspace, f'{place2}_canopy_merged_output_54008.tif')
    if not os.path.exists(masked_tif):
        print(f"文件不存在: {masked_tif}")
    vector_path3 = os.path.join(workspace, f'{place2}_output_file(MEAN_BD_FAR)_54008.shp')
    output_shp3 = os.path.join(workspace, f'{place2}_output_file(MEAN_BD_FAR_CD_CR).shp')

    # if os.path.exists(output_shp3):
    #      print(f'{output_shp3}已经存在，跳过')
    #      continue

    zonal_statistics(masked_tif, vector_path3, output_shp3)

Pixel size (meters): 23.455409327230882, 23.455409326590598
Pixel area: 550.1562266929313
Vector CRS: PROJCS["World_Sinusoidal",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["Sinusoidal"],PARAMETER["longitude_of_center",0],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["ESRI","54008"]]
0      0.547243
1      0.494180
2      0.732001
3      0.823421
4      0.784424
         ...   
527    0.806160
528    0.679578
529    0.592633
530    0.500574
531    0.704511
Name: GreenRatio, Length: 532, dtype: float64
Zonal statistics saved to: D:\file\d_zonalst\0\0_output_file(MEAN_BD_FAR_CD_CR).shp
Pixel size (meters): 23.217065943870693, 23.217065943405032
Pixel area: 539.0321510312291
Vector CRS: PROJCS["World_Sinusoidal",GEOGCS["WGS 84",DATUM["WG

In [68]:
import os
import rasterio
from rasterio.mask import mask
import geopandas as gpd

# 示例：定义输入栅格、矢量和输出栅格路径
input_raster =r'D:\file\d_zonalst\264\264_canopy_merged_output_54008.tif'
input_vector = r"D:\file\d_zonalst\264\264_output_file(MEAN_BD_FAR_CD_CR).shp"
output_raster = r"D:\file\d_zonalst\264\264_clipped_raster.tif"

# 读取矢量文件，这里假设我们取第一个要素
vector = gpd.read_file(input_vector)
geom = vector.iloc[60].geometry

# 裁剪栅格
with rasterio.open(input_raster) as src:
    out_image, out_transform = mask(src, [geom], crop=True, all_touched=False)
    out_meta = src.meta.copy()

    # 更新元数据：新的高度、宽度和仿射变换
    out_meta.update({
        "driver": "GTiff",
        "height": out_image.shape[1],
        "width": out_image.shape[2],
        "transform": out_transform
    })

    # 写入输出栅格
    with rasterio.open(output_raster, "w", **out_meta) as dest:
        dest.write(out_image)

print("Clipped raster saved to:", output_raster)


Clipped raster saved to: D:\file\d_zonalst\264\264_clipped_raster.tif


In [69]:
import rasterio
import numpy as np
from pyproj import Transformer, CRS

def calculate_pixel_size_in_meters(raster_path, target_crs_str="ESRI:54009"):
    """
    通过获取中心像元的三个角点，在目标投影下计算像元的宽度、高度及面积（平行四边形假设）。
    """
    with rasterio.open(raster_path) as src:
        height, width = src.height, src.width
        center_row = height // 2
        center_col = width // 2

        # 获取中心像元的左上 (ul)、右上 (ur) 和左下 (ll) 角点坐标
        ul = src.xy(center_row, center_col, offset="ul")
        ur = src.xy(center_row, center_col, offset="ur")
        ll = src.xy(center_row, center_col, offset="ll")

        # 创建目标 CRS 对象
        target_crs = CRS.from_string(target_crs_str)
        transformer = Transformer.from_crs(src.crs, target_crs, always_xy=True)

        # 将角点转换到目标投影
        ul_t = transformer.transform(ul[0], ul[1])
        ur_t = transformer.transform(ur[0], ur[1])
        ll_t = transformer.transform(ll[0], ll[1])

        # 计算像元宽度和高度（平行四边形假设）
        pixel_width = abs(ur_t[0] - ul_t[0])
        pixel_height = abs(ul_t[1] - ll_t[1])
        pixel_area = pixel_width * pixel_height

        return pixel_width, pixel_height, pixel_area

# 定义裁剪后栅格的路径（例如保存了一个小区域的栅格）
raster_path = r"D:\file\d_zonalst\264\264_clipped_raster.tif"

# 读取裁剪栅格，统计有效像元数量
with rasterio.open(raster_path) as src:
    data = src.read(1)
    nodata = src.nodata
    if nodata is not None:
        valid_data = data[(data > 0)] #valid_data = data[(data != nodata) & (data != 0)]
    else:
        valid_data = data[data != 0]  # 如果不排除 0，请去掉判断valid_data = data[data != 0]
    count_pixels = valid_data.size

# 获取像元尺寸和面积
px_width, px_height, px_area = calculate_pixel_size_in_meters(raster_path)
total_area = count_pixels * px_area

print("Valid pixel count: {}".format(count_pixels))
print("Pixel width: {:.3f} m, Pixel height: {:.3f} m, Pixel area: {:.3f} m^2".format(px_width, px_height, px_area))
print("Total area (based on valid pixels): {:.3f} m^2".format(total_area))


Valid pixel count: 725
Pixel width: 33.571 m, Pixel height: 36.427 m, Pixel area: 1222.897 m^2
Total area (based on valid pixels): 886599.974 m^2


# 11_00 merge file with city and refer

In [44]:
import pandas as pd
import os
import geopandas as gpd

# 读取CSV文件
csv_file = r'D:\file\a_clip_data\匹配问题\near_id_match.csv'
df = pd.read_csv(csv_file)

# 提取 FID 和 NEAR_FID 列
places = df['FID'].tolist()
places2 = df['NEAR_FID'].tolist()

# 参考矢量数据文件夹
city_shape_folder = r'D:\file\a_clip_data\city_SHP\a_city_split'
refer_shape_folder = r'D:\file\a_clip_data\city_refer_SHP\a_city_split'
output_shape_folder = r'D:\file\a_clip_data\c_merge'

# 确保输出文件夹存在
os.makedirs(output_shape_folder, exist_ok=True)

# 逐个合并 shapefile
for place, place2 in zip(places, places2):
    print(f"Processing: {place} with reference {place2}")

    city_shape_file = os.path.join(city_shape_folder, f'polygon_{place}.shp')
    refer_shape_file = os.path.join(refer_shape_folder, f'polygon_{place2}.shp')
    merge_file = os.path.join(output_shape_folder, f'polygon_{place}.shp')

    # 读取矢量文件
    city_gdf = gpd.read_file(city_shape_file)
    refer_gdf = gpd.read_file(refer_shape_file)

    # 统一 CRS
    if city_gdf.crs != refer_gdf.crs:
        refer_gdf = refer_gdf.to_crs(city_gdf.crs)

    # 合并两个数据集
    merged_gdf = gpd.GeoDataFrame(pd.concat([city_gdf, refer_gdf], ignore_index=True))

    # 确保 geometry 存在
    if 'geometry' not in merged_gdf.columns or merged_gdf.is_empty.any():
        print(f"Error: {merge_file} has empty geometry!")
        continue

    # **融合几何：合并相交或相邻的多边形**
    merged_geometry = merged_gdf.unary_union  # 计算融合后的几何
    merged_gdf = gpd.GeoDataFrame(geometry=[merged_geometry], crs=city_gdf.crs)

    # 保存合并后的文件
    merged_gdf.to_file(merge_file, driver="ESRI Shapefile")
    print(f"Saved: {merge_file}")


Processing: 0 with reference 0
Saved: D:\file\a_clip_data\c_merge\polygon_0.shp
Processing: 1 with reference 1
Saved: D:\file\a_clip_data\c_merge\polygon_1.shp
Processing: 2 with reference 2
Saved: D:\file\a_clip_data\c_merge\polygon_2.shp
Processing: 3 with reference 3
Saved: D:\file\a_clip_data\c_merge\polygon_3.shp
Processing: 4 with reference 4
Saved: D:\file\a_clip_data\c_merge\polygon_4.shp
Processing: 5 with reference 5
Saved: D:\file\a_clip_data\c_merge\polygon_5.shp
Processing: 6 with reference 6
Saved: D:\file\a_clip_data\c_merge\polygon_6.shp
Processing: 7 with reference 7
Saved: D:\file\a_clip_data\c_merge\polygon_7.shp
Processing: 8 with reference 8
Saved: D:\file\a_clip_data\c_merge\polygon_8.shp
Processing: 9 with reference 9
Saved: D:\file\a_clip_data\c_merge\polygon_9.shp
Processing: 10 with reference 110
Saved: D:\file\a_clip_data\c_merge\polygon_10.shp
Processing: 11 with reference 111
Saved: D:\file\a_clip_data\c_merge\polygon_11.shp
Processing: 12 with reference 11

# 11_01 water


In [2]:
import pandas as pd

# 读取CSV文件
csv_file = r'D:\file\a_clip_data\匹配问题\near_id_match.csv'
df = pd.read_csv(csv_file)

# 提取 FID 和 NEAR_FID 列
places = df['FID'].tolist()
places2 = df['NEAR_FID'].tolist()

# 分别打印 places 和 places2
print("Places (建成区的label):", places)
print("Places2 (参考区的label):", places2)
print(len(places))
print(len(places2))

csv_file2 = r'D:\file\a_clip_data\匹配问题\FID_city_match.csv'
df2 = pd.read_csv(csv_file2)
places_names = df2['UC_NM_MN'].tolist()
print(places_names)
print(len(places_names))

Places (建成区的label): [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217,

# 11_02 water get the tif file

In [22]:
import os
import rasterio
import numpy as np
import geopandas as gpd
from rasterio.merge import merge
from rasterio.mask import mask
from shapely.geometry import box
from pyproj import Transformer

def get_shapefile_bounding_box(shp_path):
    """ 获取 Shapefile 的边界框，并转换到 WGS 84 (EPSG:4326) """
    gdf = gpd.read_file(shp_path)
    if gdf.crs is None:
        raise ValueError(f"Shapefile {shp_path} 缺少坐标系信息！")

    min_x, min_y, max_x, max_y = gdf.total_bounds
    crs = gdf.crs.to_string()

    # 如果坐标系不是 WGS 84，则转换
    if crs != 'EPSG:4326':
        transformer = Transformer.from_crs(crs, 'EPSG:4326', always_xy=True)
        min_x, min_y = transformer.transform(min_x, min_y)
        max_x, max_y = transformer.transform(max_x, max_y)

    return min_x, min_y,max_x, max_y

def get_raster_bounding_box(raster_path):
    """ 获取 TIFF 栅格文件的边界框，并转换到 WGS 84 (EPSG:4326) """
    try:
        with rasterio.open(raster_path) as src:
            if src.crs is None:
                raise ValueError(f"TIFF 文件 {raster_path} 缺少坐标系信息！")

            min_x, min_y, max_x, max_y = src.bounds
            crs = src.crs.to_string()

            # 如果坐标系不是 WGS 84，则转换
            if crs != 'EPSG:4326':
                transformer = Transformer.from_crs(crs, 'EPSG:4326', always_xy=True)
                min_x, min_y = transformer.transform(min_x, min_y)
                max_x, max_y = transformer.transform(max_x, max_y)

            return min_x, min_y,max_x, max_y
    except rasterio.errors.RasterioIOError as e:
        print(f"⚠️ 无法读取 {raster_path}，可能是损坏的 TIFF 文件，已跳过。错误: {e}")
        return None

def is_bbox_intersect(file_bbox, target_bbox, buffer=0):
    """ 判断文件的边界框是否与目标范围有交集 """
    f_min_x, f_min_y, f_max_x, f_max_y = file_bbox
    t_min_x, t_min_y, t_max_x, t_max_y = target_bbox

    # 目标范围扩展 buffer
    t_min_x -= buffer
    t_max_x += buffer
    t_min_y -= buffer
    t_max_y += buffer

    return not (f_max_x < t_min_x or f_min_x > t_max_x or
                f_max_y < t_min_y or f_min_y > t_max_y)

def find_intersecting_files(raster_folder, target_bbox):
    """ 查找与目标 bbox 相交的 TIFF 文件 """
    intersecting_files = []
    for file in os.listdir(raster_folder):
        if file.endswith(".tif") or file.endswith(".tiff"):
            raster_path = os.path.join(raster_folder, file)
            file_bbox = get_raster_bounding_box(raster_path)
            if file_bbox and is_bbox_intersect(file_bbox, target_bbox):
                print(file_bbox)
                print(file)
                intersecting_files.append(raster_path)
    return intersecting_files

from rasterio.warp import transform_bounds

def mask_and_merge_rasters(raster_files, output_path, target_bbox):
    """ 合并多个栅格文件并裁剪到 target_bbox，并只保留值为 80 的像素 """
    if not raster_files:
        print("❌ 没有找到相交的栅格文件，跳过处理。")
        return

    # 打开栅格文件
    src_files = [rasterio.open(fp) for fp in raster_files]
    print(f'{src_files} = src files')

    # 获取栅格文件的坐标系
    with rasterio.open(raster_files[0]) as ref_src:
        raster_crs = ref_src.crs

    # 如果目标 bbox 的坐标系与栅格文件的坐标系不同，进行转换
    # 假设 target_bbox 是以 WGS 84（EPSG:4326）为坐标系
    target_crs = "EPSG:4326"
    if raster_crs != target_crs:
        print("⚠️ 正在转换 target_bbox 坐标系...")
        min_x, min_y, max_x, max_y = transform_bounds(target_crs, raster_crs, *target_bbox)

        target_bbox = (min_x, min_y, max_x, max_y)

    # 合并栅格文件
    mosaic, out_trans = merge(src_files)

    out_meta = src_files[0].meta.copy()

    out_meta.update({
        "driver": "GTiff",
        "count": 1,
        "height": mosaic.shape[1],
        "width": mosaic.shape[2],
        "transform": out_trans,
        "dtype": "uint8",
        "nodata": 255
    })

    # 生成裁剪范围
    min_x, min_y, max_x, max_y = target_bbox
    bbox_geom = [box(min_x, min_y, max_x, max_y)]

    with rasterio.io.MemoryFile() as memfile:
        with memfile.open(**out_meta) as mem_raster:
            mem_raster.write(mosaic)

            # 使用目标坐标系进行裁剪
            bbox_mask, bbox_transform = mask(
                dataset=mem_raster,
                shapes=bbox_geom,
                crop=True
            )

    data = bbox_mask[0]
    data = np.where((data == 80), 80, 255).astype(np.uint8)

    # 如果整个栅格都是无效值，则不保存
    if np.all(data == 255):
        print(f"🚫 {output_path} 仅包含 nodata 值，已跳过。")
        return
    mosaic = mosaic.astype(np.uint8)

    out_meta.update({
        "height": data.shape[0],
        "width": data.shape[1],
        "transform": bbox_transform,
        "dtype": mosaic.dtype.name,
        "nodata": 255
    })

    # 保存裁剪后的栅格文件
    with rasterio.open(output_path, "w", compress="LZW", **out_meta) as dest:
        dest.write(data, 1)

    print(f"✅ 已保存: {output_path}")


# 批量处理多个地区
target_workspace = r'D:\file'
raster_workspace = r'C:\ESA'

# 设置输入文件夹和输出文件路径
input_folder = r"D:\file\d_zonalst"

for place in os.listdir(input_folder):
    shp_file = os.path.join(target_workspace, f'a_clip_data\\c_merge\\polygon_{place}.shp')
    output_workspace = os.path.join(target_workspace, f"d_water\\{place}")
    os.makedirs(output_workspace, exist_ok=True)

    output_path = os.path.join(output_workspace, f'water_{place}.tif')

    if os.path.exists(output_path):
        print(f"⚠️ 文件已存在，跳过: {output_path}")
        continue

    target_bbox = get_shapefile_bounding_box(shp_file)
    print(f'📍 目标区域 {place} 边界: {target_bbox}')

    intersecting_files = find_intersecting_files(raster_workspace, target_bbox)

    if intersecting_files:
        print(f"📂 {place} 相交的栅格文件: {len(intersecting_files)} 个")
        mask_and_merge_rasters(intersecting_files, output_path, target_bbox)
    else:
        print(f"🚫 {place} 没有相交的栅格文件，跳过")

⚠️ 文件已存在，跳过: D:\file\d_water\0\water_0.tif
⚠️ 文件已存在，跳过: D:\file\d_water\1\water_1.tif
⚠️ 文件已存在，跳过: D:\file\d_water\10\water_10.tif
⚠️ 文件已存在，跳过: D:\file\d_water\100\water_100.tif
⚠️ 文件已存在，跳过: D:\file\d_water\101\water_101.tif
⚠️ 文件已存在，跳过: D:\file\d_water\103\water_103.tif
⚠️ 文件已存在，跳过: D:\file\d_water\104\water_104.tif
⚠️ 文件已存在，跳过: D:\file\d_water\105\water_105.tif
⚠️ 文件已存在，跳过: D:\file\d_water\106\water_106.tif
⚠️ 文件已存在，跳过: D:\file\d_water\107\water_107.tif
⚠️ 文件已存在，跳过: D:\file\d_water\108\water_108.tif
⚠️ 文件已存在，跳过: D:\file\d_water\109\water_109.tif
⚠️ 文件已存在，跳过: D:\file\d_water\11\water_11.tif
⚠️ 文件已存在，跳过: D:\file\d_water\111\water_111.tif
⚠️ 文件已存在，跳过: D:\file\d_water\112\water_112.tif
⚠️ 文件已存在，跳过: D:\file\d_water\113\water_113.tif
⚠️ 文件已存在，跳过: D:\file\d_water\114\water_114.tif
⚠️ 文件已存在，跳过: D:\file\d_water\115\water_115.tif
⚠️ 文件已存在，跳过: D:\file\d_water\116\water_116.tif
⚠️ 文件已存在，跳过: D:\file\d_water\117\water_117.tif
⚠️ 文件已存在，跳过: D:\file\d_water\118\water_118.tif
⚠️ 文件已存在，跳过: D:\file\d_wa

# 11_03 tif -> shp file

In [23]:
import os
import rasterio
from rasterio.features import shapes
import numpy as np
import fiona
from shapely.geometry import shape, MultiPolygon
from shapely.ops import unary_union
from rasterio.warp import calculate_default_transform, reproject, Resampling

def raster_to_polygon(input_tif, output_shp, min_area=30000, target_crs="EPSG:3857"):
    """
    Convert a raster to a single merged polygon and save as a shapefile. Filters polygons smaller than `min_area`.
    :param input_tif: Path to the input raster file
    :param output_shp: Path to the output shapefile
    :param min_area: Minimum area (in square meters) to keep the polygons
    :param target_crs: The target coordinate reference system (CRS) to project to if necessary
    :return: None
    """
    try:
        with rasterio.open(input_tif) as src:
            # 获取原始坐标系
            src_crs = src.crs
            transform = src.transform

            # 打印原始数据的 CRS 和范围
            print(f"原始 CRS: {src_crs}")
            print(f"原始范围: {src.bounds}")

            # 如果输入是 WGS 84 (EPSG:4326)，则需要进行重投影
            if src_crs.to_epsg() == 4326:
                print(f"⚠️ 输入 TIFF 是 WGS 84，需要重投影到 {target_crs}...")

                # 计算重投影所需的 transform 和目标宽高
                transform, width, height = calculate_default_transform(
                    src.crs, target_crs, src.width, src.height, *src.bounds)
                kwargs = src.meta.copy()
                kwargs.update({'crs': target_crs, 'transform': transform, 'width': width, 'height': height})

                # 创建临时文件进行重投影
                with rasterio.open('temp_reprojected.tif', 'w', **kwargs) as dst:
                    for i in range(1, src.count + 1):
                        reproject(
                            source=rasterio.band(src, i),
                            destination=rasterio.band(dst, i),
                            src_transform=src.transform,
                            src_crs=src.crs,
                            dst_transform=transform,
                            dst_crs=target_crs,
                            resampling=Resampling.nearest
                        )

                # 打开重投影后的文件
                with rasterio.open('temp_reprojected.tif') as src_reprojected:
                    image = src_reprojected.read(1)
                    transform = src_reprojected.transform
                    # 打印重投影后的范围
                    print(f"重投影后的范围: {src_reprojected.bounds}")
                    print(f"重投影后的 CRS: {src_reprojected.crs}")

            else:
                image = src.read(1)

            # 获取 NoData 值
            no_data_value = src.nodata

            # 过滤无效区域
            mask = (image != no_data_value) & (~np.isnan(image)) if no_data_value is not None else ~np.isnan(image)
            image[np.isnan(image)] = 0  # NaN 设为 0
            image[np.isinf(image)] = 0  # Inf 设为 0
            image = np.round(image * 10000).astype(np.int32)  # 转为整数

            # 获取栅格对应的多边形
            polygons = [
                shape(s) for s, v in shapes(image, mask=mask, transform=transform)
                if v != no_data_value and not np.isnan(v)
            ]

            # 如果没有有效几何体，则不生成 shp
            if not polygons:
                print(f"跳过 {input_tif}，因为没有有效的矢量数据")
                return

            # 过滤面积小于 min_area 的多边形
            filtered_polygons = [
                geom for geom in polygons if geom.area >= min_area
            ]

            # 如果没有符合面积要求的区域，跳过该文件
            if not filtered_polygons:
                print(f"跳过 {input_tif}，因为没有符合面积要求的区域")
                return

            # 合并所有过滤后的 Polygon
            final_geom = unary_union(filtered_polygons)

            # 定义 Shapefile 的结构
            schema = {'geometry': 'MultiPolygon' if isinstance(final_geom, MultiPolygon) else 'Polygon',
                      'properties': {'value': 'int'}}

            # 写入 Shapefile
            with fiona.open(output_shp, 'w', driver='ESRI Shapefile', schema=schema, crs=target_crs) as shp:
                shp.write({'geometry': final_geom.__geo_interface__, 'properties': {'value': 1}})

        print(f"Shapefile 已创建: {output_shp}")

    except Exception as e:
        print(f"错误: {e}")


# 设置输入文件夹和输出文件路径
input_folder = r"D:\file\d_zonalst"

for place in os.listdir(input_folder):
    workspace = rf'D:\file\d_water\{place}'
    process_tif = os.path.join(workspace, f'water_{place}.tif')
    output_shp = os.path.join(workspace, f'water_{place}.shp')
    if os.path.exists(output_shp):
        print(f'{output_shp} already exists, continue')
        continue
    raster_to_polygon(process_tif, output_shp)


D:\file\d_water\0\water_0.shp already exists, continue
D:\file\d_water\1\water_1.shp already exists, continue
D:\file\d_water\10\water_10.shp already exists, continue
D:\file\d_water\100\water_100.shp already exists, continue
D:\file\d_water\101\water_101.shp already exists, continue
D:\file\d_water\103\water_103.shp already exists, continue
D:\file\d_water\104\water_104.shp already exists, continue
D:\file\d_water\105\water_105.shp already exists, continue
D:\file\d_water\106\water_106.shp already exists, continue
D:\file\d_water\107\water_107.shp already exists, continue
D:\file\d_water\108\water_108.shp already exists, continue
D:\file\d_water\109\water_109.shp already exists, continue
D:\file\d_water\11\water_11.shp already exists, continue
D:\file\d_water\111\water_111.shp already exists, continue
D:\file\d_water\112\water_112.shp already exists, continue
D:\file\d_water\113\water_113.shp already exists, continue
D:\file\d_water\114\water_114.shp already exists, continue
D:\file\d

# 11_04 shp -> tif file clean the 30000m2 file

In [35]:
import geopandas as gpd
import rasterio
from rasterio.features import rasterize
from shapely.geometry import mapping
import numpy as np

def shapefile_to_raster(shapefile, output_raster, pixel_size=10):
    """
    将 Shapefile 转换为 Raster 格式
    :param shapefile: 输入的 Shapefile 路径
    :param output_raster: 输出的 Raster 文件路径
    :param pixel_size: 栅格像素大小（单位：米）
    """
    # 读取 Shapefile 文件
    gdf = gpd.read_file(shapefile)

    # 获取 Shapefile 的边界
    bounds = gdf.total_bounds  # [minx, miny, maxx, maxy]
    minx, miny, maxx, maxy = bounds

    # 计算栅格的宽度和高度
    width = int((maxx - minx) / pixel_size)
    height = int((maxy - miny) / pixel_size)

    # 创建栅格文件的元数据
    transform = rasterio.transform.from_origin(minx, maxy, pixel_size, pixel_size)
    metadata = {
        'driver': 'GTiff',
        'count': 1,
        'dtype': 'int32',
        'crs': gdf.crs,
        'width': width,
        'height': height,
        'nodata': 0,
        'transform': transform
    }

    # 创建一个空的栅格数据
    with rasterio.open(output_raster, 'w', **metadata) as dst:
        # 将 Shapefile 中的几何体栅格化
        burned = rasterize(
            ((geom, 1) for geom in gdf.geometry),  # 以每个几何体为单元，值设为 1
            out_shape=(height, width),
            transform=transform,
            dtype='int32'
        )

        # 写入栅格数据
        dst.write(burned, 1)

    print(f"栅格文件已创建: {output_raster}")

# 设置输入文件夹和输出文件路径
input_folder = r"D:\file\d_zonalst"

for place in os.listdir(input_folder):
    workspace = rf'D:\file\d_water\{place}'

    EPSG3857_shp = os.path.join(workspace, f'water_{place}.shp')
    if os.path.exists(EPSG3857_shp):
        output84_tif = os.path.join(workspace, f'water_{place}_final.tif')
        if os.path.exists(output84_tif):
            print(f'{output84_tif} already exists, continue.')
            continue

        # 调用 polygon_to_raster 函数
        shapefile_to_raster(EPSG3857_shp, output84_tif)


栅格文件已创建: D:\file\d_water\0\water_0_final.tif
D:\file\d_water\1\water_1_final.tif already exists, continue.
D:\file\d_water\10\water_10_final.tif already exists, continue.
D:\file\d_water\100\water_100_final.tif already exists, continue.
D:\file\d_water\101\water_101_final.tif already exists, continue.
D:\file\d_water\103\water_103_final.tif already exists, continue.
D:\file\d_water\104\water_104_final.tif already exists, continue.
D:\file\d_water\105\water_105_final.tif already exists, continue.
D:\file\d_water\106\water_106_final.tif already exists, continue.
D:\file\d_water\107\water_107_final.tif already exists, continue.
D:\file\d_water\108\water_108_final.tif already exists, continue.
D:\file\d_water\109\water_109_final.tif already exists, continue.
D:\file\d_water\11\water_11_final.tif already exists, continue.
D:\file\d_water\111\water_111_final.tif already exists, continue.
D:\file\d_water\112\water_112_final.tif already exists, continue.
D:\file\d_water\113\water_113_final.tif

In [97]:
# def get_subfolder_names(parent_folder):
#     if not os.path.isdir(parent_folder):
#         raise FileNotFoundError("The directory {} does not exist".format(parent_folder))
#
#     # 获取所有子文件夹的名字
#     subfolders = [f for f in os.listdir(parent_folder) if os.path.isdir(os.path.join(parent_folder, f))]
#
#     return subfolders
#
# parent_folder = r'D:\file\a_clip_data\city_SHP\b_output_final_tem'
# places = get_subfolder_names(parent_folder)
# places2=[]
# for place in places:
#     place1= os.path.splitext(place)[0]
#     place2 = place1.split('_')[-1]
#     places2.append(place2)
#
# print(places2)

['0', '1', '10', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '11', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '12', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '13', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '14', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '15', '150', '151', '152', '153', '154', '155', '156', '157', '158', '159', '16', '160', '161', '162', '163', '164', '165', '166', '167', '168', '169', '17', '170', '171', '172', '173', '174', '175', '176', '177', '178', '179', '18', '180', '181', '182', '183', '184', '185', '186', '187', '188', '189', '19', '190', '191', '192', '193', '194', '195', '196', '197', '198', '199', '2', '20', '200', '201', '202', '203', '204', '205', '206', '207', '208', '209', '21', '210', '211', '212', '213', '214', '215', '216', '217', '218', '219', '22', '220', '221', '222', '223', '224', '225', '226', '227', '228', '22

# 11_05 arcpy
'''
这里打开Cost_DistWB.py
然后用python2.7运行,得到对应的cost distance
'''

In [ ]:
# -*- coding: utf-8 -*-
import arcpy
import os

# 设置输入文件夹和输出路径
input_folder = r"D:\file\d_zonalst"
target_workspace = r"D:\file"

# 遍历文件夹中的所有 `place2` 子文件夹
for place2 in os.listdir(input_folder):
    try:
        # 输入栅格文件路径
        input_raster = r"D:\file\d_water\{}\water_{}_final.tif".format(place2, place2)
        boundary_shapefile = os.path.join(target_workspace, r'a_clip_data\c_merge\polygon_{}.shp'.format(place2))

        # 输出栅格文件路径
        output_raster = r"D:\file\d_water\{}\water_{}_cost_DistWB.tif".format(place2, place2)

        # 检查输入文件是否存在
        if not arcpy.Exists(input_raster):
            print("❌ 输入栅格文件不存在: {}".format(input_raster))
            continue
        if not arcpy.Exists(boundary_shapefile):
            print("❌ 边界文件不存在: {}".format(boundary_shapefile))
            continue
        # if arcpy.Exists(output_raster):
        #     print("✅ {} 已存在，跳过计算".format(output_raster))
        #     continue

        # 设置环境变量，使计算的范围与 `boundary_shapefile` 一致
        arcpy.env.extent = boundary_shapefile  # 限制计算范围
        arcpy.env.mask = boundary_shapefile    # 只保留 `boundary_shapefile` 内的结果
        arcpy.env.snapRaster = input_raster    # 确保栅格对齐
        arcpy.env.cellSize = input_raster      # 让计算分辨率与 `input_raster` 一致

        # 计算欧式距离
        out_distance_raster = arcpy.sa.EucDistance(input_raster, None, None, None, "PLANAR", None, None)
        out_distance_raster.save(output_raster)

        print("🎯 欧式距离计算完成，并已保存：{}".format(output_raster))

    except Exception as e:
        print("❌ 处理 {} 时出错: {}".format(place2, e))

print("✅ 所有计算完成！")

# 11_06 Perform zonal statistics to compute DistWB.

In [74]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio
from rasterio.mask import mask

def remove_nodata2(data, no_data):
    if no_data is not None:
        # Create a mask for NoData values and NaNs
        mask = (data == no_data) | np.isnan(data)
    else:
        # Create a mask for NaNs and zeros
        mask = np.isnan(data)

    valid_data = data[~mask]
    return valid_data
def zonal_statistics_cost_distance(raster_path, vector_path, output_path):
    """
    Calculate zonal statistics for vector geometries based on cost distance raster data.
    :param raster_path: Path to the cost distance raster file
    :param vector_path: Path to the input vector (shapefile) file
    :param output_path: Path to the output shapefile with statistics
    :return: None
    """
    try:
        # Open the cost distance raster file and get NoData value
        with rasterio.open(raster_path) as raster:
            no_data = raster.nodata
            raster_crs = raster.crs
            #print(f"Raster CRS: {raster_crs}, NoData value: {no_data}")

        # Open the vector file
        vector = gpd.read_file(vector_path)
        # 如果 raster 文件不存在，直接写入 NaN 并返回
        if not os.path.exists(raster_path):
            print(f"Raster file {raster_path} not found. Setting MIN_DistWB to NoData.")
            vector["MIN_DistWB"] = -9999  # 赋值 NaN
            vector.to_file(output_path)  # 仍然保存
            return

        vector_crs = vector.crs
        #print(f"Vector CRS: {vector_crs}")

        # Reproject vector data if necessary
        if vector_crs != raster_crs:
            vector = vector.to_crs(raster_crs)
            #print("Reprojected vector data to raster CRS.")

        results = []
        for idx, row in vector.iterrows():
            geom = row['geometry']
            #print(f"Processing feature {idx + 1}/{vector.shape[0]}")

            # Mask the raster using the geometry
            with rasterio.open(raster_path) as raster:
                out_image, _ = mask(raster, [geom], crop=True)

                # Remove NoData values
                data = out_image[0]
                valid_data = remove_nodata2(data, no_data)

                # Calculate statistics
                if valid_data.size > 0:
                    min_val = float(np.nanmin(valid_data))
                else:
                    min_val = -9999

                stats = {
                    'MIN_DistWB': min_val
                }
                results.append(stats)

        # Create DataFrame to store results
        stats_df = pd.DataFrame(results)
        vector = vector.join(stats_df)

        # Save results to a new shapefile
        vector.to_file(output_path)
        print(f"Zonal statistics (cost distance) saved to: {output_path}")

    except Exception as e:
        print(f"Error calculating zonal statistics for cost distance: {e}")

# 设置输入文件夹和输出文件路径
input_folder = r"D:\file\d_zonalst"

# 遍历文件夹中的所有 `place2` 子文件夹
for place2 in os.listdir(input_folder):
    workspace = rf'D:\file\d_zonalst\{place2}'
    vector_path3 = os.path.join(workspace, f'{place2}_output_file(MEAN_BD_FAR_CD_CR).shp')
    cost_distance_tif = rf'D:\file\d_water\{place2}\water_{place2}_cost_DistWB.tif'  # 成本距离栅格路径
    output_shp_distwb = os.path.join(workspace, f'{place2}_output_file(MEAN_BD_FAR_CD_CR_DistWB).shp')

    # if os.path.exists(output_shp_distwb):
    #     print(f'{output_shp_distwb} already exists, continue')
    #     continue

    # 计算分区统计
    zonal_statistics_cost_distance(cost_distance_tif, vector_path3, output_shp_distwb)

Zonal statistics (cost distance) saved to: D:\file\d_zonalst\0\0_output_file(MEAN_BD_FAR_CD_CR_DistWB).shp
Zonal statistics (cost distance) saved to: D:\file\d_zonalst\1\1_output_file(MEAN_BD_FAR_CD_CR_DistWB).shp
Zonal statistics (cost distance) saved to: D:\file\d_zonalst\10\10_output_file(MEAN_BD_FAR_CD_CR_DistWB).shp
Zonal statistics (cost distance) saved to: D:\file\d_zonalst\100\100_output_file(MEAN_BD_FAR_CD_CR_DistWB).shp
Zonal statistics (cost distance) saved to: D:\file\d_zonalst\101\101_output_file(MEAN_BD_FAR_CD_CR_DistWB).shp
Zonal statistics (cost distance) saved to: D:\file\d_zonalst\103\103_output_file(MEAN_BD_FAR_CD_CR_DistWB).shp
Zonal statistics (cost distance) saved to: D:\file\d_zonalst\104\104_output_file(MEAN_BD_FAR_CD_CR_DistWB).shp
Zonal statistics (cost distance) saved to: D:\file\d_zonalst\105\105_output_file(MEAN_BD_FAR_CD_CR_DistWB).shp
Zonal statistics (cost distance) saved to: D:\file\d_zonalst\106\106_output_file(MEAN_BD_FAR_CD_CR_DistWB).shp
Zonal stati

# 11_07 Converged into one file

In [21]:
import geopandas as gpd
import os
import pandas as pd

# 设置输入文件夹和输出文件路径
input_folder = r"D:\file\d_zonalst"
output_shapefile = r"D:\file\e_index\223cities_output_file(MEAN_BD_FAR_CD_CR_DistWB).shp"

# 初始化一个空的 GeoDataFrame
merged_gdf = gpd.GeoDataFrame()

# 遍历文件夹中的所有 `place2` 子文件夹
for place2 in os.listdir(input_folder):
    input_shapefile = os.path.join(input_folder, place2, f"{place2}_output_file(MEAN_BD_FAR_CD_CR_DistWB).shp")

    # 检查 shapefile 是否存在
    if os.path.exists(input_shapefile):
        try:
            # 读取当前的 shapefile
            gdf = gpd.read_file(input_shapefile)

            # 检查 CRS，如果不是 WGS84（EPSG:4326），则转换
            if gdf.crs and gdf.crs.to_epsg() != 4326:
                gdf = gdf.to_crs(epsg=4326)
                print(f"{input_shapefile} 坐标系已转换为 WGS84 (EPSG:4326)。")

            # 合并数据
            merged_gdf = gpd.GeoDataFrame(pd.concat([merged_gdf, gdf], ignore_index=True))
            print(f"{input_shapefile} 已成功合并。")
        except Exception as e:
            print(f"无法读取 {input_shapefile}: {e}")
    else:
        print(f"文件 {input_shapefile} 不存在，跳过。")

# 保存最终合并的 shapefile
if not merged_gdf.empty:
    merged_gdf.to_file(output_shapefile, driver="ESRI Shapefile")
    print(f"所有文件已合并并保存到 {output_shapefile}（WGS84 坐标系）。")
else:
    print("没有有效的 shapefile 被合并。")


D:\file\d_zonalst\0\0_output_file(MEAN_BD_FAR_CD_CR_DistWB).shp 坐标系已转换为 WGS84 (EPSG:4326)。
D:\file\d_zonalst\0\0_output_file(MEAN_BD_FAR_CD_CR_DistWB).shp 已成功合并。
D:\file\d_zonalst\1\1_output_file(MEAN_BD_FAR_CD_CR_DistWB).shp 坐标系已转换为 WGS84 (EPSG:4326)。
D:\file\d_zonalst\1\1_output_file(MEAN_BD_FAR_CD_CR_DistWB).shp 已成功合并。
D:\file\d_zonalst\10\10_output_file(MEAN_BD_FAR_CD_CR_DistWB).shp 坐标系已转换为 WGS84 (EPSG:4326)。
D:\file\d_zonalst\10\10_output_file(MEAN_BD_FAR_CD_CR_DistWB).shp 已成功合并。
D:\file\d_zonalst\100\100_output_file(MEAN_BD_FAR_CD_CR_DistWB).shp 坐标系已转换为 WGS84 (EPSG:4326)。
D:\file\d_zonalst\100\100_output_file(MEAN_BD_FAR_CD_CR_DistWB).shp 已成功合并。
D:\file\d_zonalst\101\101_output_file(MEAN_BD_FAR_CD_CR_DistWB).shp 坐标系已转换为 WGS84 (EPSG:4326)。
D:\file\d_zonalst\101\101_output_file(MEAN_BD_FAR_CD_CR_DistWB).shp 已成功合并。
文件 D:\file\d_zonalst\102\102_output_file(MEAN_BD_FAR_CD_CR_DistWB).shp 不存在，跳过。
D:\file\d_zonalst\103\103_output_file(MEAN_BD_FAR_CD_CR_DistWB).shp 坐标系已转换为 WGS84 (EPSG:432

# 12_02 greenland get the tif file

In [26]:
import os
import rasterio
import numpy as np
import geopandas as gpd
from rasterio.merge import merge
from rasterio.mask import mask
from shapely.geometry import box
from pyproj import Transformer

def get_shapefile_bounding_box(shp_path):
    """ 获取 Shapefile 的边界框，并转换到 WGS 84 (EPSG:4326) """
    gdf = gpd.read_file(shp_path)
    if gdf.crs is None:
        raise ValueError(f"Shapefile {shp_path} 缺少坐标系信息！")

    min_x, min_y, max_x, max_y = gdf.total_bounds
    crs = gdf.crs.to_string()

    # 如果坐标系不是 WGS 84，则转换
    if crs != 'EPSG:4326':
        transformer = Transformer.from_crs(crs, 'EPSG:4326', always_xy=True)
        min_x, min_y = transformer.transform(min_x, min_y)
        max_x, max_y = transformer.transform(max_x, max_y)

    return min_x, min_y,max_x, max_y

def get_raster_bounding_box(raster_path):
    """ 获取 TIFF 栅格文件的边界框，并转换到 WGS 84 (EPSG:4326) """
    try:
        with rasterio.open(raster_path) as src:
            if src.crs is None:
                raise ValueError(f"TIFF 文件 {raster_path} 缺少坐标系信息！")

            min_x, min_y, max_x, max_y = src.bounds
            crs = src.crs.to_string()

            # 如果坐标系不是 WGS 84，则转换
            if crs != 'EPSG:4326':
                transformer = Transformer.from_crs(crs, 'EPSG:4326', always_xy=True)
                min_x, min_y = transformer.transform(min_x, min_y)
                max_x, max_y = transformer.transform(max_x, max_y)

            return min_x, min_y,max_x, max_y
    except rasterio.errors.RasterioIOError as e:
        print(f"⚠️ 无法读取 {raster_path}，可能是损坏的 TIFF 文件，已跳过。错误: {e}")
        return None

def is_bbox_intersect(file_bbox, target_bbox, buffer=0):
    """ 判断文件的边界框是否与目标范围有交集 """
    f_min_x, f_min_y, f_max_x, f_max_y = file_bbox
    t_min_x, t_min_y, t_max_x, t_max_y = target_bbox

    # 目标范围扩展 buffer
    t_min_x -= buffer
    t_max_x += buffer
    t_min_y -= buffer
    t_max_y += buffer

    return not (f_max_x < t_min_x or f_min_x > t_max_x or
                f_max_y < t_min_y or f_min_y > t_max_y)

def find_intersecting_files(raster_folder, target_bbox):
    """ 查找与目标 bbox 相交的 TIFF 文件 """
    intersecting_files = []
    for file in os.listdir(raster_folder):
        if file.endswith(".tif") or file.endswith(".tiff"):
            raster_path = os.path.join(raster_folder, file)
            file_bbox = get_raster_bounding_box(raster_path)
            if file_bbox and is_bbox_intersect(file_bbox, target_bbox):
                print(file_bbox)
                print(file)
                intersecting_files.append(raster_path)
    return intersecting_files

from rasterio.warp import transform_bounds

def mask_and_merge_rasters(raster_files, output_path, target_bbox):
    """ 合并多个栅格文件并裁剪到 target_bbox，并只保留值为 80 的像素 """
    if not raster_files:
        print("❌ 没有找到相交的栅格文件，跳过处理。")
        return

    # 打开栅格文件
    src_files = [rasterio.open(fp) for fp in raster_files]
    print(f'{src_files} = src files')

    # 获取栅格文件的坐标系
    with rasterio.open(raster_files[0]) as ref_src:
        raster_crs = ref_src.crs

    # 如果目标 bbox 的坐标系与栅格文件的坐标系不同，进行转换
    # 假设 target_bbox 是以 WGS 84（EPSG:4326）为坐标系
    target_crs = "EPSG:4326"
    if raster_crs != target_crs:
        print("⚠️ 正在转换 target_bbox 坐标系...")
        min_x, min_y, max_x, max_y = transform_bounds(target_crs, raster_crs,  *target_bbox)

        target_bbox = (min_x, min_y, max_x, max_y)

    # 合并栅格文件
    mosaic, out_trans = merge(src_files)

    out_meta = src_files[0].meta.copy()

    out_meta.update({
        "driver": "GTiff",
        "count": 1,
        "height": mosaic.shape[1],
        "width": mosaic.shape[2],
        "transform": out_trans,
        "dtype": "uint8",
        "nodata": 255
    })

    # 生成裁剪范围
    min_x, min_y, max_x, max_y = target_bbox
    bbox_geom = [box(min_x, min_y, max_x, max_y)]


    # # 使用目标坐标系进行裁剪
    # with rasterio.open(raster_files[0]) as ref_src:
    #     bbox_mask, bbox_transform = mask(ref_src, bbox_geom, crop=True)
    with rasterio.io.MemoryFile() as memfile:
        with memfile.open(**out_meta) as mem_raster:
            mem_raster.write(mosaic)

            # 使用目标坐标系进行裁剪
            bbox_mask, bbox_transform = mask(
                dataset=mem_raster,
                shapes=bbox_geom,
                crop=True
            )

    data = bbox_mask[0]
    data = np.where((data == 10) | (data == 20) | (data == 30) | (data == 70) | (data == 90), 20, 255).astype(np.uint8)

    # 如果整个栅格都是无效值，则不保存
    if np.all(data == 255):
        print(f"🚫 {output_path} 仅包含 nodata 值，已跳过。")
        return
    mosaic = mosaic.astype(np.uint8)

    out_meta.update({
        "height": data.shape[0],
        "width": data.shape[1],
        "transform": bbox_transform,
        "dtype": mosaic.dtype.name,
        "nodata": 255
    })

    # 保存裁剪后的栅格文件
    with rasterio.open(output_path, "w", compress="LZW", **out_meta) as dest:
        dest.write(data, 1)

    print(f"✅ 已保存: {output_path}")

# 批量处理多个地区
target_workspace = r'D:\file'
raster_workspace = r'C:\ESA'
# 设置输入文件夹和输出文件路径
input_folder = r"D:\file\d_zonalst"

for place in os.listdir(input_folder):
    shp_file = os.path.join(target_workspace, f'a_clip_data\\c_merge\\polygon_{place}.shp')
    output_workspace = os.path.join(target_workspace, f"d_greenland\\{place}")
    os.makedirs(output_workspace, exist_ok=True)

    output_path = os.path.join(output_workspace, f'greenland_{place}.tif')

    if os.path.exists(output_path):
        print(f"⚠️ 文件已存在，跳过: {output_path}")
        continue

    target_bbox = get_shapefile_bounding_box(shp_file)
    print(f'📍 目标区域 {place} 边界: {target_bbox}')

    intersecting_files = find_intersecting_files(raster_workspace, target_bbox)

    if intersecting_files:
        print(f"📂 {place} 相交的栅格文件: {len(intersecting_files)} 个")
        mask_and_merge_rasters(intersecting_files, output_path, target_bbox)
    else:
        print(f"🚫 {place} 没有相交的栅格文件，跳过")


⚠️ 文件已存在，跳过: D:\file\d_greenland\0\greenland_0.tif
⚠️ 文件已存在，跳过: D:\file\d_greenland\1\greenland_1.tif
⚠️ 文件已存在，跳过: D:\file\d_greenland\10\greenland_10.tif
⚠️ 文件已存在，跳过: D:\file\d_greenland\100\greenland_100.tif
⚠️ 文件已存在，跳过: D:\file\d_greenland\101\greenland_101.tif
⚠️ 文件已存在，跳过: D:\file\d_greenland\103\greenland_103.tif
⚠️ 文件已存在，跳过: D:\file\d_greenland\104\greenland_104.tif
⚠️ 文件已存在，跳过: D:\file\d_greenland\105\greenland_105.tif
⚠️ 文件已存在，跳过: D:\file\d_greenland\106\greenland_106.tif
⚠️ 文件已存在，跳过: D:\file\d_greenland\107\greenland_107.tif
⚠️ 文件已存在，跳过: D:\file\d_greenland\108\greenland_108.tif
⚠️ 文件已存在，跳过: D:\file\d_greenland\109\greenland_109.tif
⚠️ 文件已存在，跳过: D:\file\d_greenland\11\greenland_11.tif
⚠️ 文件已存在，跳过: D:\file\d_greenland\111\greenland_111.tif
⚠️ 文件已存在，跳过: D:\file\d_greenland\112\greenland_112.tif
⚠️ 文件已存在，跳过: D:\file\d_greenland\113\greenland_113.tif
⚠️ 文件已存在，跳过: D:\file\d_greenland\114\greenland_114.tif
⚠️ 文件已存在，跳过: D:\file\d_greenland\115\greenland_115.tif
⚠️ 文件已存在，跳过: D:\file\d

# 12_03 tif -> shp file

In [28]:
import os
import rasterio
from rasterio.features import shapes
import numpy as np
import fiona
from shapely.geometry import shape, MultiPolygon
from shapely.ops import unary_union
from rasterio.warp import calculate_default_transform, reproject, Resampling

def raster_to_polygon(input_tif, output_shp, min_area=30000, target_crs="EPSG:3857"):
    """
    Convert a raster to a single merged polygon and save as a shapefile. Filters polygons smaller than `min_area`.
    :param input_tif: Path to the input raster file
    :param output_shp: Path to the output shapefile
    :param min_area: Minimum area (in square meters) to keep the polygons
    :param target_crs: The target coordinate reference system (CRS) to project to if necessary
    :return: None
    """
    try:
        with rasterio.open(input_tif) as src:
            # 获取原始坐标系
            src_crs = src.crs
            transform = src.transform

            # 打印原始数据的 CRS 和范围
            print(f"原始 CRS: {src_crs}")
            print(f"原始范围: {src.bounds}")

            # 如果输入是 WGS 84 (EPSG:4326)，则需要进行重投影
            if src_crs.to_epsg() == 4326:
                print(f"⚠️ 输入 TIFF 是 WGS 84，需要重投影到 {target_crs}...")

                # 计算重投影所需的 transform 和目标宽高
                transform, width, height = calculate_default_transform(
                    src.crs, target_crs, src.width, src.height, *src.bounds)
                kwargs = src.meta.copy()
                kwargs.update({'crs': target_crs, 'transform': transform, 'width': width, 'height': height})

                # 创建临时文件进行重投影
                with rasterio.open('temp_reprojected.tif', 'w', **kwargs) as dst:
                    for i in range(1, src.count + 1):
                        reproject(
                            source=rasterio.band(src, i),
                            destination=rasterio.band(dst, i),
                            src_transform=src.transform,
                            src_crs=src.crs,
                            dst_transform=transform,
                            dst_crs=target_crs,
                            resampling=Resampling.nearest
                        )

                # 打开重投影后的文件
                with rasterio.open('temp_reprojected.tif') as src_reprojected:
                    image = src_reprojected.read(1)
                    transform = src_reprojected.transform
                    # 打印重投影后的范围
                    print(f"重投影后的范围: {src_reprojected.bounds}")
                    print(f"重投影后的 CRS: {src_reprojected.crs}")

            else:
                image = src.read(1)

            # 获取 NoData 值
            no_data_value = src.nodata

            # 过滤无效区域
            mask = (image != no_data_value) & (~np.isnan(image)) if no_data_value is not None else ~np.isnan(image)
            image[np.isnan(image)] = 0  # NaN 设为 0
            image[np.isinf(image)] = 0  # Inf 设为 0
            image = np.round(image * 10000).astype(np.int32)  # 转为整数

            # 获取栅格对应的多边形
            polygons = [
                shape(s) for s, v in shapes(image, mask=mask, transform=transform)
                if v != no_data_value and not np.isnan(v)
            ]

            # 如果没有有效几何体，则不生成 shp
            if not polygons:
                print(f"跳过 {input_tif}，因为没有有效的矢量数据")
                return

            # 过滤面积小于 min_area 的多边形
            filtered_polygons = [
                geom for geom in polygons if geom.area >= min_area
            ]

            # 如果没有符合面积要求的区域，跳过该文件
            if not filtered_polygons:
                print(f"跳过 {input_tif}，因为没有符合面积要求的区域")
                return

            # 合并所有过滤后的 Polygon
            final_geom = unary_union(filtered_polygons)

            # 定义 Shapefile 的结构
            schema = {'geometry': 'MultiPolygon' if isinstance(final_geom, MultiPolygon) else 'Polygon',
                      'properties': {'value': 'int'}}

            # 写入 Shapefile
            with fiona.open(output_shp, 'w', driver='ESRI Shapefile', schema=schema, crs=target_crs) as shp:
                shp.write({'geometry': final_geom.__geo_interface__, 'properties': {'value': 1}})

        print(f"Shapefile 已创建: {output_shp}")

    except Exception as e:
        print(f"错误: {e}")
# 设置输入文件夹和输出文件路径
input_folder = r"D:\file\d_zonalst"

for place in os.listdir(input_folder):
    print(place)
    workspace = rf'D:\file\d_greenland\{place}'
    process_tif = os.path.join(workspace, f'greenland_{place}.tif')
    output_shp = os.path.join(workspace, f'greenland_{place}.shp')
    if os.path.exists(output_shp):
        print(f'{output_shp} already exists, continue')
        continue
    raster_to_polygon(process_tif, output_shp)

0
D:\file\d_greenland\0\greenland_0.shp already exists, continue
1
D:\file\d_greenland\1\greenland_1.shp already exists, continue
10
D:\file\d_greenland\10\greenland_10.shp already exists, continue
100
D:\file\d_greenland\100\greenland_100.shp already exists, continue
101
D:\file\d_greenland\101\greenland_101.shp already exists, continue
103
D:\file\d_greenland\103\greenland_103.shp already exists, continue
104
D:\file\d_greenland\104\greenland_104.shp already exists, continue
105
D:\file\d_greenland\105\greenland_105.shp already exists, continue
106
D:\file\d_greenland\106\greenland_106.shp already exists, continue
107
D:\file\d_greenland\107\greenland_107.shp already exists, continue
108
D:\file\d_greenland\108\greenland_108.shp already exists, continue
109
D:\file\d_greenland\109\greenland_109.shp already exists, continue
11
D:\file\d_greenland\11\greenland_11.shp already exists, continue
111
D:\file\d_greenland\111\greenland_111.shp already exists, continue
112
D:\file\d_greenland\

# 12_04 shp -> tif file clean the 30000m2 file

In [9]:
import geopandas as gpd
import rasterio
from rasterio.features import rasterize
from shapely.geometry import mapping
import numpy as np
import os

def shapefile_to_raster(shapefile, output_raster, pixel_size=10):
    """
    将 Shapefile 转换为 Raster 格式
    :param shapefile: 输入的 Shapefile 路径
    :param output_raster: 输出的 Raster 文件路径
    :param pixel_size: 栅格像素大小（单位：米）
    """
    # 读取 Shapefile 文件
    gdf = gpd.read_file(shapefile)

    # 获取 Shapefile 的边界
    bounds = gdf.total_bounds  # [minx, miny, maxx, maxy]
    minx, miny, maxx, maxy = bounds

    # 计算栅格的宽度和高度
    width = int((maxx - minx) / pixel_size)
    height = int((maxy - miny) / pixel_size)

    # 创建栅格文件的元数据
    transform = rasterio.transform.from_origin(minx, maxy, pixel_size, pixel_size)
    metadata = {
        'driver': 'GTiff',
        'count': 1,
        'dtype': 'int32',
        'crs': gdf.crs,
        'width': width,
        'height': height,
        'nodata': 0,
        'transform': transform
    }

    # 创建一个空的栅格数据
    with rasterio.open(output_raster, 'w', **metadata) as dst:
        # 将 Shapefile 中的几何体栅格化
        burned = rasterize(
            ((geom, 1) for geom in gdf.geometry),  # 以每个几何体为单元，值设为 1
            out_shape=(height, width),
            transform=transform,
            dtype='int32'
        )

        # 写入栅格数据
        dst.write(burned, 1)

    print(f"栅格文件已创建: {output_raster}")

input_folder = r"D:\file\d_greenland"

for place in os.listdir(input_folder):
    workspace = rf'D:\file\d_greenland\{place}'

    EPSG3857_shp = os.path.join(workspace, f'greenland_{place}.shp')
    if os.path.exists(EPSG3857_shp):
        output84_tif = os.path.join(workspace, f'greenland_{place}_final.tif')
        if os.path.exists(output84_tif):
            print(f'{output84_tif} already exists, continue.')
            continue

        #调用 polygon_to_raster 函数
        shapefile_to_raster(EPSG3857_shp, output84_tif)

D:\file\d_greenland\0\greenland_0_final.tif already exists, continue.
D:\file\d_greenland\1\greenland_1_final.tif already exists, continue.
D:\file\d_greenland\10\greenland_10_final.tif already exists, continue.
D:\file\d_greenland\100\greenland_100_final.tif already exists, continue.
D:\file\d_greenland\101\greenland_101_final.tif already exists, continue.
栅格文件已创建: D:\file\d_greenland\102\greenland_102_final.tif
D:\file\d_greenland\103\greenland_103_final.tif already exists, continue.
D:\file\d_greenland\104\greenland_104_final.tif already exists, continue.
D:\file\d_greenland\105\greenland_105_final.tif already exists, continue.
D:\file\d_greenland\106\greenland_106_final.tif already exists, continue.
D:\file\d_greenland\107\greenland_107_final.tif already exists, continue.
D:\file\d_greenland\108\greenland_108_final.tif already exists, continue.
D:\file\d_greenland\109\greenland_109_final.tif already exists, continue.
栅格文件已创建: D:\file\d_greenland\11\greenland_11_final.tif
栅格文件已创建: D

# 12_05 arcpy

In [ ]:
# -*- coding: utf-8 -*-
import arcpy
import os

# 设置输入文件夹和输出路径
input_folder = r"D:\file\d_zonalst"
target_workspace = r"D:\file"

# 遍历文件夹中的所有 `place2` 子文件夹
for place2 in os.listdir(input_folder):
    try:
        # 输入栅格文件路径
        input_raster = r"D:\file\d_greenland\{}\greenland_{}_final.tif".format(place2, place2)
        boundary_shapefile = os.path.join(target_workspace, r'a_clip_data\c_merge\polygon_{}.shp'.format(place2))

        # 输出栅格文件路径
        output_raster = r"D:\file\d_greenland\{}\greenland_{}_cost_DistWB.tif".format(place2, place2)

        # 检查输入文件是否存在
        if not arcpy.Exists(input_raster):
            print("❌ 输入栅格文件不存在: {}".format(input_raster))
            continue
        if not arcpy.Exists(boundary_shapefile):
            print("❌ 边界文件不存在: {}".format(boundary_shapefile))
            continue
        # if arcpy.Exists(output_raster):
        #     print("✅ {} 已存在，跳过计算".format(output_raster))
        #     continue

        # 设置环境变量，使计算的范围与 `boundary_shapefile` 一致
        arcpy.env.extent = boundary_shapefile  # 限制计算范围
        arcpy.env.mask = boundary_shapefile    # 只保留 `boundary_shapefile` 内的结果
        arcpy.env.snapRaster = input_raster    # 确保栅格对齐
        arcpy.env.cellSize = input_raster      # 让计算分辨率与 `input_raster` 一致

        # 计算欧式距离
        out_distance_raster = arcpy.sa.EucDistance(input_raster, None, None, None, "GEODESIC", None, None)
        out_distance_raster.save(output_raster)

        print("🎯 欧式距离计算完成，并已保存：{}".format(output_raster))

    except Exception as e:
        print("❌ 处理 {} 时出错: {}".format(place2, e))

print("✅ 所有计算完成！")

# 12_06 Perform zonal statistics to compute DistGL.

In [75]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio
from rasterio.mask import mask

def remove_nodata2(data, no_data):
    if no_data is not None:
        # Create a mask for NoData values and NaNs
        mask = (data == no_data) | np.isnan(data)
    else:
        # Create a mask for NaNs
        mask = np.isnan(data)

    valid_data = data[~mask]
    return valid_data
def zonal_statistics_cost_distance(raster_path, vector_path, output_path):
    """
    Calculate zonal statistics for vector geometries based on cost distance raster data.
    :param raster_path: Path to the cost distance raster file
    :param vector_path: Path to the input vector (shapefile) file
    :param output_path: Path to the output shapefile with statistics
    :return: None
    """
    try:
        # Open the cost distance raster file and get NoData value
        with rasterio.open(raster_path) as raster:
            no_data = raster.nodata
            raster_crs = raster.crs
            #print(f"Raster CRS: {raster_crs}, NoData value: {no_data}")

        # Open the vector file
        vector = gpd.read_file(vector_path)
        # 如果 raster 文件不存在，直接写入 NaN 并返回
        if not os.path.exists(raster_path):
            print(f"Raster file {raster_path} not found. Setting MIN_DistGL to NoData.")
            vector["MIN_DistGL"] = -9999  # 赋值 NaN
            vector.to_file(output_path)  # 仍然保存
            return

        vector_crs = vector.crs
        #print(f"Vector CRS: {vector_crs}")

        # Reproject vector data if necessary
        if vector_crs != raster_crs:
            vector = vector.to_crs(raster_crs)
            #print("Reprojected vector data to raster CRS.")

        results = []
        for idx, row in vector.iterrows():
            geom = row['geometry']
            #print(f"Processing feature {idx + 1}/{vector.shape[0]}")

            # Mask the raster using the geometry
            with rasterio.open(raster_path) as raster:
                out_image, _ = mask(raster, [geom], crop=True)

                # Remove NoData values
                data = out_image[0]
                valid_data = remove_nodata2(data, no_data)

                # Calculate statistics
                if valid_data.size > 0:
                    min_val = float(np.nanmin(valid_data))
                    #print(f'min_val:{min_val}, max_val:{max_val}, mean_val:{mean_val}, std_val:{std_val}, sum_val:{sum_val}, count_val:{count_val}')
                else:
                    min_val = -9999

                stats = {
                    'MIN_DistGL': min_val
                }
                results.append(stats)

        # Create DataFrame to store results
        stats_df = pd.DataFrame(results)
        vector = vector.join(stats_df)

        # Save results to a new shapefile
        vector.to_file(output_path)
        print(f"Zonal statistics (cost distance) saved to: {output_path}")

    except Exception as e:
        print(f"Error calculating zonal statistics for cost distance: {e}")

# 设置输入文件夹和输出文件路径
input_folder = r"D:\file\d_zonalst"

for place2 in os.listdir(input_folder):
    workspace = rf'D:\file\d_zonalst\{place2}'
    vector_path3 = os.path.join(workspace, f'{place2}_output_file(MEAN_BD_FAR_CD_CR_DistWB).shp')
    cost_distance_tif = rf'D:\file\d_greenland\{place2}\greenland_{place2}_cost_DistWB.tif'  # 成本距离栅格路径
    output_shp_distwb = os.path.join(workspace, f'{place2}_output_file(MEAN_BD_FAR_CD_CR_DistWB_DistGL).shp')
    # if os.path.exists(output_shp_distwb):
    #     print(f'{output_shp_distwb} already exist, continue')
    #     continue

    # 计算分区统计
    zonal_statistics_cost_distance(cost_distance_tif, vector_path3, output_shp_distwb)

Zonal statistics (cost distance) saved to: D:\file\d_zonalst\0\0_output_file(MEAN_BD_FAR_CD_CR_DistWB_DistGL).shp
Zonal statistics (cost distance) saved to: D:\file\d_zonalst\1\1_output_file(MEAN_BD_FAR_CD_CR_DistWB_DistGL).shp
Zonal statistics (cost distance) saved to: D:\file\d_zonalst\10\10_output_file(MEAN_BD_FAR_CD_CR_DistWB_DistGL).shp
Zonal statistics (cost distance) saved to: D:\file\d_zonalst\100\100_output_file(MEAN_BD_FAR_CD_CR_DistWB_DistGL).shp
Zonal statistics (cost distance) saved to: D:\file\d_zonalst\101\101_output_file(MEAN_BD_FAR_CD_CR_DistWB_DistGL).shp
Zonal statistics (cost distance) saved to: D:\file\d_zonalst\103\103_output_file(MEAN_BD_FAR_CD_CR_DistWB_DistGL).shp
Zonal statistics (cost distance) saved to: D:\file\d_zonalst\104\104_output_file(MEAN_BD_FAR_CD_CR_DistWB_DistGL).shp
Zonal statistics (cost distance) saved to: D:\file\d_zonalst\105\105_output_file(MEAN_BD_FAR_CD_CR_DistWB_DistGL).shp
Zonal statistics (cost distance) saved to: D:\file\d_zonalst\106\1

# 12_07 Converged into one file

In [23]:
import geopandas as gpd
import os
import pandas as pd

# 设置输入文件夹和输出文件路径
input_folder = r"D:\file\d_zonalst"
output_shapefile = r"D:\file\e_index\223cities_output_file(MEAN_BD_FAR_CD_CR_DistWB_DistGL).shp"

# 初始化一个空的 GeoDataFrame
merged_gdf = gpd.GeoDataFrame()

# 遍历文件夹中的所有 `place2` 子文件夹
for place2 in os.listdir(input_folder):
    input_shapefile = os.path.join(input_folder, place2, f"{place2}_output_file(MEAN_BD_FAR_CD_CR_DistWB_DistGL).shp")

    # 检查 shapefile 是否存在
    if os.path.exists(input_shapefile):
        try:
            # 读取当前的 shapefile
            gdf = gpd.read_file(input_shapefile)

            # 检查 CRS，如果不是 WGS84（EPSG:4326），则转换
            if gdf.crs and gdf.crs.to_epsg() != 4326:
                gdf = gdf.to_crs(epsg=4326)
                print(f"{input_shapefile} 坐标系已转换为 WGS84 (EPSG:4326)。")

            # 合并数据
            merged_gdf = gpd.GeoDataFrame(pd.concat([merged_gdf, gdf], ignore_index=True))
            print(f"{input_shapefile} 已成功合并。")
        except Exception as e:
            print(f"无法读取 {input_shapefile}: {e}")
    else:
        print(f"文件 {input_shapefile} 不存在，跳过。")

# 保存最终合并的 shapefile
if not merged_gdf.empty:
    merged_gdf.to_file(output_shapefile, driver="ESRI Shapefile")
    print(f"所有文件已合并并保存到 {output_shapefile}（WGS84 坐标系）。")
else:
    print("没有有效的 shapefile 被合并。")


D:\file\d_zonalst\0\0_output_file(MEAN_BD_FAR_CD_CR_DistWB_DistGL).shp 坐标系已转换为 WGS84 (EPSG:4326)。
D:\file\d_zonalst\0\0_output_file(MEAN_BD_FAR_CD_CR_DistWB_DistGL).shp 已成功合并。
D:\file\d_zonalst\1\1_output_file(MEAN_BD_FAR_CD_CR_DistWB_DistGL).shp 坐标系已转换为 WGS84 (EPSG:4326)。
D:\file\d_zonalst\1\1_output_file(MEAN_BD_FAR_CD_CR_DistWB_DistGL).shp 已成功合并。
D:\file\d_zonalst\10\10_output_file(MEAN_BD_FAR_CD_CR_DistWB_DistGL).shp 坐标系已转换为 WGS84 (EPSG:4326)。
D:\file\d_zonalst\10\10_output_file(MEAN_BD_FAR_CD_CR_DistWB_DistGL).shp 已成功合并。
D:\file\d_zonalst\100\100_output_file(MEAN_BD_FAR_CD_CR_DistWB_DistGL).shp 坐标系已转换为 WGS84 (EPSG:4326)。
D:\file\d_zonalst\100\100_output_file(MEAN_BD_FAR_CD_CR_DistWB_DistGL).shp 已成功合并。
D:\file\d_zonalst\101\101_output_file(MEAN_BD_FAR_CD_CR_DistWB_DistGL).shp 坐标系已转换为 WGS84 (EPSG:4326)。
D:\file\d_zonalst\101\101_output_file(MEAN_BD_FAR_CD_CR_DistWB_DistGL).shp 已成功合并。
文件 D:\file\d_zonalst\102\102_output_file(MEAN_BD_FAR_CD_CR_DistWB_DistGL).shp 不存在，跳过。
D:\file\d_zona

# 13_01 mountain
https://mp.weixin.qq.com/s/BXw60JSqv71KECIeDO7UHg
https://zenodo.org/records/13187969
GBLU data
Paper: Global basic landform units derived from multi-source digital elevation models at 1 arc-second resolution
DOI: 10.5194/essd-2024-401

# 13_02 mountain get the tif file

In [28]:
import os
import rasterio
import numpy as np
import geopandas as gpd
from rasterio.merge import merge
from rasterio.mask import mask
from shapely.geometry import box
from pyproj import Transformer

def get_shapefile_bounding_box(shp_path):
    """ 获取 Shapefile 的边界框，并转换到 WGS 84 (EPSG:4326) """
    gdf = gpd.read_file(shp_path)
    if gdf.crs is None:
        raise ValueError(f"Shapefile {shp_path} 缺少坐标系信息！")

    min_x, min_y, max_x, max_y = gdf.total_bounds
    crs = gdf.crs.to_string()

    # 如果坐标系不是 WGS 84，则转换
    if crs != 'EPSG:4326':
        transformer = Transformer.from_crs(crs, 'EPSG:4326', always_xy=True)
        min_x, min_y = transformer.transform(min_x, min_y)
        max_x, max_y = transformer.transform(max_x, max_y)

    return min_x, min_y,max_x, max_y

def get_raster_bounding_box(raster_path):
    """ 获取 TIFF 栅格文件的边界框，并转换到 WGS 84 (EPSG:4326) """
    try:
        with rasterio.open(raster_path) as src:
            if src.crs is None:
                raise ValueError(f"TIFF 文件 {raster_path} 缺少坐标系信息！")

            min_x, min_y, max_x, max_y = src.bounds
            crs = src.crs.to_string()

            # 如果坐标系不是 WGS 84，则转换
            if crs != 'EPSG:4326':
                transformer = Transformer.from_crs(crs, 'EPSG:4326', always_xy=True)
                min_x, min_y = transformer.transform(min_x, min_y)
                max_x, max_y = transformer.transform(max_x, max_y)

            return min_x, min_y,max_x, max_y
    except rasterio.errors.RasterioIOError as e:
        print(f"⚠️ 无法读取 {raster_path}，可能是损坏的 TIFF 文件，已跳过。错误: {e}")
        return None

def is_bbox_intersect(file_bbox, target_bbox, buffer=0):
    """ 判断文件的边界框是否与目标范围有交集 """
    f_min_x, f_min_y, f_max_x, f_max_y = file_bbox
    t_min_x, t_min_y, t_max_x, t_max_y = target_bbox

    # 目标范围扩展 buffer
    t_min_x -= buffer
    t_max_x += buffer
    t_min_y -= buffer
    t_max_y += buffer

    return not (f_max_x < t_min_x or f_min_x > t_max_x or
                f_max_y < t_min_y or f_min_y > t_max_y)

def find_intersecting_files(raster_folder, target_bbox):
    """ 查找与目标 bbox 相交的 TIFF 文件 """
    intersecting_files = []
    for file in os.listdir(raster_folder):
        if file.endswith(".tif") or file.endswith(".tiff"):
            raster_path = os.path.join(raster_folder, file)
            file_bbox = get_raster_bounding_box(raster_path)
            if file_bbox and is_bbox_intersect(file_bbox, target_bbox):
                print(file_bbox)
                print(file)
                intersecting_files.append(raster_path)
    return intersecting_files

from rasterio.warp import transform_bounds

def mask_and_merge_rasters(raster_files, output_path, target_bbox):
    """ 合并多个栅格文件并裁剪到 target_bbox，并只保留值为 80 的像素 """
    if not raster_files:
        print("❌ 没有找到相交的栅格文件，跳过处理。")
        return

    # 打开栅格文件
    src_files = [rasterio.open(fp) for fp in raster_files]
    print(f'{src_files} = src files')

    # 获取栅格文件的坐标系
    with rasterio.open(raster_files[0]) as ref_src:
        raster_crs = ref_src.crs

    # 如果目标 bbox 的坐标系与栅格文件的坐标系不同，进行转换
    # 假设 target_bbox 是以 WGS 84（EPSG:4326）为坐标系
    target_crs = "EPSG:4326"
    if raster_crs != target_crs:
        print("⚠️ 正在转换 target_bbox 坐标系...")
        min_x, min_y, max_x, max_y = transform_bounds(target_crs, raster_crs, *target_bbox)

        target_bbox = (min_x, min_y, max_x, max_y)

    # 合并栅格文件
    mosaic, out_trans = merge(src_files)

    out_meta = src_files[0].meta.copy()

    out_meta.update({
        "driver": "GTiff",
        "count": 1,
        "height": mosaic.shape[1],
        "width": mosaic.shape[2],
        "transform": out_trans,
        "dtype": "uint8",
        "nodata": 255
    })

    # 生成裁剪范围
    min_x, min_y, max_x, max_y = target_bbox
    bbox_geom = [box(min_x, min_y, max_x, max_y)]

    with rasterio.io.MemoryFile() as memfile:
        with memfile.open(**out_meta) as mem_raster:
            mem_raster.write(mosaic)

            # 使用目标坐标系进行裁剪
            bbox_mask, bbox_transform = mask(
                dataset=mem_raster,
                shapes=bbox_geom,
                crop=True
            )

    data = bbox_mask[0]
    data = np.where((data == 211) | (data == 212) | (data == 213) | (data == 214) | (data == 221) | (data == 222) | (data == 223) | (data == 224) | (data == 231) | (data == 232) | (data == 233) | (data == 234) | (data == 241) | (data == 242) | (data == 243) | (data == 244) | (data == 252) | (data == 253) | (data == 254), 200, 255).astype(np.uint8)

    # 如果整个栅格都是无效值，则不保存
    if np.all(data == 255):
        print(f"🚫 {output_path} 仅包含 nodata 值，已跳过。")
        return
    mosaic = mosaic.astype(np.uint8)

    out_meta.update({
        "height": data.shape[0],
        "width": data.shape[1],
        "transform": bbox_transform,
        "dtype": mosaic.dtype.name,
        "nodata": 255
    })

    # 保存裁剪后的栅格文件
    with rasterio.open(output_path, "w", compress="LZW", **out_meta) as dest:
        dest.write(data, 1)

    print(f"✅ 已保存: {output_path}")


# 批量处理多个地区
target_workspace = r'D:\file'
raster_workspace = r'C:\GBLU'
# 设置输入文件夹和输出文件路径
input_folder = r"D:\file\d_zonalst"

for place in os.listdir(input_folder):
    shp_file = os.path.join(target_workspace, f'a_clip_data\\c_merge\\polygon_{place}.shp')
    output_workspace = os.path.join(target_workspace, f"d_mountain\\{place}")
    os.makedirs(output_workspace, exist_ok=True)

    output_path = os.path.join(output_workspace, f'mountain_{place}.tif')

    if os.path.exists(output_path):
        print(f"⚠️ 文件已存在，跳过: {output_path}")
        continue

    target_bbox = get_shapefile_bounding_box(shp_file)
    print(f'📍 目标区域 {place} 边界: {target_bbox}')

    intersecting_files = find_intersecting_files(raster_workspace, target_bbox)

    if intersecting_files:
        print(f"📂 {place} 相交的栅格文件: {len(intersecting_files)} 个")
        mask_and_merge_rasters(intersecting_files, output_path, target_bbox)
    else:
        print(f"🚫 {place} 没有相交的栅格文件，跳过")

⚠️ 文件已存在，跳过: D:\file\d_mountain\0\mountain_0.tif
⚠️ 文件已存在，跳过: D:\file\d_mountain\1\mountain_1.tif
⚠️ 文件已存在，跳过: D:\file\d_mountain\10\mountain_10.tif
📍 目标区域 100 边界: (-78.35824917936122, 33.87961194842762, -77.97741256263424, 33.98144856470492)
(-79.99999999999994, 32.60756843707994, -74.0576443955493, 39.999986607238114)
GBLU_v1_N30W080.tif
📂 100 相交的栅格文件: 1 个
[<open DatasetReader name='C:/GBLU/GBLU_v1_N30W080.tif' mode='r'>] = src files
⚠️ 正在转换 target_bbox 坐标系...
🚫 D:\file\d_mountain\100\mountain_100.tif 仅包含 nodata 值，已跳过。
⚠️ 文件已存在，跳过: D:\file\d_mountain\101\mountain_101.tif
📍 目标区域 103 边界: (-98.62046478479716, 33.813396342991645, -98.41319695719834, 34.02966417014087)
(-99.99999999999989, 29.999999999999993, -90.0001339202382, 39.999983236686035)
GBLU_v1_N30W100.tif
📂 103 相交的栅格文件: 1 个
[<open DatasetReader name='C:/GBLU/GBLU_v1_N30W100.tif' mode='r'>] = src files
⚠️ 正在转换 target_bbox 坐标系...
🚫 D:\file\d_mountain\103\mountain_103.tif 仅包含 nodata 值，已跳过。
⚠️ 文件已存在，跳过: D:\file\d_mountain\104\moun

KeyboardInterrupt: 

# 13_03 tif -> shp file

In [29]:
import os
import rasterio
from rasterio.features import shapes
import numpy as np
import fiona
from shapely.geometry import shape, MultiPolygon
from shapely.ops import unary_union
from rasterio.warp import calculate_default_transform, reproject, Resampling

def raster_to_polygon(input_tif, output_shp, min_area=30000, target_crs="EPSG:3857"):
    """
    Convert a raster to a single merged polygon and save as a shapefile. Filters polygons smaller than `min_area`.
    :param input_tif: Path to the input raster file
    :param output_shp: Path to the output shapefile
    :param min_area: Minimum area (in square meters) to keep the polygons
    :param target_crs: The target coordinate reference system (CRS) to project to if necessary
    :return: None
    """
    try:
        with rasterio.open(input_tif) as src:
            # 获取原始坐标系
            src_crs = src.crs
            transform = src.transform

            # 打印原始数据的 CRS 和范围
            print(f"原始 CRS: {src_crs}")
            print(f"原始范围: {src.bounds}")

            # 如果输入是 WGS 84 (EPSG:4326)，则需要进行重投影
            if src_crs.to_epsg() == 4326:
                print(f"⚠️ 输入 TIFF 是 WGS 84，需要重投影到 {target_crs}...")

                # 计算重投影所需的 transform 和目标宽高
                transform, width, height = calculate_default_transform(
                    src.crs, target_crs, src.width, src.height, *src.bounds)
                kwargs = src.meta.copy()
                kwargs.update({'crs': target_crs, 'transform': transform, 'width': width, 'height': height})

                # 创建临时文件进行重投影
                with rasterio.open('temp_reprojected.tif', 'w', **kwargs) as dst:
                    for i in range(1, src.count + 1):
                        reproject(
                            source=rasterio.band(src, i),
                            destination=rasterio.band(dst, i),
                            src_transform=src.transform,
                            src_crs=src.crs,
                            dst_transform=transform,
                            dst_crs=target_crs,
                            resampling=Resampling.nearest
                        )

                # 打开重投影后的文件
                with rasterio.open('temp_reprojected.tif') as src_reprojected:
                    image = src_reprojected.read(1)
                    transform = src_reprojected.transform
                    # 打印重投影后的范围
                    print(f"重投影后的范围: {src_reprojected.bounds}")
                    print(f"重投影后的 CRS: {src_reprojected.crs}")

            else:
                image = src.read(1)

            # 获取 NoData 值
            no_data_value = src.nodata

            # 过滤无效区域
            mask = (image != no_data_value) & (~np.isnan(image)) if no_data_value is not None else ~np.isnan(image)
            image[np.isnan(image)] = 0  # NaN 设为 0
            image[np.isinf(image)] = 0  # Inf 设为 0
            image = np.round(image * 10000).astype(np.int32)  # 转为整数

            # 获取栅格对应的多边形
            polygons = [
                shape(s) for s, v in shapes(image, mask=mask, transform=transform)
                if v != no_data_value and not np.isnan(v)
            ]

            # 如果没有有效几何体，则不生成 shp
            if not polygons:
                print(f"跳过 {input_tif}，因为没有有效的矢量数据")
                return

            # 过滤面积小于 min_area 的多边形
            filtered_polygons = [
                geom for geom in polygons if geom.area >= min_area
            ]

            # 如果没有符合面积要求的区域，跳过该文件
            if not filtered_polygons:
                print(f"跳过 {input_tif}，因为没有符合面积要求的区域")
                return

            # 合并所有过滤后的 Polygon
            final_geom = unary_union(filtered_polygons)

            # 定义 Shapefile 的结构
            schema = {'geometry': 'MultiPolygon' if isinstance(final_geom, MultiPolygon) else 'Polygon',
                      'properties': {'value': 'int'}}

            # 写入 Shapefile
            with fiona.open(output_shp, 'w', driver='ESRI Shapefile', schema=schema, crs=target_crs) as shp:
                shp.write({'geometry': final_geom.__geo_interface__, 'properties': {'value': 1}})

        print(f"Shapefile 已创建: {output_shp}")

    except Exception as e:
        print(f"错误: {e}")

# 设置输入文件夹和输出文件路径
input_folder = r"D:\file\d_zonalst"
for place in os.listdir(input_folder):

    workspace = rf'D:\file\d_mountain\{place}'
    process_tif = os.path.join(workspace, f'mountain_{place}.tif')
    output_shp = os.path.join(workspace, f'mountain_{place}.shp')
    if os.path.exists(output_shp):
        continue
    raster_to_polygon(process_tif, output_shp)

错误: D:/file/d_mountain/100/mountain_100.tif: No such file or directory
错误: D:/file/d_mountain/103/mountain_103.tif: No such file or directory
错误: D:/file/d_mountain/107/mountain_107.tif: No such file or directory
原始 CRS: EPSG:3857
原始范围: BoundingBox(left=-9015134.171394622, bottom=3946709.843504375, right=-8991194.171394622, top=3973019.843504375)
Shapefile 已创建: D:\file\d_mountain\109\mountain_109.shp
错误: D:/file/d_mountain/111/mountain_111.tif: No such file or directory
错误: D:/file/d_mountain/114/mountain_114.tif: No such file or directory
错误: D:/file/d_mountain/119/mountain_119.tif: No such file or directory
错误: D:/file/d_mountain/125/mountain_125.tif: No such file or directory
错误: D:/file/d_mountain/129/mountain_129.tif: No such file or directory
错误: D:/file/d_mountain/137/mountain_137.tif: No such file or directory
错误: D:/file/d_mountain/145/mountain_145.tif: No such file or directory
错误: D:/file/d_mountain/148/mountain_148.tif: No such file or directory
错误: D:/file/d_mountain/152/m

# 13_04 shp -> tif file clean the 30000m2 file

In [30]:
import geopandas as gpd
import rasterio
from rasterio.features import rasterize
from shapely.geometry import mapping
import numpy as np

def shapefile_to_raster(shapefile, output_raster, pixel_size=10):
    """
    将 Shapefile 转换为 Raster 格式
    :param shapefile: 输入的 Shapefile 路径
    :param output_raster: 输出的 Raster 文件路径
    :param pixel_size: 栅格像素大小（单位：米）
    """
    # 读取 Shapefile 文件
    gdf = gpd.read_file(shapefile)

    # 获取 Shapefile 的边界
    bounds = gdf.total_bounds  # [minx, miny, maxx, maxy]
    minx, miny, maxx, maxy = bounds

    # 计算栅格的宽度和高度
    width = int((maxx - minx) / pixel_size)
    height = int((maxy - miny) / pixel_size)

    # 创建栅格文件的元数据
    transform = rasterio.transform.from_origin(minx, maxy, pixel_size, pixel_size)
    metadata = {
        'driver': 'GTiff',
        'count': 1,
        'dtype': 'int32',
        'crs': gdf.crs,
        'width': width,
        'height': height,
        'nodata': 0,
        'transform': transform
    }

    # 创建一个空的栅格数据
    with rasterio.open(output_raster, 'w', **metadata) as dst:
        # 将 Shapefile 中的几何体栅格化
        burned = rasterize(
            ((geom, 1) for geom in gdf.geometry),  # 以每个几何体为单元，值设为 1
            out_shape=(height, width),
            transform=transform,
            dtype='int32'
        )

        # 写入栅格数据
        dst.write(burned, 1)

    print(f"栅格文件已创建: {output_raster}")

# 设置输入文件夹和输出文件路径
input_folder = r"D:\file\d_zonalst"
for place in os.listdir(input_folder):

    workspace = rf'D:\file\d_mountain\{place}'

    EPSG3857_shp = os.path.join(workspace, f'mountain_{place}.shp')
    if os.path.exists(EPSG3857_shp):
        output84_tif = os.path.join(workspace, f'mountain_{place}_final.tif')
        if os.path.exists(output84_tif):
            print(f'{output84_tif} already exists, continue.')
            continue

        # 调用 polygon_to_raster 函数
        shapefile_to_raster(EPSG3857_shp, output84_tif)

D:\file\d_mountain\0\mountain_0_final.tif already exists, continue.
D:\file\d_mountain\1\mountain_1_final.tif already exists, continue.
D:\file\d_mountain\10\mountain_10_final.tif already exists, continue.
D:\file\d_mountain\101\mountain_101_final.tif already exists, continue.
D:\file\d_mountain\104\mountain_104_final.tif already exists, continue.
D:\file\d_mountain\105\mountain_105_final.tif already exists, continue.
D:\file\d_mountain\106\mountain_106_final.tif already exists, continue.
D:\file\d_mountain\108\mountain_108_final.tif already exists, continue.
栅格文件已创建: D:\file\d_mountain\109\mountain_109_final.tif
D:\file\d_mountain\11\mountain_11_final.tif already exists, continue.
D:\file\d_mountain\112\mountain_112_final.tif already exists, continue.
D:\file\d_mountain\113\mountain_113_final.tif already exists, continue.
D:\file\d_mountain\115\mountain_115_final.tif already exists, continue.
D:\file\d_mountain\116\mountain_116_final.tif already exists, continue.
D:\file\d_mountain\11

# 13_05 arcpy
'''
这里打开Cost_DistWB.py
然后用python2.7运行,得到对应的cost distance
'''

In [ ]:
# -*- coding: utf-8 -*-
import arcpy
import os

# 设置输入文件夹和输出路径
input_folder = r"D:\file\d_zonalst"
target_workspace = r"D:\file"

# 遍历文件夹中的所有 `place2` 子文件夹
for place2 in os.listdir(input_folder):
    try:
        # 输入栅格文件路径
        input_raster = r"D:\file\d_mountain\{}\mountain_{}_final.tif".format(place2, place2)
        boundary_shapefile = os.path.join(target_workspace, r'a_clip_data\c_merge\polygon_{}.shp'.format(place2))

        # 输出栅格文件路径
        output_raster = r"D:\file\d_mountain\{}\mountain_{}_cost_DistWB.tif".format(place2, place2) # 这个distance写不对了 懒得改了

        # 检查输入文件是否存在
        if not arcpy.Exists(input_raster):
            print("❌ 输入栅格文件不存在: {}".format(input_raster))
            continue
        if not arcpy.Exists(boundary_shapefile):
            print("❌ 边界文件不存在: {}".format(boundary_shapefile))
            continue
        # if arcpy.Exists(output_raster):
        #     print("✅ {} 已存在，跳过计算".format(output_raster))
        #     continue

        # 设置环境变量，使计算的范围与 `boundary_shapefile` 一致
        arcpy.env.extent = boundary_shapefile  # 限制计算范围
        arcpy.env.mask = boundary_shapefile    # 只保留 `boundary_shapefile` 内的结果
        arcpy.env.snapRaster = input_raster    # 确保栅格对齐
        arcpy.env.cellSize = input_raster      # 让计算分辨率与 `input_raster` 一致

        # 计算欧式距离
        out_distance_raster = arcpy.sa.EucDistance(input_raster, None, None, None, "PLANAR", None, None)
        out_distance_raster.save(output_raster)

        print("🎯 欧式距离计算完成，并已保存：{}".format(output_raster))

    except Exception as e:
        print("❌ 处理 {} 时出错: {}".format(place2, e))

print("✅ 所有计算完成！")

# 13_06 Perform zonal statistics to compute DistMT.

In [76]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import rasterio
from rasterio.mask import mask

def remove_nodata2(data, no_data):
    if no_data is not None:
        # Create a mask for NoData values and NaNs
        mask = (data == no_data) | np.isnan(data)
    else:
        # Create a mask for NaNs
        mask = np.isnan(data)

    valid_data = data[~mask]
    return valid_data

def zonal_statistics_cost_distance(raster_path, vector_path, output_path):
    """
    Calculate zonal statistics for vector geometries based on cost distance raster data.
    If the raster file does not exist, set the result to NoData.
    """
    try:
        # Open the vector file
        vector = gpd.read_file(vector_path)

        # Check if raster file exists
        if not os.path.exists(raster_path):
            print(f"Raster file {raster_path} not found. Setting MIN_DistMT into NoData values.")
            vector["MIN_DistMT"] = -9999  # Assign NoData to the new field
            vector.to_file(output_path)
            return

        # Open the cost distance raster file
        with rasterio.open(raster_path) as raster:
            no_data = raster.nodata
            raster_crs = raster.crs

        # Reproject vector data if necessary
        if vector.crs != raster_crs:
            vector = vector.to_crs(raster_crs)
            print("Reprojected vector data to raster CRS.")

        results = []
        for idx, row in vector.iterrows():
            geom = row['geometry']

            # Mask the raster using the geometry
            with rasterio.open(raster_path) as raster:
                out_image, _ = mask(raster, [geom], crop=True)

                # Remove NoData values
                data = out_image[0]
                valid_data = remove_nodata2(data, no_data)

                # Calculate statistics
                min_val = float(np.nanmin(valid_data)) if valid_data.size > 0 else -9999
                results.append({'MIN_DistMT': min_val})

        # Create DataFrame to store results
        stats_df = pd.DataFrame(results)
        vector = vector.join(stats_df)

        # Save results to a new shapefile
        vector.to_file(output_path)
        print(f"Zonal statistics (cost distance) saved to: {output_path}")

    except Exception as e:
        print(f"Error calculating zonal statistics for cost distance: {e}")

# 设置输入文件夹和输出文件路径
input_folder = r"D:\file\d_zonalst"
for place2 in os.listdir(input_folder):
    workspace = rf'D:\file\d_zonalst\{place2}'
    vector_path3 = os.path.join(workspace, f'{place2}_output_file(MEAN_BD_FAR_CD_CR_DistWB_DistGL).shp')
    cost_distance_tif = rf'D:\file\d_mountain\{place2}\mountain_{place2}_cost_DistWB.tif'  # 成本距离栅格路径
    output_shp_distwb = os.path.join(workspace, f'{place2}_output_file(MEAN_BD_FAR_CD_CR_DistWB_DistGL_DistMT).shp')
    # 计算分区统计
    zonal_statistics_cost_distance(cost_distance_tif, vector_path3, output_shp_distwb)

Zonal statistics (cost distance) saved to: D:\file\d_zonalst\0\0_output_file(MEAN_BD_FAR_CD_CR_DistWB_DistGL_DistMT).shp
Zonal statistics (cost distance) saved to: D:\file\d_zonalst\1\1_output_file(MEAN_BD_FAR_CD_CR_DistWB_DistGL_DistMT).shp
Zonal statistics (cost distance) saved to: D:\file\d_zonalst\10\10_output_file(MEAN_BD_FAR_CD_CR_DistWB_DistGL_DistMT).shp
Raster file D:\file\d_mountain\100\mountain_100_cost_DistWB.tif not found. Setting MIN_DistMT into NoData values.
Zonal statistics (cost distance) saved to: D:\file\d_zonalst\101\101_output_file(MEAN_BD_FAR_CD_CR_DistWB_DistGL_DistMT).shp
Raster file D:\file\d_mountain\103\mountain_103_cost_DistWB.tif not found. Setting MIN_DistMT into NoData values.
Zonal statistics (cost distance) saved to: D:\file\d_zonalst\104\104_output_file(MEAN_BD_FAR_CD_CR_DistWB_DistGL_DistMT).shp
Zonal statistics (cost distance) saved to: D:\file\d_zonalst\105\105_output_file(MEAN_BD_FAR_CD_CR_DistWB_DistGL_DistMT).shp
Zonal statistics (cost distance)

# 13_07 Converged into one file

In [72]:
import geopandas as gpd
import os
import pandas as pd

# 设置输入文件夹和输出文件路径
input_folder = r"D:\file\d_zonalst"
output_shapefile = r"D:\file\e_index\223cities_output_file(MEAN_BD_FAR)_54008.shp"

# 初始化一个空的 GeoDataFrame
merged_gdf = gpd.GeoDataFrame()

# 遍历文件夹中的所有 `place2` 子文件夹
for place2 in os.listdir(input_folder):
    input_shapefile = os.path.join(input_folder, place2, f"{place2}_output_file(MEAN_BD_FAR)_54008.shp")

    # 检查 shapefile 是否存在
    if os.path.exists(input_shapefile):
        try:
            # 读取当前的 shapefile
            gdf = gpd.read_file(input_shapefile)

            # 检查 CRS，如果不是 WGS84（EPSG:4326），则转换
            if gdf.crs and gdf.crs.to_epsg() != 4326:
                gdf = gdf.to_crs(epsg=4326)
                print(f"{input_shapefile} 坐标系已转换为 WGS84 (EPSG:4326)。")

            # 合并数据
            merged_gdf = gpd.GeoDataFrame(pd.concat([merged_gdf, gdf], ignore_index=True))
            print(f"{input_shapefile} 已成功合并。")
        except Exception as e:
            print(f"无法读取 {input_shapefile}: {e}")
    else:
        print(f"文件 {input_shapefile} 不存在，跳过。")

# 保存最终合并的 shapefile
if not merged_gdf.empty:
    merged_gdf.to_file(output_shapefile, driver="ESRI Shapefile")
    print(f"所有文件已合并并保存到 {output_shapefile}（WGS84 坐标系）。")
else:
    print("没有有效的 shapefile 被合并。")


D:\file\d_zonalst\0\0_output_file(MEAN_BD_FAR)_54008.shp 坐标系已转换为 WGS84 (EPSG:4326)。
D:\file\d_zonalst\0\0_output_file(MEAN_BD_FAR)_54008.shp 已成功合并。
D:\file\d_zonalst\1\1_output_file(MEAN_BD_FAR)_54008.shp 坐标系已转换为 WGS84 (EPSG:4326)。
D:\file\d_zonalst\1\1_output_file(MEAN_BD_FAR)_54008.shp 已成功合并。
D:\file\d_zonalst\10\10_output_file(MEAN_BD_FAR)_54008.shp 坐标系已转换为 WGS84 (EPSG:4326)。
D:\file\d_zonalst\10\10_output_file(MEAN_BD_FAR)_54008.shp 已成功合并。
D:\file\d_zonalst\100\100_output_file(MEAN_BD_FAR)_54008.shp 坐标系已转换为 WGS84 (EPSG:4326)。
D:\file\d_zonalst\100\100_output_file(MEAN_BD_FAR)_54008.shp 已成功合并。
D:\file\d_zonalst\101\101_output_file(MEAN_BD_FAR)_54008.shp 坐标系已转换为 WGS84 (EPSG:4326)。
D:\file\d_zonalst\101\101_output_file(MEAN_BD_FAR)_54008.shp 已成功合并。
D:\file\d_zonalst\103\103_output_file(MEAN_BD_FAR)_54008.shp 坐标系已转换为 WGS84 (EPSG:4326)。
D:\file\d_zonalst\103\103_output_file(MEAN_BD_FAR)_54008.shp 已成功合并。
D:\file\d_zonalst\104\104_output_file(MEAN_BD_FAR)_54008.shp 坐标系已转换为 WGS84 (EPSG:432

In [73]:
import geopandas as gpd
import os
import pandas as pd

# 设置输入文件夹和输出文件路径
input_folder = r"D:\file\d_zonalst"
output_shapefile = r"D:\file\e_index\223cities_output_file(MEAN_BD_FAR_CD_CR).shp"

# 初始化一个空的 GeoDataFrame
merged_gdf = gpd.GeoDataFrame()

# 遍历文件夹中的所有 `place2` 子文件夹
for place2 in os.listdir(input_folder):
    input_shapefile = os.path.join(input_folder, place2, f"{place2}_output_file(MEAN_BD_FAR_CD_CR).shp")

    # 检查 shapefile 是否存在
    if os.path.exists(input_shapefile):
        try:
            # 读取当前的 shapefile
            gdf = gpd.read_file(input_shapefile)

            # 检查 CRS，如果不是 WGS84（EPSG:4326），则转换
            if gdf.crs and gdf.crs.to_epsg() != 4326:
                gdf = gdf.to_crs(epsg=4326)
                print(f"{input_shapefile} 坐标系已转换为 WGS84 (EPSG:4326)。")

            # 合并数据
            merged_gdf = gpd.GeoDataFrame(pd.concat([merged_gdf, gdf], ignore_index=True))
            print(f"{input_shapefile} 已成功合并。")
        except Exception as e:
            print(f"无法读取 {input_shapefile}: {e}")
    else:
        print(f"文件 {input_shapefile} 不存在，跳过。")

# 保存最终合并的 shapefile
if not merged_gdf.empty:
    merged_gdf.to_file(output_shapefile, driver="ESRI Shapefile")
    print(f"所有文件已合并并保存到 {output_shapefile}（WGS84 坐标系）。")
else:
    print("没有有效的 shapefile 被合并。")


D:\file\d_zonalst\0\0_output_file(MEAN_BD_FAR_CD_CR).shp 坐标系已转换为 WGS84 (EPSG:4326)。
D:\file\d_zonalst\0\0_output_file(MEAN_BD_FAR_CD_CR).shp 已成功合并。
D:\file\d_zonalst\1\1_output_file(MEAN_BD_FAR_CD_CR).shp 坐标系已转换为 WGS84 (EPSG:4326)。
D:\file\d_zonalst\1\1_output_file(MEAN_BD_FAR_CD_CR).shp 已成功合并。
D:\file\d_zonalst\10\10_output_file(MEAN_BD_FAR_CD_CR).shp 坐标系已转换为 WGS84 (EPSG:4326)。
D:\file\d_zonalst\10\10_output_file(MEAN_BD_FAR_CD_CR).shp 已成功合并。
D:\file\d_zonalst\100\100_output_file(MEAN_BD_FAR_CD_CR).shp 坐标系已转换为 WGS84 (EPSG:4326)。
D:\file\d_zonalst\100\100_output_file(MEAN_BD_FAR_CD_CR).shp 已成功合并。
D:\file\d_zonalst\101\101_output_file(MEAN_BD_FAR_CD_CR).shp 坐标系已转换为 WGS84 (EPSG:4326)。
D:\file\d_zonalst\101\101_output_file(MEAN_BD_FAR_CD_CR).shp 已成功合并。
D:\file\d_zonalst\103\103_output_file(MEAN_BD_FAR_CD_CR).shp 坐标系已转换为 WGS84 (EPSG:4326)。
D:\file\d_zonalst\103\103_output_file(MEAN_BD_FAR_CD_CR).shp 已成功合并。
D:\file\d_zonalst\104\104_output_file(MEAN_BD_FAR_CD_CR).shp 坐标系已转换为 WGS84 (EPSG:432

In [77]:
import geopandas as gpd
import os
import pandas as pd

# 设置输入文件夹和输出文件路径
input_folder = r"D:\file\d_zonalst"
output_shapefile = r"D:\file\e_index\223cities_output_file(MEAN_BD_FAR_CD_CR_DistWB_DistGL_DistMT).shp"

# 初始化一个空的 GeoDataFrame
merged_gdf = gpd.GeoDataFrame()

# 遍历文件夹中的所有 `place2` 子文件夹
for place2 in os.listdir(input_folder):
    input_shapefile = os.path.join(input_folder, place2, f"{place2}_output_file(MEAN_BD_FAR_CD_CR_DistWB_DistGL_DistMT).shp")

    # 检查 shapefile 是否存在
    if os.path.exists(input_shapefile):
        try:
            # 读取当前的 shapefile
            gdf = gpd.read_file(input_shapefile)

            # 检查 CRS，如果不是 WGS84（EPSG:4326），则转换
            if gdf.crs and gdf.crs.to_epsg() != 4326:
                gdf = gdf.to_crs(epsg=4326)
                print(f"{input_shapefile} 坐标系已转换为 WGS84 (EPSG:4326)。")

            # 合并数据
            merged_gdf = gpd.GeoDataFrame(pd.concat([merged_gdf, gdf], ignore_index=True))
            print(f"{input_shapefile} 已成功合并。")
        except Exception as e:
            print(f"无法读取 {input_shapefile}: {e}")
    else:
        print(f"文件 {input_shapefile} 不存在，跳过。")

# 保存最终合并的 shapefile
if not merged_gdf.empty:
    merged_gdf.to_file(output_shapefile, driver="ESRI Shapefile")
    print(f"所有文件已合并并保存到 {output_shapefile}（WGS84 坐标系）。")
else:
    print("没有有效的 shapefile 被合并。")


D:\file\d_zonalst\0\0_output_file(MEAN_BD_FAR_CD_CR_DistWB_DistGL_DistMT).shp 坐标系已转换为 WGS84 (EPSG:4326)。
D:\file\d_zonalst\0\0_output_file(MEAN_BD_FAR_CD_CR_DistWB_DistGL_DistMT).shp 已成功合并。
D:\file\d_zonalst\1\1_output_file(MEAN_BD_FAR_CD_CR_DistWB_DistGL_DistMT).shp 坐标系已转换为 WGS84 (EPSG:4326)。
D:\file\d_zonalst\1\1_output_file(MEAN_BD_FAR_CD_CR_DistWB_DistGL_DistMT).shp 已成功合并。
D:\file\d_zonalst\10\10_output_file(MEAN_BD_FAR_CD_CR_DistWB_DistGL_DistMT).shp 坐标系已转换为 WGS84 (EPSG:4326)。
D:\file\d_zonalst\10\10_output_file(MEAN_BD_FAR_CD_CR_DistWB_DistGL_DistMT).shp 已成功合并。
D:\file\d_zonalst\100\100_output_file(MEAN_BD_FAR_CD_CR_DistWB_DistGL_DistMT).shp 坐标系已转换为 WGS84 (EPSG:4326)。
D:\file\d_zonalst\100\100_output_file(MEAN_BD_FAR_CD_CR_DistWB_DistGL_DistMT).shp 已成功合并。
D:\file\d_zonalst\101\101_output_file(MEAN_BD_FAR_CD_CR_DistWB_DistGL_DistMT).shp 坐标系已转换为 WGS84 (EPSG:4326)。
D:\file\d_zonalst\101\101_output_file(MEAN_BD_FAR_CD_CR_DistWB_DistGL_DistMT).shp 已成功合并。
D:\file\d_zonalst\103\103_outp

# 14_01 built-up area get the tif file form ESA data

In [1]:
import os
import rasterio
import numpy as np
import geopandas as gpd
from rasterio.merge import merge
from rasterio.mask import mask
from shapely.geometry import box
from pyproj import Transformer

def get_shapefile_bounding_box(shp_path):
    """ 获取 Shapefile 的边界框，并转换到 WGS 84 (EPSG:4326) """
    gdf = gpd.read_file(shp_path)
    if gdf.crs is None:
        raise ValueError(f"Shapefile {shp_path} 缺少坐标系信息！")

    min_x, min_y, max_x, max_y = gdf.total_bounds
    crs = gdf.crs.to_string()

    # 如果坐标系不是 WGS 84，则转换
    if crs != 'EPSG:4326':
        transformer = Transformer.from_crs(crs, 'EPSG:4326', always_xy=True)
        min_x, min_y = transformer.transform(min_x, min_y)
        max_x, max_y = transformer.transform(max_x, max_y)

    return min_x, min_y,max_x, max_y

def get_raster_bounding_box(raster_path):
    """ 获取 TIFF 栅格文件的边界框，并转换到 WGS 84 (EPSG:4326) """
    try:
        with rasterio.open(raster_path) as src:
            if src.crs is None:
                raise ValueError(f"TIFF 文件 {raster_path} 缺少坐标系信息！")

            min_x, min_y, max_x, max_y = src.bounds
            crs = src.crs.to_string()

            # 如果坐标系不是 WGS 84，则转换
            if crs != 'EPSG:4326':
                transformer = Transformer.from_crs(crs, 'EPSG:4326', always_xy=True)
                min_x, min_y = transformer.transform(min_x, min_y)
                max_x, max_y = transformer.transform(max_x, max_y)

            return min_x, min_y,max_x, max_y
    except rasterio.errors.RasterioIOError as e:
        print(f"⚠️ 无法读取 {raster_path}，可能是损坏的 TIFF 文件，已跳过。错误: {e}")
        return None

def is_bbox_intersect(file_bbox, target_bbox, buffer=0):
    """ 判断文件的边界框是否与目标范围有交集 """
    f_min_x, f_min_y, f_max_x, f_max_y = file_bbox
    t_min_x, t_min_y, t_max_x, t_max_y = target_bbox

    # 目标范围扩展 buffer
    t_min_x -= buffer
    t_max_x += buffer
    t_min_y -= buffer
    t_max_y += buffer

    return not (f_max_x < t_min_x or f_min_x > t_max_x or
                f_max_y < t_min_y or f_min_y > t_max_y)

def find_intersecting_files(raster_folder, target_bbox):
    """ 查找与目标 bbox 相交的 TIFF 文件 """
    intersecting_files = []
    for file in os.listdir(raster_folder):
        if file.endswith(".tif") or file.endswith(".tiff"):
            raster_path = os.path.join(raster_folder, file)
            file_bbox = get_raster_bounding_box(raster_path)
            if file_bbox and is_bbox_intersect(file_bbox, target_bbox):
                print(file_bbox)
                print(file)
                intersecting_files.append(raster_path)
    return intersecting_files

from rasterio.warp import transform_bounds

def mask_and_merge_rasters(raster_files, output_path, target_bbox):
    """ 合并多个栅格文件并裁剪到 target_bbox，并只保留值为 80 的像素 """
    if not raster_files:
        print("❌ 没有找到相交的栅格文件，跳过处理。")
        return

    # 打开栅格文件
    src_files = [rasterio.open(fp) for fp in raster_files]
    print(f'{src_files} = src files')

    # 获取栅格文件的坐标系
    with rasterio.open(raster_files[0]) as ref_src:
        raster_crs = ref_src.crs

    # 如果目标 bbox 的坐标系与栅格文件的坐标系不同，进行转换
    # 假设 target_bbox 是以 WGS 84（EPSG:4326）为坐标系
    target_crs = "EPSG:4326"
    if raster_crs != target_crs:
        print("⚠️ 正在转换 target_bbox 坐标系...")
        min_x, min_y, max_x, max_y = transform_bounds(target_crs, raster_crs,  *target_bbox)

        target_bbox = (min_x, min_y, max_x, max_y)

    # 合并栅格文件
    mosaic, out_trans = merge(src_files)

    out_meta = src_files[0].meta.copy()

    out_meta.update({
        "driver": "GTiff",
        "count": 1,
        "height": mosaic.shape[1],
        "width": mosaic.shape[2],
        "transform": out_trans,
        "dtype": "uint8",
        "nodata": 255
    })

    # 生成裁剪范围
    min_x, min_y, max_x, max_y = target_bbox
    bbox_geom = [box(min_x, min_y, max_x, max_y)]


    # # 使用目标坐标系进行裁剪
    # with rasterio.open(raster_files[0]) as ref_src:
    #     bbox_mask, bbox_transform = mask(ref_src, bbox_geom, crop=True)
    with rasterio.io.MemoryFile() as memfile:
        with memfile.open(**out_meta) as mem_raster:
            mem_raster.write(mosaic)

            # 使用目标坐标系进行裁剪
            bbox_mask, bbox_transform = mask(
                dataset=mem_raster,
                shapes=bbox_geom,
                crop=True
            )

    data = bbox_mask[0]
    data = np.where((data == 50), 20, 255).astype(np.uint8)

    # 如果整个栅格都是无效值，则不保存
    if np.all(data == 255):
        print(f"🚫 {output_path} 仅包含 nodata 值，已跳过。")
        return
    mosaic = mosaic.astype(np.uint8)

    out_meta.update({
        "height": data.shape[0],
        "width": data.shape[1],
        "transform": bbox_transform,
        "dtype": mosaic.dtype.name,
        "nodata": 255
    })

    # 保存裁剪后的栅格文件
    with rasterio.open(output_path, "w", compress="LZW", **out_meta) as dest:
        dest.write(data, 1)

    print(f"✅ 已保存: {output_path}")

# 批量处理多个地区
target_workspace = r'D:\file'
raster_workspace = r'C:\ESA'
# 设置输入文件夹和输出文件路径
input_folder = r"D:\file\d_zonalst"

for place in os.listdir(input_folder):
    shp_file = os.path.join(target_workspace, f'a_clip_data\\c_merge\\polygon_{place}.shp')
    output_workspace = os.path.join(target_workspace, f"d_builtup\\{place}")
    os.makedirs(output_workspace, exist_ok=True)

    output_path = os.path.join(output_workspace, f'builtup_{place}.tif')

    if os.path.exists(output_path):
        print(f"⚠️ 文件已存在，跳过: {output_path}")
        continue

    target_bbox = get_shapefile_bounding_box(shp_file)
    print(f'📍 目标区域 {place} 边界: {target_bbox}')

    intersecting_files = find_intersecting_files(raster_workspace, target_bbox)

    if intersecting_files:
        print(f"📂 {place} 相交的栅格文件: {len(intersecting_files)} 个")
        mask_and_merge_rasters(intersecting_files, output_path, target_bbox)
    else:
        print(f"🚫 {place} 没有相交的栅格文件，跳过")

📍 目标区域 0 边界: (4.6335091048, 45.5733702317, 5.1798291537, 46.002690281)
(3.0, 45.0, 6.0, 48.0)
ESA_WorldCover_10m_2020_v100_N45E003_Map.tif
📂 0 相交的栅格文件: 1 个
[<open DatasetReader name='C:/ESA/ESA_WorldCover_10m_2020_v100_N45E003_Map.tif' mode='r'>] = src files
✅ 已保存: D:\file\d_builtup\0\builtup_0.tif
📍 目标区域 1 边界: (15.796022312469578, 45.67488343980875, 16.193315945085256, 45.90117707287408)
(15.0, 45.0, 18.0, 48.0)
ESA_WorldCover_10m_2020_v100_N45E015_Map.tif
📂 1 相交的栅格文件: 1 个
[<open DatasetReader name='C:/ESA/ESA_WorldCover_10m_2020_v100_N45E015_Map.tif' mode='r'>] = src files
✅ 已保存: D:\file\d_builtup\1\builtup_1.tif
📍 目标区域 10 边界: (33.9414361165534, 44.84829724389254, 34.20990214190079, 45.08076326923993)
(33.0, 42.0, 36.0, 45.0)
ESA_WorldCover_10m_2020_v100_N42E033_Map.tif
(33.0, 45.0, 36.0, 48.0)
ESA_WorldCover_10m_2020_v100_N45E033_Map.tif
📂 10 相交的栅格文件: 2 个
[<open DatasetReader name='C:/ESA/ESA_WorldCover_10m_2020_v100_N42E033_Map.tif' mode='r'>, <open DatasetReader name='C:/ESA/ESA_W

In [ ]:
# # -*- coding: utf-8 -*-
# import arcpy
# import os
#
# # 设置输入文件夹和输出路径
# input_folder = r"D:\file\d_zonalst"
# target_workspace = r"D:\file"
#
# # 遍历文件夹中的所有 `place2` 子文件夹
# for place2 in os.listdir(input_folder):
#     try:
#         # 输入栅格文件路径
#         input_raster = r"D:\file\d_builtup\{}\builtup_{}.tif".format(place2, place2)
#         boundary_shapefile = os.path.join(target_workspace, r'a_clip_data\c_merge\polygon_{}.shp'.format(place2))
#
#         # 输出栅格文件路径
#         output_raster = r"D:\file\d_builtup\{}\builtup_{}_cost_DistWB.tif".format(place2, place2)
#
#         # 检查输入文件是否存在
#         if not arcpy.Exists(input_raster):
#             print("❌ 输入栅格文件不存在: {}".format(input_raster))
#             continue
#         if not arcpy.Exists(boundary_shapefile):
#             print("❌ 边界文件不存在: {}".format(boundary_shapefile))
#             continue
#         # if arcpy.Exists(output_raster):
#         #     print("✅ {} 已存在，跳过计算".format(output_raster))
#         #     continue
#
#         # 设置环境变量，使计算的范围与 `boundary_shapefile` 一致
#         arcpy.env.extent = boundary_shapefile  # 限制计算范围
#         arcpy.env.mask = boundary_shapefile    # 只保留 `boundary_shapefile` 内的结果
#         arcpy.env.snapRaster = input_raster    # 确保栅格对齐
#         arcpy.env.cellSize = input_raster      # 让计算分辨率与 `input_raster` 一致
#
#         # 计算欧式距离
#         out_distance_raster = arcpy.sa.EucDistance(input_raster, None, None, None, "GEODESIC", None, None)
#         out_distance_raster.save(output_raster)
#
#         print("🎯 欧式距离计算完成，并已保存：{}".format(output_raster))
#
#     except Exception as e:
#         print("❌ 处理 {} 时出错: {}".format(place2, e))
#
# print("✅ 所有计算完成！")

# 14_03 perform zonalst

In [ ]:
import os
import rasterio
from rasterio.mask import mask
import geopandas as gpd
import pandas as pd
import numpy as np
from pyproj import Transformer, CRS

def create_show_progress(threshold=10):
    """
    创建一个进度回调函数，每当进度增加达到阈值时打印一次信息。
    """
    last_percentage = -threshold
    def Show_Progress(complete, message, unknown):
        nonlocal last_percentage
        percentage = int(complete * 100)
        if percentage >= last_percentage + threshold:
            last_percentage = percentage
            print("Progress: {}% - {}".format(percentage, message))
    return Show_Progress

def calculate_pixel_size_in_meters(raster_path, target_crs_str="ESRI:54009"):
    """
    利用中心像元的角点坐标，将其转换到目标投影后，
    计算像元的宽度、高度及面积（平行四边形假设）。
    """
    with rasterio.open(raster_path) as src:
        height, width = src.height, src.width
        center_row = height // 2
        center_col = width // 2

        # 获取中心像元的左上 (ul)、右上 (ur) 和左下 (ll) 角点坐标
        ul = src.xy(center_row, center_col, offset="ul")
        ur = src.xy(center_row, center_col, offset="ur")
        ll = src.xy(center_row, center_col, offset="ll")

        # 创建目标 CRS 对象
        target_crs = CRS.from_string(target_crs_str)
        transformer = Transformer.from_crs(src.crs, target_crs, always_xy=True)

        # 将角点坐标转换到目标投影
        ul_t = transformer.transform(ul[0], ul[1])
        ur_t = transformer.transform(ur[0], ur[1])
        ll_t = transformer.transform(ll[0], ll[1])

        # 计算转换后像元宽度和高度
        pixel_width = abs(ur_t[0] - ul_t[0])
        pixel_height = abs(ul_t[1] - ll_t[1])
        pixel_area = pixel_width * pixel_height

        return pixel_width, pixel_height, pixel_area

def remove_nodata(data, no_data):
    """
    移除数据中的 NoData 值、0 值以及 NaN 值。
    """
    if no_data is not None:
        mask_arr = (data == no_data) | (data == 0) | np.isnan(data)
    else:
        mask_arr = (data == 0) | np.isnan(data)
    return data[~mask_arr]

def zonal_statistics(raster_path, vector_path, output_path):
    """
    对每个分区（矢量多边形）进行栅格掩模和统计：
    1. 计算目标投影下的像元尺寸（以便后续计算像元面积）。
    2. 针对每个分区，利用 rasterio.mask.mask 提取该区域栅格数据，
       并计算有效像元个数、乘以像元面积和建筑覆盖率后得到建筑面积。
    3. 结果合并到矢量数据中，并保存输出。
    """
    try:
        # 打开栅格文件，获取 NoData 值和 CRS
        with rasterio.open(raster_path) as raster:
            no_data = raster.nodata
            raster_crs = raster.crs

        # 计算像元尺寸（目标投影下的尺寸，单位：米）
        pixel_size_x, pixel_size_y, pixel_area = calculate_pixel_size_in_meters(raster_path)
        print("Pixel width: {:.3f} m, Pixel height: {:.3f} m, Pixel area: {:.3f} m^2".format(pixel_size_x, pixel_size_y, pixel_area))

        # 读取矢量数据
        vector = gpd.read_file(vector_path)
        # 如果栅格不存在，则给出缺省值后退出
        if not os.path.exists(raster_path):
            print("Raster file {} not found. Setting ImpSurf, ISF to NoData.".format(raster_path))
            vector["ImpSurf"] = -9999
            vector["ISF"] = -9999

            vector.to_file(output_path)
            return

        # 保证矢量数据和栅格数据在相同 CRS 下
        if vector.crs != raster_crs:
            vector = vector.to_crs(raster_crs)

        # 初始化进度回调
        show_progress = create_show_progress(threshold=10)
        total_zones = vector.shape[0]
        results = []

        # 对每个分区（矢量多边形）进行处理
        for idx, row in vector.iterrows():
            geom = row['geometry']
            #print("Processing feature {}/{}".format(idx+1, total_zones))

            # 裁出一个小片栅格（掩模），默认只考虑像元中心落在多边形内部的像元
            with rasterio.open(raster_path) as raster:
                out_image, out_transform = mask(raster, [geom], crop=True, all_touched=False)
                data = out_image[0]
                valid_data = remove_nodata(data, no_data)

                # 如果存在有效数据，则计算建筑面积
                if valid_data.size > 0:
                    # 此处假设建筑覆盖率为有效像元平均值/100，
                    # 并以像元总面积乘以覆盖率得到建筑面积
                    count_val = valid_data.size
                    valid_building_area = count_val * pixel_area
                    area_val = valid_building_area
                    #print("Computed ImpSurf:", area_val)
                else:
                    area_val = -9999

                results.append({'ImpSurf': area_val})

            completed = (idx + 1) / float(total_zones)
            show_progress(completed, 'Processing zones', None)

        # 将统计结果合并到矢量数据中
        stats_df = pd.DataFrame(results)
        vector = vector.join(stats_df)

        # 这里CoverRatio和FAR的计算示例仅供参考，根据实际需求调整公式
        vector['ISF'] = np.where(vector['ImpSurf'] == -9999, -9999, vector['ImpSurf'] / vector['LandArea'])

        # 保存结果到输出文件
        vector.to_file(output_path)
        print("Zonal statistics saved to:", output_path)

    except Exception as e:
        print("Error calculating zonal statistics:", e)

#设置输入文件夹和输出文件路径
input_folder = r"D:\file\d_zonalst"

for place2 in os.listdir(input_folder):
    workspace = rf'D:\file\d_zonalst\{place2}'
    vector_path2 = os.path.join(workspace, f'{place2}_output_file(MEAN_BD_FAR_CD_CR_DistWB_DistGL_DistMT).shp')
    raster_path2 = rf'D:\file\d_builtup\{place2}\builtup_{place2}.tif'
    output_path2 = os.path.join(workspace, f'{place2}_output_file(+isf).shp')
    #
    # if os.path.exists(output_path2):
    #     print(rf'跳过{output_path2}:输出文件已经存在')
    #     continue
    zonal_statistics(raster_path2, vector_path2, output_path2)

15_01 计算山体的信息

In [ ]:
import geopandas as gpd
from shapely.geometry import box
import re
import os

# === 1. 定义输入输出路径 === #
shp_folder = r'D:\file\a_clip_data\c_merge'
dsm_folder = r'D:\file\d_height'
output_file = r'D:\file\match_results.txt'  # 存储匹配结果的 txt 文件

shp_files = [f for f in os.listdir(shp_folder) if f.endswith('.shp')]
dsm_files = [f for f in os.listdir(dsm_folder) if f.endswith('_DEM.tif')]

# 提取经纬度的函数
pattern = re.compile(r'Copernicus_DSM_10_([NS])(\d+)_00_([EW])(\d+)_00_DEM')

def get_bbox_from_name(name):
    """从 DSM 文件名提取边界框"""
    match = pattern.search(name)
    if match:
        ns, lat, ew, lon = match.groups()
        lat = int(lat) * (1 if ns == 'N' else -1)
        lon = int(lon) * (1 if ew == 'E' else -1)
        return name, box(lon, lat, lon + 1, lat + 1)
    return None

# 创建 DSM 文件的边界框字典
dsm_bboxes = dict(filter(None, [get_bbox_from_name(f) for f in dsm_files]))

# === 2. 处理每个 shapefile 并写入结果文件 === #
with open(output_file, 'w', encoding='utf-8') as f_out:
    for shp_name in shp_files:
        shp_path = os.path.join(shp_folder, shp_name)

        # 读取 shapefile
        gdf = gpd.read_file(shp_path)

        # 过滤空 geometry
        gdf = gdf[gdf.geometry.notna()]
        gdf = gdf[~gdf.geometry.is_empty]

        # 计算 polygon 的 bbox
        gdf_bounds = gdf.geometry.bounds
        gdf['bbox_geom'] = gdf_bounds.apply(lambda row: box(row.minx, row.miny, row.maxx, row.maxy), axis=1)

        # === 3. 匹配 polygon bbox 和 DSM bbox === #
        gdf['intersecting_dsm_files'] = gdf['bbox_geom'].apply(
            lambda poly_box: [fname for fname, dsm_box in dsm_bboxes.items() if poly_box.intersects(dsm_box)]
        )

        # === 4. 将结果写入文件 === #
        #f_out.write(f"\n=== {shp_name} 匹配结果 ===\n")
        for idx, row in gdf.iterrows():
            matched_files = ", ".join(row['intersecting_dsm_files']) if row['intersecting_dsm_files'] else "无匹配"
            #f_out.write(f"Polygon {idx} 匹配到 DSM 文件: {matched_files}\n")
            f_out.write(f"{matched_files}\n")
            print(f"{matched_files}\n")

print(f"匹配结果已保存到 {output_file}")

# 15_02

In [ ]:
file_path = r'D:\file\match_results.txt'

with open(file_path, 'r', encoding='utf-8') as f:
    line_count = sum(1 for _ in f)

print(f"文件共有 {line_count} 行")

In [1]:
import os
import rasterio
from rasterio.merge import merge

# === 1. 定义路径 === #
match_results_file = r'D:\file\match_results.txt'
dsm_folder = r'D:\file\d_height_filter'
output_base_folder = r'D:\file\d_mountain'

# 确保输出文件夹存在
os.makedirs(output_base_folder, exist_ok=True)

# === 2. 读取匹配的 DSM 文件名 === #
with open(match_results_file, 'r', encoding='utf-8') as f:
    lines = [line.strip() for line in f if line.strip()]  # 去除空行

# === 3. 逐行合并 TIFF 并存储 === #
for i, line in enumerate(lines):
    tiff_files = [os.path.join(dsm_folder, f.strip()) for f in line.split(",") if f.strip()]

    if not tiff_files:
        print(f"跳过第 {i} 组, 没有找到匹配文件")
        continue

    output_folder = os.path.join(output_base_folder, str(i))
    os.makedirs(output_folder, exist_ok=True)  # 创建对应的输出目录
    output_tiff = os.path.join(output_folder, f"mountain_{i}.tif")

    # 读取 TIFF 并合并
    src_files = []
    for file in tiff_files:
        if os.path.exists(file):  # 确保文件存在
            src = rasterio.open(file)
            src_files.append(src)
        else:
            print(f"警告: 文件 {file} 不存在，跳过")

    if not src_files:
        print(f"跳过第 {i} 组, 没有可用的 TIFF 文件")
        continue

    # 合并影像
    mosaic, out_trans = merge(src_files)

    # 复制元数据
    out_meta = src_files[0].meta.copy()
    out_meta.update({
        "driver": "GTiff",
        "height": mosaic.shape[1],
        "width": mosaic.shape[2],
        "transform": out_trans
    })

    # 写入合并后的 TIFF
    with rasterio.open(output_tiff, "w", **out_meta) as dest:
        dest.write(mosaic)

    # 关闭所有打开的 TIFF 文件
    for src in src_files:
        src.close()

    print(f"合并完成: {output_tiff}")

print("所有 TIFF 影像合并完成！")

合并完成: D:\file\d_mountain\0\mountain_0.tif
合并完成: D:\file\d_mountain\1\mountain_1.tif
合并完成: D:\file\d_mountain\2\mountain_2.tif
合并完成: D:\file\d_mountain\3\mountain_3.tif
合并完成: D:\file\d_mountain\4\mountain_4.tif
合并完成: D:\file\d_mountain\5\mountain_5.tif
合并完成: D:\file\d_mountain\6\mountain_6.tif
合并完成: D:\file\d_mountain\7\mountain_7.tif
合并完成: D:\file\d_mountain\8\mountain_8.tif
合并完成: D:\file\d_mountain\9\mountain_9.tif
合并完成: D:\file\d_mountain\10\mountain_10.tif
合并完成: D:\file\d_mountain\11\mountain_11.tif
合并完成: D:\file\d_mountain\12\mountain_12.tif
合并完成: D:\file\d_mountain\13\mountain_13.tif
合并完成: D:\file\d_mountain\14\mountain_14.tif
合并完成: D:\file\d_mountain\15\mountain_15.tif
合并完成: D:\file\d_mountain\16\mountain_16.tif
合并完成: D:\file\d_mountain\17\mountain_17.tif
合并完成: D:\file\d_mountain\18\mountain_18.tif
合并完成: D:\file\d_mountain\19\mountain_19.tif
合并完成: D:\file\d_mountain\20\mountain_20.tif
合并完成: D:\file\d_mountain\21\mountain_21.tif
合并完成: D:\file\d_mountain\22\mountain_22.tif
合并完成: D:

In [ ]:
import os
import subprocess

# 设定输入/输出文件夹
mountain_folder = r"D:\file\d_mountain"
slope_folder = mountain_folder

# 确保坡度输出目录存在
os.makedirs(slope_folder, exist_ok=True)

# 遍历所有 mountain_x.tif 文件
for place in os.listdir(mountain_folder):
    mountain_path = os.path.join(mountain_folder, place, f"mountain_{place}.tif")
    slope_output_folder = os.path.join(slope_folder, place)
    os.makedirs(slope_output_folder, exist_ok=True)  # 确保子目录存在
    slope_path = os.path.join(slope_output_folder, f"slope_{place}.tif")

    # 检查是否已存在坡度文件，避免重复计算
    if os.path.exists(slope_path):
        print(f"跳过 {slope_path}，文件已存在")
        continue

    # 调用 GDAL 计算坡度
    cmd = [
        "gdaldem", "slope", mountain_path, slope_path,
        "-s", "1.0",  # 这里假设高程单位是米，如需换算可调整 -s 参数
        "-of", "GTiff"
    ]

    try:
        subprocess.run(cmd, check=True)
        print(f"成功生成坡度: {slope_path}")
    except subprocess.CalledProcessError as e:
        print(f"错误: 计算 {slope_path} 失败 - {e}")


成功生成坡度: D:\file\d_mountain\0\slope_0.tif
成功生成坡度: D:\file\d_mountain\1\slope_1.tif
成功生成坡度: D:\file\d_mountain\10\slope_10.tif
成功生成坡度: D:\file\d_mountain\100\slope_100.tif
成功生成坡度: D:\file\d_mountain\101\slope_101.tif
成功生成坡度: D:\file\d_mountain\102\slope_102.tif
成功生成坡度: D:\file\d_mountain\103\slope_103.tif
成功生成坡度: D:\file\d_mountain\104\slope_104.tif
成功生成坡度: D:\file\d_mountain\105\slope_105.tif
成功生成坡度: D:\file\d_mountain\106\slope_106.tif
成功生成坡度: D:\file\d_mountain\107\slope_107.tif
成功生成坡度: D:\file\d_mountain\108\slope_108.tif
成功生成坡度: D:\file\d_mountain\109\slope_109.tif
成功生成坡度: D:\file\d_mountain\11\slope_11.tif
成功生成坡度: D:\file\d_mountain\110\slope_110.tif
成功生成坡度: D:\file\d_mountain\111\slope_111.tif
成功生成坡度: D:\file\d_mountain\112\slope_112.tif
成功生成坡度: D:\file\d_mountain\113\slope_113.tif
成功生成坡度: D:\file\d_mountain\114\slope_114.tif
成功生成坡度: D:\file\d_mountain\115\slope_115.tif
成功生成坡度: D:\file\d_mountain\116\slope_116.tif
成功生成坡度: D:\file\d_mountain\117\slope_117.tif
成功生成坡度: D:\file\d_moun

# 15_03 perform zonalst

In [ ]:
import os
import rasterio
from rasterio.mask import mask
import geopandas as gpd
import pandas as pd
import numpy as np
from pyproj import Transformer, CRS


def remove_nodata(data, no_data):
    """
    移除数据中的 NoData 值、0 值以及 NaN 值。
    """
    if no_data is not None:
        mask_arr = (data == no_data) | np.isnan(data)
    else:
        mask_arr = np.isnan(data)
    return data[~mask_arr]


def calculate_statistics(data):
    """
    计算统计指标：平均值、标准差、极差（最大值-最小值）
    """
    if len(data) == 0:
        return -9999, -9999, -9999  # NoData 值
    return np.mean(data), np.std(data), np.max(data) - np.min(data)


def zonal_statistics(elev_raster, slope_raster, vector_path, output_path):
    """
    计算城市范围的高程和坡度统计信息
    """
    try:
        # 读取矢量数据
        vector = gpd.read_file(vector_path)

        # 确保矢量数据的 CRS 与栅格一致
        with rasterio.open(elev_raster) as src:
            raster_crs = src.crs
            no_data_elev = src.nodata
        with rasterio.open(slope_raster) as src:
            no_data_slope = src.nodata

        if vector.crs != raster_crs:
            vector = vector.to_crs(raster_crs)

        results = []

        for idx, row in vector.iterrows():
            geom = row['geometry']
            zone_stats = {}

            # 处理高程数据
            with rasterio.open(elev_raster) as elev_src:
                out_image, _ = mask(elev_src, [geom], crop=True, all_touched=False)
                elev_data = remove_nodata(out_image[0], no_data_elev)
                zone_stats["elev_mean"], zone_stats["elev_std"], zone_stats["elev_range"] = calculate_statistics(elev_data)

            # 处理坡度数据
            with rasterio.open(slope_raster) as slope_src:
                out_image, _ = mask(slope_src, [geom], crop=True, all_touched=False)
                slope_data = remove_nodata(out_image[0], no_data_slope)
                zone_stats["slope_mean"], zone_stats["slope_std"], zone_stats["slope_range"] = calculate_statistics(slope_data)

            results.append(zone_stats)

        # 合并计算结果到矢量数据
        stats_df = pd.DataFrame(results)
        vector = vector.join(stats_df)

        # 保存输出
        vector.to_file(output_path)
        print("Zonal statistics saved to:", output_path)

    except Exception as e:
        print("Error calculating zonal statistics:", e)


# 设置输入数据
input_folder = r"D:\file\d_zonalst"

for place in os.listdir(input_folder):
    workspace = os.path.join(input_folder, place)
    vector_path = os.path.join(workspace, f"{place}_output_file(+isf).shp")
    elev_raster = os.path.join(rf"D:\file\d_mountain\{place}", f"mountain_{place}.tif")
    slope_raster = os.path.join(rf"D:\file\d_mountain\{place}", f"slope_{place}.tif")
    output_path = os.path.join(workspace, f"{place}_output_file(+isf+elev_slope).shp")

    if os.path.exists(output_path):
        print(f"跳过 {output_path}: 输出文件已存在")
        continue

    zonal_statistics(elev_raster, slope_raster, vector_path, output_path)
